In [ ]:
from transformers import AutoTokenizer, AutoModel
from transformers import pipeline
import numpy as np
import time
from concurrent.futures import ProcessPoolExecutor

In [28]:
model_name = "typeform/distilbert-base-uncased-mnli"
classifier = pipeline(
    "zero-shot-classification",
    model= model_name,
    device=-1
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Device set to use cpu
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can saf

In [38]:
def chunk_text(text, max_length=512):
    encoded_text = tokenizer.encode(text)
    chunks = [encoded_text[i:i + max_length] for i in range(0, len(encoded_text), max_length)]
    return [tokenizer.decode(chunk) for chunk in chunks]

def classify_chunk(chunk, topics):
    return classifier(chunk, topics, multi_label=True)

def check_long_text(text, topics, threshold=0.75, k=3):
    text_chunks = chunk_text(text, max_length=100)

    # Use ProcessPoolExecutor for parallelism
    with ProcessPoolExecutor() as executor:
        results = list(executor.map(classify_chunk, text_chunks, [topics] * len(text_chunks)))

    consistent_labels = sorted(topics)
    final_arr = []
    for result in results:
        label_score_map = dict(zip(result['labels'], result['scores']))
        aligned_scores = [label_score_map[label] for label in consistent_labels]
        final_arr.append(aligned_scores)

    np_arr = np.array(final_arr)
    top_k_scores = np.sort(np_arr, axis=0)[-min(k, np_arr.shape[0]):]
    zipped = zip(np.mean(top_k_scores, axis=0), consistent_labels)
    sorted_zipped = sorted(zipped, key=lambda x: x[0], reverse=True)
    arr, labels = zip(*sorted_zipped)
    
    print("Labels:", labels)
    print("Scores:", arr)
    return max(arr) >= threshold

In [ ]:
text = '''
Donald John Trump (born June 14, 1946) is an American politician, media personality, and businessman who served as the 45th president of the United States from 2017 to 2021. Having won the 2024 presidential election as the nominee of the Republican Party, he is the president-elect and will be inaugurated as the 47th president on January 20, 2025.

Trump graduated with a bachelor's degree in economics from the University of Pennsylvania in 1968. Becoming president of the family real estate business in 1971, he focused on luxury accommodation. After a series of business bankruptcies in the 1990s, he launched side ventures. From 2004 to 2015, he produced and hosted the reality television series The Apprentice.

Trump won the 2016 presidential election as the Republican nominee. His immigration policy included ordering a travel ban on several Muslim-majority countries, expanding the U.S.–Mexico border wall, and implementing a family separation policy. He rolled back more than 100 environmental policies and regulations, signed the Tax Cuts and Jobs Act of 2017, and nominated Neil Gorsuch, Brett Kavanaugh and Amy Coney Barrett to the U.S. Supreme Court. In 2018, he initiated a trade war with China. He withdrew the U.S. from international agreements on climate, trade, and the nuclear program of Iran. He met with North Korean leader Kim Jong Un without progress on denuclearization. In response to the COVID-19 pandemic, he downplayed its severity, contradicted guidance from international public health bodies, and signed the CARES Act economic stimulus. He was impeached in 2019 for abuse of power and obstruction of Congress, and in 2021 for incitement of insurrection; the Senate acquitted him in both cases.

Trump's politics and rhetoric led to the Trumpism movement. Many of his comments and actions have been characterized as racially charged, racist, and misogynistic. He promoted conspiracy theories and made false and misleading statements during his campaigns and presidency, to a degree unprecedented in American politics. After his first term, scholars and historians ranked him as one of the worst presidents in American history. He lost the 2020 presidential election but did not concede, falsely claiming widespread electoral fraud and attempting to overturn the results, including through his involvement in the January 6 Capitol attack. In civil proceedings, Trump was found liable for sexual abuse and defamation in 2023, and financial fraud in 2024. In May 2024, he was found guilty of falsifying business records, making him the first U.S. president to be convicted of a felony. He faced more felony indictments related to his interference in the 2020 election and his handling of classified documents, which were dismissed after his victory in the 2024 election.
Early life and education
A black-and-white photograph of Trump as a teenager, smiling, wearing a dark pseudo-military uniform with various badges and a light-colored stripe crossing his right shoulder
Trump at New York Military Academy, 1964

Trump was born on June 14, 1946, at Jamaica Hospital in Queens, New York City, the fourth child of Fred Trump and Mary Anne MacLeod Trump.[1] He is of German and Scottish descent.[2] He grew up with his older siblings, Maryanne, Fred Jr., and Elizabeth, and his younger brother, Robert, in a mansion in the Jamaica Estates neighborhood of Queens.[3]

Trump attended the private college-preparatory Kew-Forest School through seventh grade.[4] He was a difficult child and showed an early interest in his father's business.[5] His father enrolled him in New York Military Academy, a private boarding school, to complete secondary school;[6] he learned to excel in a strict regimen.[7]

In 1964, Trump enrolled at Fordham University. Two years later, he transferred to the Wharton School of the University of Pennsylvania,[8] graduating in May 1968 with a Bachelor of Science in economics.[9] He was exempted from the draft during the Vietnam War due to bone spurs in his heels.[10] In 2015, he threatened his high school, colleges, and the College Board with legal action if they released his academic records.[11]
Personal life
Further information: Family of Donald Trump
Family

In 1977, Trump married the Czech model Ivana Zelníčková.[12] They had three children: Donald Jr. (1977–), Ivanka (1981–), and Eric (1984–). The couple divorced in 1990, following his affair with the actress Marla Maples.[13] He and Maples married in 1993 and divorced in 1999. They have one daughter, Tiffany (1993–), whom Maples raised in California.[14] In 2005, he married the Slovenian model Melania Knauss.[15] They have one son, Barron (2006–).[16]
Health
Main article: Age and health concerns about Donald Trump

Trump says he has never drunk alcohol, smoked cigarettes, or used drugs.[17][18] He sleeps about four or five hours a night.[19][20] He has called golfing his "primary form of exercise", but usually does not walk the course.[21] He considers exercise a waste of energy because he believes the body is "like a battery, with a finite amount of energy", which is depleted by exercise.[22][23] In 2015, his campaign released a letter from his longtime personal physician, Harold Bornstein, stating that he would "be the healthiest individual ever elected to the presidency".[24] In 2018, Bornstein said Trump had dictated the contents of the letter and that three of Trump's agents had seized his medical records in a February 2017 raid on Bornstein's office.[24][25]
Views

Many of Trump's comments and actions have been described as racist.[26] In national polling, about half of respondents said that he is racist; a greater proportion believed that he emboldened racists.[27] Several studies and surveys found that racist attitudes fueled his political ascent and were more important than economic factors in determining the allegiance of Trump voters.[28] Racist and Islamophobic attitudes are a powerful indicator of support for Trump.[29] He has also been accused of racism for insisting a group of black and Latino teenagers were guilty of raping a white woman in the 1989 Central Park jogger case, even after they were exonerated in 2002 when the actual rapist confessed and his DNA matched the evidence. In October 2024, the men sued Trump for defamation after he said in a televised September debate that they had committed the crime and killed the woman.[30]

In 2011, when he was reportedly considering a presidential run, Trump became the leading proponent of the racist "birther" conspiracy theory, alleging that Barack Obama, the first black U.S. president, was not born in the U.S.[31] In April, he claimed credit for pressuring the White House to publish the "long-form" birth certificate, which he considered fraudulent, and later said this made him "very popular".[32] In September 2016, amid pressure, he acknowledged that Obama was born in the U.S.[33] In 2017, he reportedly expressed birther views privately.[34]
Answering questions about the Unite the Right rally in Charlottesville

Trump has a history of belittling women when speaking to the media and on social media.[35][36] He made lewd comments, disparaged women's physical appearances, and referred to them using derogatory epithets.[36] At least 25 women publicly accused him of sexual misconduct, including rape, kissing without consent, groping, looking under women's skirts, and walking in on naked teenage pageant contestants. He has denied the allegations.[37] In October 2016, a 2005 "hot mic" recording surfaced in which Trump bragged about kissing and groping women without their consent, saying that, "when you're a star, they let you do it. You can do anything. ... Grab 'em by the pussy."[38] Trump characterized the comments as "locker-room talk",[39][40] and the incident's widespread media exposure led to Trump's first public apology during his 2016 presidential campaign.[41]
Business career
Main article: Business career of Donald Trump
Further information: Business projects of Donald Trump in Russia and Tax returns of Donald Trump
Real estate
Trump in 1985 with a model of one of his aborted Manhattan development projects[42]

Starting in 1968, Trump was employed at his father's real estate company, Trump Management, which owned racially segregated middle-class rental housing in New York City's outer boroughs.[43][44] In 1971, his father made him president of the company and he began using the Trump Organization as an umbrella brand.[45]

Roy Cohn was Trump's fixer, lawyer, and mentor for 13 years in the 1970s and 1980s.[46] In 1973, Cohn helped Trump countersue the U.S. government for $100 million (equivalent to $686 million in 2023)[47] over its charges that Trump's properties had racial discriminatory practices. Trump's counterclaims were dismissed, and the government's case was settled with the Trumps signing a consent decree agreeing to desegregate.[48] Helping Trump projects, Cohn was a consigliere whose Mafia connections controlled construction unions.[49] Cohn introduced political consultant Roger Stone to Trump, who enlisted Stone's services to deal with the federal government.[50]

Between 1991 and 2009, he filed for Chapter 11 bankruptcy protection for six of his businesses: the Plaza Hotel in Manhattan, the casinos in Atlantic City, New Jersey, and the Trump Hotels & Casino Resorts company.[51]
Manhattan and Chicago developments

Trump attracted public attention in 1978 with the launch of his family's first Manhattan venture, the renovation of the derelict Commodore Hotel, adjacent to Grand Central Terminal.[52] The financing was facilitated by a $400 million city property tax abatement arranged for him by his father who also, jointly with Hyatt, guaranteed a $70 million bank construction loan.[44][53] The hotel reopened in 1980 as the Grand Hyatt Hotel,[54] and that same year, he obtained rights to develop Trump Tower, a mixed-use skyscraper in Midtown Manhattan.[55] The building houses the headquarters of the Trump Corporation and Trump's PAC and was his primary residence until 2019.[56]

In 1988, Trump acquired the Plaza Hotel with a loan from a consortium of sixteen banks.[57] The hotel filed for bankruptcy protection in 1992, and a reorganization plan was approved a month later, with the banks taking control of the property.[58] In 1995, he defaulted on over $3 billion of bank loans, and the lenders seized the Plaza Hotel along with most of his other properties in a "vast and humiliating restructuring" that allowed him to avoid personal bankruptcy.[59][60] The lead bank's attorney said of the banks' decision that they "all agreed that he'd be better alive than dead".[59]

In 1996, Trump acquired and renovated the mostly vacant 71-story skyscraper at 40 Wall Street, later rebranded as the Trump Building.[61] In the early 1990s, he won the right to develop a 70-acre (28 ha) tract in the Lincoln Square neighborhood near the Hudson River. Struggling with debt from other ventures in 1994, he sold most of his interest in the project to Asian investors, who financed the project's completion, Riverside South.[62]

Trump's last major construction project was the 92-story mixed-use Trump International Hotel and Tower in Chicago which opened in 2008. In 2024, the New York Times and ProPublica reported that the Internal Revenue Service was investigating whether he had twice written off losses incurred through construction cost overruns and lagging sales of residential units in the building he had declared to be worthless on his 2008 tax return.[63]
Atlantic City casinos
The entrance of the Trump Taj Mahal, a casino in Atlantic City. It has motifs evocative of the Taj Mahal in India.
Entrance of the Trump Taj Mahal in Atlantic City

In 1984, Trump opened Harrah's at Trump Plaza, a hotel and casino, with financing and management help from the Holiday Corporation.[64] It was unprofitable, and he paid Holiday $70 million in May 1986 to take sole control.[65] In 1985, he bought the unopened Atlantic City Hilton Hotel and renamed it Trump Castle.[66] Both casinos filed for Chapter 11 bankruptcy protection in 1992.[67]

Trump bought a third Atlantic City venue in 1988, the Trump Taj Mahal. It was financed with $675 million in junk bonds and completed for $1.1 billion, opening in April 1990.[64] He filed for Chapter 11 bankruptcy protection in 1991. Under the provisions of the restructuring agreement, he gave up half his initial stake and personally guaranteed future performance.[68] To reduce his $900 million of personal debt, he sold the Trump Shuttle airline; his megayacht, the Trump Princess, which had been leased to his casinos and kept docked; and other businesses.[69]

In 1995, Trump founded Trump Hotels & Casino Resorts (THCR), which assumed ownership of the Trump Plaza.[70] THCR purchased the Taj Mahal and the Trump Castle in 1996 and went bankrupt in 2004 and 2009, leaving him with 10 percent ownership.[64] He remained chairman until 2009.[71]
Clubs

In 1985, Trump acquired the Mar-a-Lago estate in Palm Beach, Florida.[72] In 1995, he converted the estate into a private club with an initiation fee and annual dues. He continued to use a wing of the house as a private residence.[73] He declared the club his primary residence in 2019.[56] The Trump Organization began building and buying golf courses in 1999.[74] It owns fourteen and manages another three Trump-branded courses worldwide.[75]
Side ventures
See also: List of things named after Donald Trump
Trump, Doug Flutie, and an unnamed official standing behind a lectern with big, round New Jersey Generals sign, with members of the press seated in the background
Trump and New Jersey Generals quarterback Doug Flutie at a 1985 press conference in Trump Tower

The Trump Organization has licensed the Trump name for consumer products and services, including foodstuffs, apparel, learning courses, and home furnishings.[76] According to The Washington Post, there are more than 50 licensing or management deals involving his name, and they have generated at least $59 million in revenue for his companies.[77] By 2018, only two consumer goods companies continued to license his name.[76]

In September 1983, Trump purchased the New Jersey Generals, a team in the United States Football League. After the 1985 season, the league folded, largely due to his attempt to move to a fall schedule (when it would have competed with the NFL for audience) and trying to force a merger with the NFL by bringing an antitrust suit.[78]

Trump and his Plaza Hotel hosted several boxing matches at the Atlantic City Convention Hall.[64][79] In 1989 and 1990, he lent his name to the Tour de Trump cycling stage race, an attempt to create an American equivalent of European races such as the Tour de France or the Giro d'Italia.[80]

From 1986 to 1988, Trump purchased significant blocks of shares in various public companies while suggesting that he intended to take over the company and then sold his shares for a profit,[81] leading some observers to think he was engaged in greenmail.[82] The New York Times found that he initially made millions of dollars in such stock transactions, but "lost most, if not all, of those gains after investors stopped taking his takeover talk seriously".[81]

In 1988, Trump purchased the Eastern Air Lines Shuttle, financing the purchase with $380 million (equivalent to $979 million in 2023)[47] in loans from a syndicate of 22 banks. He renamed the airline Trump Shuttle and operated it until 1992.[83] He defaulted on his loans in 1991, and ownership passed to the banks.[84]
A red star with a bronze outline and "Donald Trump" and a TV icon written on it in bronze, embedded in a black terrazzo sidewalk
Trump's star on the Hollywood Walk of Fame

In 1992, Trump, his siblings Maryanne, Elizabeth, and Robert, and his cousin John W. Walter, each with a 20 percent share, formed All County Building Supply & Maintenance Corp. The company had no offices and is alleged to have been a shell company for paying the vendors providing services and supplies for Trump's rental units, then billing those services and supplies to Trump Management with markups of 20–50 percent and more. The owners shared the proceeds generated by the markups. The increased costs were used to get state approval for increasing the rents of his rent-stabilized units.[85]

In 1996, Trump purchased the Miss Universe pageants, including Miss USA and Miss Teen USA.[86] Due to disagreements with CBS about scheduling, he took both pageants to NBC in 2002.[87][88] In 2007, he received a star on the Hollywood Walk of Fame for his work as producer of Miss Universe.[89] NBC and Univision dropped the pageants in June 2015 in reaction to Trump's comments about Mexican immigrants.[90]
Trump University
Main article: Trump University

In 2005, Trump co-founded Trump University, a company that sold real estate seminars for up to $35,000. After New York State authorities notified the company that its use of "university" violated state law (as it was not an academic institution), its name was changed to the Trump Entrepreneur Initiative in 2010.[91]

In 2013, the State of New York filed a $40 million civil suit against Trump University, alleging that the company made false statements and defrauded consumers. Additionally, two class actions were filed in federal court against Trump and his companies. Internal documents revealed that employees were instructed to use a hard-sell approach, and former employees testified that Trump University had defrauded or lied to its students.[92] Shortly after he won the 2016 presidential election, he agreed to pay a total of $25 million to settle the three cases.[93]
Foundation
Main article: Donald J. Trump Foundation

The Donald J. Trump Foundation was a private foundation established in 1988.[94] From 1987 to 2006, Trump gave his foundation $5.4 million which had been spent by the end of 2006. After donating a total of $65,000 in 2007–2008, he stopped donating any personal funds to the charity,[95] which received millions from other donors, including $5 million from Vince McMahon.[96] The foundation gave to health- and sports-related charities, conservative groups,[97] and charities that held events at Trump properties.[95]

In 2016, The Washington Post reported that the charity committed several potential legal and ethical violations, including alleged self-dealing and possible tax evasion.[98] Also in 2016, the New York attorney general determined the foundation to be in violation of state law, for soliciting donations without submitting to required annual external audits, and ordered it to cease its fundraising activities in New York immediately.[99] Trump's team announced in December 2016 that the foundation would be dissolved.[100]

In June 2018, the New York attorney general's office filed a civil suit against the foundation, Trump, and his adult children, seeking $2.8 million in restitution and additional penalties.[101] In December 2018, the foundation ceased operation and disbursed its assets to other charities.[102] In November 2019, a New York state judge ordered Trump to pay $2 million to a group of charities for misusing the foundation's funds, in part to finance his presidential campaign.[103]
Legal affairs and bankruptcies
Main article: Personal and business legal affairs of Donald Trump

According to a review of state and federal court files conducted by USA Today in 2018, Trump and his businesses had been involved in more than 4,000 state and federal legal actions.[104] While he has not filed for personal bankruptcy, his over-leveraged hotel and casino businesses in Atlantic City and New York filed for Chapter 11 bankruptcy protection six times between 1991 and 2009.[105] They continued to operate while the banks restructured debt and reduced his shares in the properties.[105]

During the 1980s, more than 70 banks had lent Trump $4 billion.[106] After his corporate bankruptcies of the early 1990s, most major banks, with the exception of Deutsche Bank, declined to lend to him.[107] After the January 6 Capitol attack, the bank decided not to do business with him or his company in the future.[108]
Wealth
Main article: Wealth of Donald Trump
Ivana Trump and King Fahd shake hands, with Ronald Reagan standing next to them smiling
Trump (rightmost) and wife Ivana at a 1985 state dinner for King Fahd of Saudi Arabia with President Ronald Reagan and First Lady Nancy Reagan

Trump has often said he began his career with "a small loan of a million dollars" from his father and that he had to pay it back with interest.[109] In 2024 dollars, Trump was a millionaire by age eight.[a] He borrowed at least $60 million from his father, largely failed to repay those loans, and received another $413 million (2018 dollars adjusted for inflation) from his father's company.[111][85]

Trying to get a higher ranking on the Forbes 400 list of wealthy Americans, Trump called journalist Jonathan Greenberg in 1984, pretending to be a Trump Organization official named "John Barron".[112]

Trump self-reported his net worth over a wide range: from a low of minus $900 million in 1990,[b] to a high of $10 billion in 2015.[115] In 2024 dollars according to Forbes, Trump's wealth in 2024 was made up of approximately $1.1 billion in real estate, about $1 billion in golf clubs and resorts, and $3.5 billion in stock in Trump Media & Technology Group—today his primary asset.[116] As of December 2024, Forbes listed Trump's net worth at $6.3 billion.[117]
Media career
Main article: Media career of Donald Trump
See also: Bibliography of Donald Trump

Trump has produced 19 books under his name, most written or co-written by ghostwriters.[118] His first book, The Art of the Deal (1987), was a New York Times Best Seller. While he was credited as co-author, the entire book was written by Tony Schwartz. The New Yorker said the book made Trump famous as an "emblem of the successful tycoon".[119]

Trump had cameos in many films and television shows from 1985 to 2001.[120]

Starting in the 1990s, Trump was a guest 24 times on the nationally syndicated Howard Stern Show.[121] He had his own short-form talk radio program, Trumped!, from 2004 to 2008.[122] From 2011 until 2015, he was a guest commentator on Fox & Friends.[123]

From 2004 to 2015, Trump was co-producer and host of reality shows The Apprentice and The Celebrity Apprentice. On the shows, he was a superrich and successful chief executive who eliminated contestants with the catchphrase "you're fired". The New York Times called his portrayal a "highly flattering, highly fictionalized version of Mr. Trump".[124] The shows remade his image for millions of viewers nationwide.[125] With the related licensing agreements, they earned him more than $400 million.[126]

In 2021, Trump, who had been a member of SAG-AFTRA since 1989, resigned to avoid a disciplinary hearing regarding the January 6 attack.[127] Two days later, the union permanently barred him.[128]
Early political aspirations (1987–2014)
Further information: Political career of Donald Trump
Donald Trump shakes hands with Bill Clinton in a lobby; Trump is speaking and Clinton is smiling, and both are wearing suits.
Trump and President Bill Clinton, June 2000

Trump registered as a Republican in 1987;[129] a member of the Independence Party, the New York state affiliate of the Reform Party, in 1999;[130] a Democrat in 2001; a Republican in 2009; unaffiliated in 2011; and a Republican in 2012.[129]
Trump, leaning heavily onto a lectern, with his mouth open mid-speech and a woman clapping politely next to him
Trump speaking at CPAC 2011

In 1987, Trump placed full-page advertisements in three major newspapers,[131] expressing his views on foreign policy and how to eliminate the federal budget deficit.[132] In 1988, he approached Lee Atwater, asking to be put into consideration to be Republican nominee George H. W. Bush's running mate. Bush found the request "strange and unbelievable".[133][134] Trump was a candidate in the 2000 Reform Party presidential primaries for three months, but withdrew from the race in February 2000.[135][136][137] In 2011, Trump speculated about running against President Barack Obama in the 2012 election, making his first speaking appearance at the Conservative Political Action Conference (CPAC) in February and giving speeches in early primary states.[138][139] In May, he announced he would not run.[138] His presidential ambitions were generally not taken seriously at the time.[140]
2016 presidential election
Campaign
Main article: Donald Trump 2016 presidential campaign
Further information: 2016 Republican Party presidential primaries and 2016 United States presidential election § General election campaign

Trump announced his candidacy in June 2015.[141][142] His campaign was initially not taken seriously by political analysts, but he quickly rose to the top of opinion polls.[143] He became the front-runner in March 2016[144] and was declared the presumptive Republican nominee in May.[145]

Trump's fame and provocative statements earned him an unprecedented amount of free media coverage, elevating his standing in the Republican primaries.[146] He adopted the phrase "truthful hyperbole", coined by his ghostwriter Tony Schwartz, to describe his public speaking style.[119][147] His campaign statements were often opaque and suggestive,[148] and a record number were false.[149][150][151] He said he disdained political correctness and frequently made claims of media bias.[152][153]
Trump speaking in front of an American flag behind a lectern, wearing a black suit and red hat. The lectern sports a blue "TRUMP" sign.
Trump campaigning in Arizona, March 2016

Hillary Clinton led Trump in national polling averages throughout the campaign, but, in early July, her lead narrowed.[154] In mid-July he selected Indiana governor Mike Pence as his running mate,[155] and the two were officially nominated at the 2016 Republican National Convention.[156] Trump and Clinton faced off in three presidential debates in September and October 2016. He twice refused to say whether he would accept the result of the election.[157]

Trump described NATO as "obsolete"[158][159] and espoused views that were described as noninterventionist and protectionist.[160] His campaign platform emphasized renegotiating U.S.–China relations and free trade agreements such as NAFTA, strongly enforcing immigration laws, and building a new wall along the U.S.–Mexico border. Other campaign positions included pursuing energy independence while opposing climate change regulations, modernizing services for veterans, repealing and replacing the Affordable Care Act, abolishing Common Core education standards, investing in infrastructure, simplifying the tax code while reducing taxes, and imposing tariffs on imports by companies that offshore jobs. He advocated increasing military spending and extreme vetting or banning of immigrants from Muslim-majority countries.[161] Trump's proposed immigration policies were a topic of bitter debate during the 2016 campaign. He promised to build a wall on the Mexico–U.S. border to restrict illegal movement and vowed that Mexico would pay for it.[162] He pledged to deport millions of illegal immigrants residing in the U.S.,[163] and criticized birthright citizenship for incentivizing "anchor babies".[164] According to an analysis in Political Science Quarterly, Trump made "explicitly racist appeals to whites" during his 2016 presidential campaign.[165] In particular, his campaign launch speech drew criticism for claiming Mexican immigrants were "bringing drugs, they're bringing crime, they're rapists";[166] in response, NBC fired him from Celebrity Apprentice.[167]
Financial disclosures
Further information: Tax returns of Donald Trump

Trump's FEC-required reports listed assets above $1.4 billion and outstanding debts of at least $315 million.[168][169] He did not release his tax returns, contrary to the practice of every major candidate since 1976 and his promises in 2014 and 2015 to do so if he ran for office.[170][171] He said his tax returns were being audited, and that his lawyers had advised him against releasing them.[172] After a lengthy court battle to block release of his tax returns and other records to the Manhattan district attorney for a criminal investigation, including two appeals by Trump to the U.S. Supreme Court, in February 2021 the high court allowed the records to be released to the prosecutor for review by a grand jury.[173][174]

In October 2016, portions of Trump's state filings for 1995 were leaked to a reporter from The New York Times. They show that he had declared a loss of $916 million that year, which could have let him avoid taxes for up to 18 years.[175]
Results
Main article: 2016 United States presidential election

On November 8, 2016, Trump received 306 pledged electoral votes versus 232 for Clinton, although, after elector defections on both sides, the official count was ultimately 304 to 227.[176] The fifth person to be elected president while losing the popular vote,[c] he received nearly 2.9 million fewer votes than Clinton.[177] He was the only president who neither served in the military nor held any government office prior to becoming president.[178]
Pennsylvania Ave., completely packed with protesters, mostly women, many wearing pink and holding signs with progressive feminist slogans
Women's March in Washington, D.C., on January 21, 2017

Trump won 30 states, including Michigan, Pennsylvania, and Wisconsin, states which had been considered a blue wall of Democratic strongholds since the 1990s. Clinton won 20 states and the District of Columbia. His victory marked the return of an undivided Republican government—a Republican president combined with Republican control of both chambers of Congress.[179]

In late 2016 and late 2024, Time named Trump its Person of the Year.[180]

Trump's election victory sparked protests in major U.S. cities.[181][182] On the day after his inauguration, an estimated 2.6 million people worldwide, including an estimated half million in Washington, D.C., protested against him in the Women's Marches.[183]
First presidency (2017–2021)
Main article: First presidency of Donald Trump
For a chronological guide, see Timeline of the Donald Trump presidencies.
Early actions
See also: First presidential transition of Donald Trump and First 100 days of the first Donald Trump presidency
Trump, with his family watching, raises his right hand and places his left hand on the Bible as he takes the oath of office. Roberts stands opposite him administering the oath.
Trump takes the oath of office administered by Chief Justice John G. Roberts Jr. at the Capitol, January 20, 2017.

Trump was inaugurated on January 20, 2017. During his first week in office, he signed six executive orders, authorizing interim procedures in anticipation of repealing the Patient Protection and Affordable Care Act ("Obamacare"), withdrawal from the Trans-Pacific Partnership negotiations, reinstatement of the Mexico City policy, advancement of the Keystone XL and Dakota Access Pipeline construction projects, reinforcement of border security, and a planning and design process to construct a wall along the U.S. border with Mexico.[184]

Trump's daughter Ivanka and son-in-law Jared Kushner became his assistant and senior advisor, respectively.[185][186]
Conflicts of interest
See also: First presidency of Donald Trump § Ethics

Before being inaugurated, Trump moved his businesses into a revocable trust,[187][188] rather than a blind trust or equivalent arrangement "to cleanly sever himself from his business interests".[189] He continued to profit from his businesses and to know how his administration's policies affected his businesses.[188][190] Although he said he would eschew "new foreign deals", the Trump Organization pursued expansions of its operations in Scotland, Dubai, and the Dominican Republic.[188][190] Lobbyists, foreign government officials, and Trump donors and allies generated hundreds of millions of dollars for his resorts and hotels.[191]

Trump was sued for violating the Domestic and Foreign Emoluments Clauses of the U.S. Constitution, marking the first time that the clauses had been substantively litigated.[192] One case was dismissed in lower court.[193] Two were dismissed by the U.S. Supreme Court as moot after the end of his term.[194]
Domestic policy
Main articles: Economic policy of the first Donald Trump administration, Environmental policy of the first Donald Trump administration, and Social policy of Donald Trump

Trump took office at the height of the longest economic expansion in American history,[195] which began in 2009 and continued until February 2020, when the COVID-19 recession began.[196]

In December 2017, Trump signed the Tax Cuts and Jobs Act of 2017. It reduced tax rates for businesses and individuals and set the penalty associated with the Affordable Care Act's individual mandate to $0.[197][198] The Trump administration claimed that the act would not decrease government revenue, but 2018 revenues were 7.6 percent lower than projected.[199]

Under Trump, the federal budget deficit increased by almost 50 percent, to nearly $1 trillion in 2019.[200] By the end of his term, the U.S. national debt increased by 39 percent, reaching $27.75 trillion, and the U.S. debt-to-GDP ratio hit a post-World War II high.[201] Trump also failed to deliver the $1 trillion infrastructure spending plan on which he had campaigned.[202]

Trump is the only modern U.S. president to leave office with a smaller workforce than when he took office, by 3 million people.[195][203]

Trump rejects the scientific consensus on climate change.[204][205][206][207] He reduced the budget for renewable energy research by 40 percent and reversed Obama-era policies directed at curbing climate change.[208] He withdrew from the Paris Agreement, making the U.S. the only nation to not ratify it.[209]

Trump aimed to boost the production and exports of fossil fuels.[210][211] Natural gas expanded under Trump, but coal continued to decline.[212][213] He rolled back more than 100 federal environmental regulations, including those that curbed greenhouse gas emissions, air and water pollution, and the use of toxic substances. He weakened protections for animals and environmental standards for federal infrastructure projects, and expanded permitted areas for drilling and resource extraction, such as allowing drilling in the Arctic Refuge.[214]

In 2017, Trump signed Executive Order 13771, which directed that, for every new regulation, federal agencies "identify" two existing regulations for elimination, although it did not require elimination.[215] He dismantled many federal regulations on health,[216][217] labor,[218][217] and the environment,[219][217] among others, including a bill that made it easier for severely mentally ill persons to buy guns.[220] During his first six weeks in office, he delayed, suspended, or reversed ninety federal regulations,[221] often "after requests by the regulated industries".[222] The Institute for Policy Integrity found that 78 percent of his proposals were blocked by courts or did not prevail over litigation.[223]

During his campaign, Trump vowed to repeal and replace the Affordable Care Act.[224] In office, he scaled back the Act's implementation through executive orders.[225][226] He expressed a desire to "let Obamacare fail"; his administration halved the enrollment period and drastically reduced funding for enrollment promotion.[227][228] In June 2018, the Trump administration joined 18 Republican-led states in arguing before the Supreme Court that the elimination of the financial penalties associated with the individual mandate had rendered the Act unconstitutional.[229][230] Their pleading would have eliminated health insurance coverage for up to 23 million Americans, but was unsuccessful.[229] During the 2016 campaign, Trump promised to protect funding for Medicare and other social safety-net programs. In January 2020, he expressed willingness to consider cuts to them.[231]

In response to the opioid epidemic, Trump signed legislation in 2018 to increase funding for drug treatments, but was widely criticized for failing to make a concrete strategy. U.S. opioid overdose deaths declined slightly in 2018, but surged to a record 50,052 in 2019.[232]

Trump barred organizations that provide abortions or abortion referrals from receiving federal funds.[233] He said he supported "traditional marriage", but considered the nationwide legality of same-sex marriage "settled".[234] His administration rolled back key components of the Obama administration's workplace protections against discrimination of LGBTQ people.[235] His attempted rollback of anti-discrimination protections for transgender patients in August 2020 was halted by a federal judge after a Supreme Court ruling extended employees' civil rights protections to gender identity and sexual orientation.[236]

Trump has said he is opposed to gun control, although his views have shifted over time.[237] After several mass shootings during his term, he said he would propose legislation related to guns, but he abandoned that effort in November 2019.[238] His administration took an anti-marijuana position, revoking Obama-era policies that provided protections for states that legalized marijuana.[239]

Trump is a long-time advocate of capital punishment.[240][241] Under his administration, the federal government executed 13 prisoners, more than in the previous 56 years combined, ending a 17-year moratorium.[242] In 2016, he said he supported the use of interrogation torture methods such as waterboarding.[243][244]
Race relations

Trump's comments on the 2017 Unite the Right rally, condemning "this egregious display of hatred, bigotry and violence on many sides" and stating that there were "very fine people on both sides", were criticized as implying a moral equivalence between the white supremacist demonstrators and the counter-protesters.[245]

In a January 2018 discussion of immigration legislation, Trump reportedly referred to El Salvador, Haiti, Honduras, and African nations as "shithole countries".[246] His remarks were condemned as racist.[247]

In July 2019, Trump tweeted that four Democratic congresswomen—all minorities, three of whom are native-born Americans—should "go back" to the countries they "came from".[248] Two days later the House of Representatives voted 240–187, mostly along party lines, to condemn his "racist comments".[249] White nationalist publications and social media praised his remarks, which continued over the following days.[250] He continued to make similar remarks during his 2020 campaign.[251]

In June 2020, during the George Floyd protests, federal law-enforcement officials controversially removed a largely peaceful crowd of lawful protesters from Lafayette Square, outside the White House.[252][253] Trump then posed with a Bible for a photo-op at the nearby St. John's Episcopal Church,[252][254][255] with religious leaders condemning both the treatment of protesters and the photo opportunity itself.[256] Many retired military leaders and defense officials condemned his proposal to use the U.S. military against anti-police-brutality protesters.[257]
Pardons and commutations
Further information: List of people granted executive clemency by Donald Trump

Trump granted 237 requests for clemency, fewer than all presidents since 1900 with the exception of George H. W. Bush and George W. Bush.[258] Only 25 of them had been vetted by the Justice Department's Office of the Pardon Attorney; the others were granted to people with personal or political connections to him, his family, and his allies, or recommended by celebrities.[259][260] In his last full day in office, he granted 73 pardons and commuted 70 sentences.[261] Several Trump allies were not eligible for pardons under Justice Department rules, and in other cases the department had opposed clemency.[259] The pardons of three military service members convicted of or charged with violent crimes were opposed by military leaders.[262]
Immigration
Main article: Immigration policy of Donald Trump

As president, he frequently described illegal immigration as an "invasion" and conflated immigrants with the criminal gang MS-13.[263] Trump drastically escalated immigration enforcement, including implementing harsher immigration enforcement policies against asylum seekers from Central America than any modern U.S. president.[264][265]

From 2018 onward, Trump deployed nearly 6,000 troops to the U.S.–Mexico border[266] to stop most Central American migrants from seeking asylum. In 2020, his administration widened the public charge rule to further restrict immigrants who might use government benefits from getting permanent residency.[267] He reduced the number of refugees admitted to record lows. When he took office, the annual limit was 110,000; he set a limit of 18,000 in the 2020 fiscal year and 15,000 in the 2021 fiscal year.[268][269] Additional restrictions implemented by the Trump administration caused significant bottlenecks in processing refugee applications, resulting in fewer refugees accepted than the allowed limits.[270]
Travel ban
Main article: Trump travel ban
Further information: Executive Order 13769 and Executive Order 13780

Following the 2015 San Bernardino attack, Trump proposed to ban Muslim foreigners from entering the U.S. until stronger vetting systems could be implemented.[271] He later reframed the proposed ban to apply to countries with a "proven history of terrorism".[272]

On January 27, 2017, Trump signed Executive Order 13769, which suspended admission of refugees for 120 days and denied entry to citizens of Iraq, Iran, Libya, Somalia, Sudan, Syria, and Yemen for 90 days, citing security concerns. The order took effect immediately and without warning, causing chaos at airports.[273][274] Protests began at airports the next day,[273][274] and legal challenges resulted in nationwide preliminary injunctions.[275] A March 6 revised order, which excluded Iraq and gave other exemptions, again was blocked by federal judges in three states.[276][277] In a decision in June 2017, the Supreme Court ruled that the ban could be enforced on visitors who lack a "credible claim of a bona fide relationship with a person or entity in the United States".[278]

The temporary order was replaced by Presidential Proclamation 9645 on September 24, 2017, which restricted travel from the originally targeted countries except Iraq and Sudan, and further banned travelers from North Korea and Chad, along with certain Venezuelan officials.[279] After lower courts partially blocked the new restrictions, the Supreme Court allowed the September version to go into full effect on December 4, 2017,[280] and ultimately upheld the travel ban in a ruling in June 2019.[281]
Family separation at the border
Main article: Trump administration family separation policy
Children sitting within a wire mesh compartment
Children and juveniles in a wire mesh compartment, showing sleeping mats and thermal blankets on floor
Children sitting within a wire mesh compartment in the Ursula detention facility in McAllen, Texas, June 2018

The Trump administration separated more than 5,400 children of migrant families from their parents at the U.S.–Mexico border, a sharp increase in the number of family separations at the border starting from the summer of 2017.[282][283] In April 2018, the administration announced a "zero tolerance" policy whereby adults suspected of illegal entry were to be detained and criminally prosecuted while their children were taken away as unaccompanied alien minors.[284][285] The policy was unprecedented in previous administrations and sparked public outrage.[286][287] Trump falsely asserted that his administration was merely following the law, blaming Democrats, despite the separations being his administration's policy.[288][289]

Although Trump originally argued that the separations could not be stopped by an executive order, he acceded to intense public objection and signed an executive order in June 2018, mandating that migrant families be detained together unless "there is a concern" of a risk to the child.[290][291] On June 26, 2018, Judge Dana Sabraw concluded that the Trump administration had "no system in place to keep track of" the separated children, nor any effective measures for family communication and reunification;[292] Sabraw ordered for the families to be reunited and family separations stopped except in limited circumstances.[293] After the order, the administration separated more than a thousand migrant children from their families; the ACLU contended that the administration had abused its discretion and asked Sabraw to more narrowly define the circumstances warranting separation.[283]
Trump wall and government shutdown
Main articles: Trump wall and 2018–2019 United States federal government shutdown
Trump speaks with U.S. Border Patrol agents. Behind him are black SUVs, four short border wall prototype designs, and the current border wall in the background
Trump examines border wall prototypes in Otay Mesa, California.

One of Trump's central campaign promises was to build a 1,000-mile (1,600 km) border wall to Mexico and have Mexico pay for it.[294] By the end of his term, the U.S. had built "40 miles [64 km] of new primary wall and 33 miles [53 km] of secondary wall" in locations where there had been no barriers and 365 miles (587 km) of primary or secondary border fencing replacing dilapidated or outdated barriers.[295]

In 2018, Trump refused to sign any appropriations bill from Congress unless it allocated $5.6 billion for the border wall,[296] resulting in the federal government partially shutting down for 35 days from December 2018 to January 2019, the longest U.S. government shutdown in history.[297][298] Around 800,000 government employees were furloughed or worked without pay.[299] Trump and Congress ended the shutdown by approving temporary funding that provided delayed payments to government workers, but no funds for the wall.[297] The shutdown resulted in an estimated permanent loss of $3 billion to the economy, according to the Congressional Budget Office.[300] About half of those polled blamed Trump for the shutdown, and his approval ratings dropped.[301]

To prevent another imminent shutdown in February 2019, Congress passed and Trump signed a funding bill that included $1.375 billion for 55 miles (89 km) of bollard border fencing.[302] He also declared a national emergency on the southern border, intending to divert $6.1 billion of funds Congress had allocated to other purposes.[302] He vetoed a joint resolution to overturn the declaration, and the Senate voted against a veto override.[303] Legal challenges to the diversion of $2.5 billion originally meant for the Department of Defense's drug interdiction efforts[304][305] and $3.6 billion originally meant for military construction[306][307] were unsuccessful.
Foreign policy
Main article: Foreign policy of the first Donald Trump administration
See also: List of international presidential trips made by Donald Trump § First presidency (2017–2021)
Trump and other G7 leaders sit at a conference table
Trump with the other G7 leaders at the 45th summit in France, 2019

Trump described himself as a "nationalist"[308] and his foreign policy as "America First".[309] He praised and supported populist, neo-nationalist, and authoritarian governments.[310] Unpredictability, uncertainty, and inconsistency characterized foreign relations during his tenure.[309][311] Tensions between the U.S. and its European allies were strained under Trump.[312] He criticized NATO allies and privately suggested on multiple occasions that the U.S. should withdraw from NATO.[313][314]
Trade
See also: Trump tariffs

Trump withdrew the U.S. from the Trans-Pacific Partnership (TPP) negotiations,[315] imposed tariffs on steel and aluminum imports,[316] and launched a trade war with China by sharply increasing tariffs on 818 categories (worth $50 billion) of Chinese goods imported into the U.S.[317] While he said that import tariffs are paid by China into the U.S. Treasury, they are paid by American companies that import goods from China.[318] Although he pledged during the campaign to significantly reduce the U.S.'s trade deficits, they skyrocketed under Trump.[319] Following a 2017–2018 renegotiation, the United States–Mexico–Canada Agreement (USMCA) became effective in July 2020 as the successor to NAFTA.[320]
Russia
See also: Russia–United States relations § First Trump administration (2017–2021)
Trump and Putin, both seated, lean over and shake hands
Vladimir Putin and Trump shaking hands at the G20 Osaka summit, June 2019

The Trump administration weakened the toughest sanctions imposed by the U.S. after Russia's 2014 annexation of Crimea.[321][322] Trump withdrew the U.S. from the Intermediate-Range Nuclear Forces Treaty, citing alleged Russian noncompliance,[323] and supported a potential return of Russia to the G7.[324]

Trump repeatedly praised and, according to some critics, rarely criticized Russian president Vladimir Putin[325][326] but opposed some actions of the Russian government.[327][328] After he met Putin at the Helsinki Summit in 2018, he drew bipartisan criticism for accepting Putin's denial of Russian interference in the 2016 presidential election rather than the findings of U.S. intelligence agencies.[329][330][331]
East Asia
China, Hong Kong, Taiwan
See also: China–United States relations § First Trump administration (2017–2021)
Donald Trump and Xi Jinping stand next to each other, both smiling and wearing suits
Trump and Chinese leader Xi Jinping at the G20 Buenos Aires summit, December 2018

Trump repeatedly accused China of taking unfair advantage of the U.S.[332] He launched a trade war against China that was widely characterized as a failure,[333][334] sanctioned Huawei for alleged ties to Iran,[335] significantly increased visa restrictions on Chinese students and scholars,[336] and classified China as a currency manipulator.[337] He juxtaposed verbal attacks on China with praise of Chinese Communist Party leader Xi Jinping,[338] which was attributed to trade war negotiations.[339] After initially praising China's handling of COVID-19, he began a campaign of criticism in March 2020.[340]

Trump said he resisted punishing China for its human rights abuses against ethnic minorities in Xinjiang for fear of jeopardizing trade negotiations.[341] In July 2020, his administration imposed sanctions and visa restrictions against senior Chinese officials, in response to expanded mass detention camps holding more than a million of the country's Uyghur minority.[342]
North Korea
See also: North Korea–United States relations § First Trump administration (2017–2021), 2017–2018 North Korea crisis, and 2018–19 Korean peace process
Trump and Kim shake hands on a stage with U.S. and North Korean flags in the background
Trump and North Korean leader Kim Jong Un at the Singapore summit, June 2018

In 2017, when North Korea's nuclear weapons were increasingly seen as a serious threat,[343] Trump escalated his rhetoric, warning that North Korean aggression would be met with "fire and fury like the world has never seen".[344][345] In 2017, he declared that he wanted North Korea's "complete denuclearization", and engaged in name-calling with leader Kim Jong Un.[344][346] After this period of tension, he and Kim exchanged at least 27 letters in which the two men described a warm personal friendship.[347][348] In March 2019, he lifted some U.S. sanctions against North Korea against the advice of his Treasury Department.[349]

Trump, the first sitting U.S. president to meet a North Korean leader, met Kim three times: in Singapore in 2018, in Hanoi in 2019, and in the Korean Demilitarized Zone in 2019.[350] However, no denuclearization agreement was reached,[351] and talks in October 2019 broke down after one day.[352] While conducting no nuclear tests since 2017, North Korea continued to build up its arsenal of nuclear bombs and ballistic missiles.[353][354]
Middle East
Afghanistan
U.S. and Taliban officials stand spaced apart in a formal room
U.S. Secretary of State Mike Pompeo meeting with Taliban delegation in Qatar in September 2020

U.S. troop numbers in Afghanistan increased from 8,500 in January 2017 to 14,000 a year later,[355] reversing Trump's preelection position critical of further involvement in Afghanistan.[356] In February 2020, his administration signed the United States–Taliban deal, which called for the withdrawal of foreign troops in 14 months "contingent on a guarantee from the Taliban that Afghan soil will not be used by terrorists with aims to attack the United States or its allies" and for the U.S. to seek the release of 5,000 Taliban imprisoned by the Afghan government.[357][358][359] By the end of his term, 5,000 Taliban had been released, and, despite the Taliban continuing attacks on Afghan forces and integrating Al-Qaeda members into its leadership, U.S. troops had been reduced to 2,500.[359]
Israel

Trump supported many of the policies of Israeli Prime Minister Benjamin Netanyahu.[360] Under Trump, the U.S. recognized Jerusalem as the capital of Israel[361] and Israeli sovereignty over the Golan Heights,[362] leading to international condemnation including from the UN General Assembly, European Union, and Arab League.[363][364] In 2020, the White House hosted the signing of the Abraham Accords between Israel and the United Arab Emirates and Bahrain to normalize their foreign relations.[365]
Saudi Arabia
Trump, King Salman of Saudi Arabia, and Abdel Fattah el-Sisi place their hands on a glowing white orb light at waist level
Trump, King Salman of Saudi Arabia, and Egyptian president Abdel Fattah el-Sisi at the 2017 Riyadh summit in Saudi Arabia

Trump actively supported the Saudi Arabian–led intervention in Yemen against the Houthis and in 2017 signed a $110 billion agreement to sell arms to Saudi Arabia.[366] In 2018, the U.S. provided limited intelligence and logistical support for the intervention.[367][368] Following the 2019 attack on Saudi oil facilities, which the U.S. and Saudi Arabia blamed on Iran, he approved the deployment of 3,000 additional U.S. troops, including two Patriot batteries and a Terminal High Altitude Area Defense system, to Saudi Arabia and the United Arab Emirates.[369]
Syria
Trump and Turkish president Recep Tayyip Erdoğan at the White House in May 2017

Trump ordered missile strikes in 2017 and 2018 against the Assad regime in Syria, in retaliation for the Khan Shaykhun and Douma chemical attacks, respectively.[370][371] In December 2018, he declared "we have won against ISIS", contradicting Department of Defense assessments, and ordered the withdrawal of troops from Syria.[372][373] Mattis resigned in protest, calling Trump's decision an abandonment of the U.S.'s Kurdish allies who played a key role in fighting ISIS.[374]

In 2019, after Trump spoke to Turkish president Recep Tayyip Erdoğan, U.S. troops in northern Syria were withdrawn from the area and Turkey invaded northern Syria, attacking and displacing American-allied Kurds.[375] The U.S. House of Representatives voted 354–60 to condemn Trump's withdrawal from northern Syria.[376][377]
Iran

In May 2018, Trump withdrew the U.S. from the Joint Comprehensive Plan of Action, the 2015 agreement that lifted most economic sanctions against Iran in return for restrictions on Iran's nuclear program.[378][379] In August 2020, his administration unsuccessfully attempted to use the nuclear deal to have the UN reimpose sanctions against Iran.[380] Analysts determined that, after the U.S. withdrawal, Iran moved closer to developing a nuclear weapon.[381][382]

On January 1, 2020, Trump ordered the assassination of Qasem Soleimani, who had planned nearly every significant Iranian and Iranian-backed operation over the preceding two decades.[383][384] Iran retaliated with missile strikes against two U.S. airbases in Iraq. Dozens of soldiers sustained traumatic brain injuries. Trump downplayed their injuries, and they were initially denied Purple Heart medals and the associated benefits.[385][381]
Personnel
Main articles: Political appointments of the first Trump administration and First cabinet of Donald Trump

The Trump administration had a high turnover of personnel, particularly among White House staff. By the end of his first year in office, 34 percent of his original staff had resigned, been fired, or been reassigned.[386] As of early July 2018, 61 percent of his senior aides had left[387] and 141 staffers had left in the previous year.[388] Both figures set a record for recent presidents—more change in the first 13 months than his four immediate predecessors saw in their first two years.[389] Notable early departures included National Security Advisor Michael Flynn (after just 25 days), and Press Secretary Sean Spicer.[389] Close personal aides to Trump including Bannon, Hope Hicks, John McEntee, and Keith Schiller quit or were forced out.[390] Some later returned in different posts.[391] He publicly disparaged several of his former top officials, calling them incompetent, stupid, or crazy.[392]

Trump had four White House chiefs of staff, marginalizing or pushing out several.[393] Reince Priebus was replaced after seven months by retired Marine general John F. Kelly.[394] Kelly resigned in December 2018 after a tumultuous tenure in which his influence waned, and Trump subsequently disparaged him.[395] Kelly was succeeded by Mick Mulvaney as acting chief of staff; he was replaced in March 2020 by Mark Meadows.[393]

On May 9, 2017, Trump dismissed FBI director James Comey. While initially attributing this action to Comey's conduct in the investigation about Hillary Clinton's emails, Trump said a few days later that he was concerned with Comey's role in the ongoing Trump-Russia investigations, and that he had intended to fire Comey earlier.[396] At a private conversation in February, he said he hoped Comey would drop the investigation into Flynn.[397] In March and April, he asked Comey to "lift the cloud impairing his ability to act" by saying publicly that the FBI was not investigating him.[397][398]

Trump lost three of his 15 original cabinet members within his first year.[399] Health and Human Services secretary Tom Price was forced to resign in September 2017 due to excessive use of private charter jets and military aircraft.[399][390] Environmental Protection Agency administrator Scott Pruitt resigned in 2018 and Secretary of the Interior Ryan Zinke in January 2019 amid multiple investigations into their conduct.[400][401]

Trump was slow to appoint second-tier officials in the executive branch, saying many of the positions are unnecessary. In October 2017, there were still hundreds of sub-cabinet positions without a nominee.[402] By January 8, 2019, of 706 key positions, 433 had been filled (61 percent) and he had no nominee for 264 (37 percent).[403]
Judiciary
Further information: List of federal judges appointed by Donald Trump and Donald Trump judicial appointment controversies
Donald Trump and Amy Coney Barrett walk side by side along the West Wing Colonnade; American flags hang between the columns to their right
Trump and his third Supreme Court nominee, Amy Coney Barrett

Trump appointed 226 Article III judges, including 54 to the courts of appeals and three to the Supreme Court: Neil Gorsuch, Brett Kavanaugh, and Amy Coney Barrett.[404] His Supreme Court nominees were noted as having politically shifted the Court to the right.[405][406][407] In the 2016 campaign, he pledged that Roe v. Wade would be overturned "automatically" if he were elected and provided the opportunity to appoint two or three anti-abortion justices. He later took credit when Roe was overturned in Dobbs v. Jackson Women's Health Organization; all three of his Supreme Court nominees voted with the majority.[408][409]

Trump disparaged courts and judges he disagreed with, often in personal terms, and questioned the judiciary's constitutional authority. His attacks on the courts drew rebukes from observers, including sitting federal judges, concerned about the effect of his statements on the judicial independence and public confidence in the judiciary.[410][411]
COVID-19 pandemic
Main article: COVID-19 pandemic in the United States
Further information: U.S. federal government response to the COVID-19 pandemic and Communication of the Trump administration during the COVID-19 pandemic
See also: Economic impact of the COVID-19 pandemic in the United States
Initial response

The first confirmed case of COVID-19 in the U.S. was reported on January 20, 2020.[412] The outbreak was officially declared a public health emergency by Health and Human Services (HHS) Secretary Alex Azar on January 31, 2020.[413] Trump initially ignored persistent public health warnings and calls for action from health officials within his administration and Secretary Azar.[414] Throughout January and February he focused on economic and political considerations of the outbreak.[415] In February 2020 he publicly asserted that the outbreak in the U.S. was less deadly than influenza, was "very much under control", and would soon be over.[416] On March 19, he privately told Bob Woodward that he was deliberately "playing it down, because I don't want to create a panic".[417]

By mid-March, most global financial markets had severely contracted in response to the pandemic.[418] On March 6, Trump signed the Coronavirus Preparedness and Response Supplemental Appropriations Act, which provided $8.3 billion in emergency funding for federal agencies.[419] On March 11, the World Health Organization (WHO) recognized COVID-19 as a pandemic,[420] and he announced partial travel restrictions for most of Europe, effective March 13.[421] That same day, he gave his first serious assessment of the virus in a nationwide Oval Office address, calling the outbreak "horrible" but "a temporary moment" and saying there was no financial crisis.[422] On March 13, he declared a national emergency, freeing up federal resources.[423] He claimed that "anybody that wants a test can get a test", despite test availability being severely limited.[424] On March 27, he signed the CARES Act—a $2.2 trillion economic stimulus bill—into law following bipartisan negotiations in Congress, becoming the largest stimulus in U.S. history.[425][426]

On April 22, Trump signed an executive order restricting some forms of immigration.[427] In late spring and early summer, with infections and deaths continuing to rise, he adopted a strategy of blaming the states rather than accepting that his initial assessments of the pandemic were overly optimistic or his failure to provide presidential leadership.[428]
White House Coronavirus Task Force
Trump speaks in the West Wing briefing room with various officials standing behind him, all in formal attire and without face masks
Trump conducts a COVID-19 press briefing with members of the White House Coronavirus Task Force on March 15, 2020.

Trump established the White House Coronavirus Task Force on January 29.[429] Beginning in mid-March, he held a daily task force press conference, joined by medical experts and other administration officials,[430] sometimes disagreeing with them by promoting unproven treatments.[431] On March 16, he acknowledged for the first time that the pandemic was not under control and that months of disruption to daily lives and a recession might occur.[432] His repeated use of "Chinese virus" and "China virus" to describe COVID-19 drew criticism from health experts.[433][434]

By early April, as the pandemic worsened and amid criticism of his administration's response, Trump refused to admit any mistakes in his handling of the outbreak, instead blaming the media, Democratic state governors, the previous administration, China, and the WHO.[435] The daily coronavirus task force briefings ended in late April, after a briefing at which he suggested the dangerous idea of injecting a disinfectant to treat COVID-19;[436] the comment was widely condemned by medical professionals.[437][438]

In early May, Trump proposed the phase-out of the coronavirus task force and its replacement with another group centered on reopening the economy. Amid a backlash, he said the task force would "indefinitely" continue.[439] By the end of May, the coronavirus task force's meetings were sharply reduced.[440]
World Health Organization

Prior to the pandemic, Trump criticized the WHO and other international bodies, which he asserted were taking advantage of U.S. aid.[441] His administration's proposed 2021 federal budget, released in February, proposed reducing WHO funding by more than half.[441] In May and April, he accused the WHO of "severely mismanaging" COVID-19, alleged without evidence that the organization was under Chinese control and had enabled the Chinese government's concealment of the pandemic's origins,[441][442][443] and announced that he was withdrawing funding for the organization.[441] These were seen as attempts to distract from his own mishandling of the pandemic.[441][444][445] In July 2020, he announced the formal withdrawal of the U.S. from the WHO, effective July 2021.[442][443] The decision was widely condemned by health and government officials as "short-sighted", "senseless", and "dangerous".[442][443]
Pressure to abandon pandemic mitigation measures
Further information: COVID-19 testing in the United States

In April 2020, Republican-connected groups organized anti-lockdown protests against the measures state governments were taking to combat the pandemic;[446][447] Trump encouraged the protests on Twitter,[448] although the targeted states did not meet his administration's guidelines for reopening.[449] In April 2020, he first supported, then later criticized, Georgia Governor Brian Kemp's plan to reopen some nonessential businesses.[450] Throughout the spring he increasingly pushed for ending the restrictions to reverse the damage to the country's economy.[451] He often refused to mask at public events, contrary to his administration's April 2020 guidance to wear masks in public[452] and despite nearly unanimous medical consensus that masks are important to preventing spread of the virus.[453] His contradiction of medical recommendations weakened national efforts to mitigate the pandemic.[452][453]

In June and July, Trump said several times that the U.S. would have fewer cases of coronavirus if it did less testing, that having a large number of reported cases "makes us look bad".[454][455] The CDC guideline at the time was that any person exposed to the virus should be "quickly identified and tested" even if they are not showing symptoms, because asymptomatic people can still spread the virus.[456][457] In August 2020 the CDC quietly lowered its recommendation for testing, advising that people who have been exposed to the virus, but are not showing symptoms, "do not necessarily need a test". The change in guidelines was made by HHS political appointees under Trump administration pressure, against the wishes of CDC scientists.[458][459] The day after this political interference was reported, the testing guideline was changed back to its original recommendation.[459]

Despite record numbers of COVID-19 cases in the U.S. from mid-June onward and an increasing percentage of positive test results, Trump largely continued to downplay the pandemic, including his false claim in early July 2020 that 99 percent of COVID-19 cases are "totally harmless".[460][461] He began insisting that all states should resume in-person education in the fall despite a July spike in reported cases.[462]
Political pressure on health agencies
Main article: Political interference with science agencies by the first Trump administration

Trump repeatedly pressured federal health agencies to take actions he favored,[458] such as approving unproven treatments[463][464] or speeding up vaccine approvals.[464] Trump administration political appointees at HHS sought to control CDC communications to the public that undermined his claims that the pandemic was under control. CDC resisted many of the changes, but increasingly allowed HHS personnel to review articles and suggest changes before publication.[465][466] Trump alleged without evidence that FDA scientists were part of a "deep state" opposing him and delaying approval of vaccines and treatments to hurt him politically.[467]
Outbreak at the White House
Main article: White House COVID-19 outbreak
Donald Trump, wearing a black face mask, boards Marine One, a large green helicopter, from the White House lawn
Trump boards Marine One for COVID-19 treatment on October 2, 2020

On October 2, 2020, Trump tweeted that he had tested positive for COVID-19, part of a White House outbreak.[468] Later that day he was hospitalized at Walter Reed National Military Medical Center, reportedly due to fever and labored breathing. He was treated with antiviral and experimental antibody drugs and a steroid. He returned to the White House on October 5, still infectious and unwell.[469][470] During and after his treatment he continued to downplay the virus.[469] In 2021, it was revealed that his condition had been far more serious; he had dangerously low blood oxygen levels, a high fever, and lung infiltrates, indicating a severe case.[470] In January 2021, he received a COVID-19 vaccination.[471]
Effects on the 2020 presidential campaign

By July 2020, Trump's handling of the COVID-19 pandemic had become a major issue in the presidential election.[472] Biden sought to make the pandemic the central issue.[473] Polls suggested voters blamed Trump for his pandemic response[472] and disbelieved his rhetoric concerning the virus, with an Ipsos/ABC News poll indicating 65 percent of respondents disapproved of his pandemic response.[474] In the final months of the campaign, he repeatedly said that the U.S. was "rounding the turn" in managing the pandemic, despite increasing cases and deaths.[475] A few days before the November 3 election, the U.S. reported more than 100,000 cases in a single day for the first time.[476]
Investigations

After he assumed office, Trump was the subject of increasing Justice Department and congressional scrutiny, with investigations covering his election campaign, transition, and inauguration, actions taken during his presidency, his private businesses, personal taxes, and charitable foundation.[477] There were ten federal criminal investigations, eight state and local investigations, and twelve congressional investigations.[478]
Financial

In April 2019, the House Oversight Committee issued subpoenas seeking financial details from Trump's banks, Deutsche Bank and Capital One, and his accounting firm, Mazars USA. He sued the banks, Mazars, and committee chair Elijah Cummings to prevent the disclosures.[479] In May, DC District Court judge Amit Mehta ruled that Mazars must comply with the subpoena,[480] and judge Edgardo Ramos of the Southern District Court of New York ruled that the banks must also comply.[481][482] Trump's attorneys appealed.[483] In September 2022, the committee and Trump agreed to a settlement about Mazars, and the accounting firm began turning over documents.[484]
Russian election interference
Main articles: Russian interference in the 2016 United States elections and Timelines related to Donald Trump and Russian interference in United States elections
See also: Senate Intelligence Committee report on Russian interference in the 2016 United States presidential election and Steele dossier

In January 2017, American intelligence agencies—the CIA, the FBI, and the NSA, represented by the Director of National Intelligence—jointly stated with "high confidence" that the Russian government interfered in the 2016 presidential election to favor the election of Trump.[485][486] In March 2017, FBI Director James Comey told Congress, "[T]he FBI, as part of our counterintelligence mission, is investigating the Russian government's efforts to interfere in the 2016 presidential election. That includes investigating the nature of any links between individuals associated with the Trump campaign and the Russian government, and whether there was any coordination between the campaign and Russia's efforts."[487]

Many suspicious[488] links between Trump associates and Russian officials and spies were discovered and the relationships between Russians and "team Trump", including Manafort, Flynn, and Stone, were widely reported by the press.[489][490][491][492] Members of Trump's campaign and his White House staff, particularly Flynn, were in contact with Russian officials both before and after the election.[493][494] On December 29, 2016, Flynn talked with Russian Ambassador Sergey Kislyak about sanctions that were imposed that same day; Flynn later resigned in the midst of controversy over whether he misled Pence.[495] Trump told Kislyak and Sergei Lavrov in May 2017 he was unconcerned about Russian interference in U.S. elections.[496]

Trump and his allies promoted a conspiracy theory that Ukraine, rather than Russia, interfered in the 2016 election—which was also promoted by Russia to frame Ukraine.[497]
FBI Crossfire Hurricane and 2017 counterintelligence investigations

In July 2016, the FBI launched an investigation, codenamed Crossfire Hurricane, into possible links between Russia and the Trump campaign.[498] After Trump fired FBI director James Comey in May 2017, the FBI opened a counterintelligence investigation into Trump's personal and business dealings with Russia.[499] Crossfire Hurricane was transferred to the Mueller investigation,[500] but Deputy Attorney General Rod Rosenstein ended the investigation into Trump's direct ties to Russia while giving the bureau the false impression that the Robert Mueller's special counsel investigation would pursue the matter.[501][502]
Mueller investigation
Main articles: Mueller special counsel investigation, Mueller report, and Criminal charges brought in the Mueller special counsel investigation

In May 2017, Rosenstein appointed former FBI director Mueller special counsel for the Department of Justice (DOJ), ordering him to "examine 'any links and/or coordination between the Russian government' and the Trump campaign". He privately told Mueller to restrict the investigation to criminal matters "in connection with Russia's 2016 election interference".[501] The special counsel also investigated whether Trump's dismissal of James Comey as FBI director constituted obstruction of justice[503] and the Trump campaign's possible ties to Saudi Arabia, the United Arab Emirates, Turkey, Qatar, Israel, and China.[504] Trump sought to fire Mueller and shut down the investigation multiple times, but backed down after his staff objected or after changing his mind.[505]

In March 2019, Mueller gave his final report to Attorney General William Barr,[506] which Barr purported to summarize in a letter to Congress. A federal court, and Mueller himself, said Barr mischaracterized the investigation's conclusions and, in so doing, confused the public.[507][508][509] Trump repeatedly claimed that the investigation exonerated him; the Mueller report expressly stated that it did not.[510]

A redacted version of the report, publicly released in April 2019, found that Russia interfered in 2016 to favor Trump.[511] Despite "numerous links between the Russian government and the Trump campaign", the report found that the prevailing evidence "did not establish" that Trump campaign members conspired or coordinated with Russian interference.[512][513] The report revealed sweeping Russian interference[513] and detailed how Trump and his campaign welcomed and encouraged it, believing it would benefit them electorally.[514][515][516][517]

The report also detailed multiple acts of potential obstruction of justice by Trump, but "did not draw ultimate conclusions about the President's conduct".[518][519] Investigators decided they could not "apply an approach that could potentially result in a judgment that the President committed crimes" as an Office of Legal Counsel opinion stated that a sitting president could not be indicted,[520] and investigators would not accuse him of a crime when he cannot clear his name in court.[521] The report concluded that Congress, having the authority to take action against a president for wrongdoing, "may apply the obstruction laws".[520] The House of Representatives subsequently launched an impeachment inquiry following the Trump–Ukraine scandal, but did not pursue an article of impeachment related to the Mueller investigation.[522][523]

Several Trump associates pleaded guilty or were convicted in connection with Mueller's investigation and related cases, including Manafort[524] and Flynn.[525][526] Trump's former attorney Michael Cohen pleaded guilty to lying to Congress about Trump's 2016 attempts to reach a deal with Russia to build a Trump Tower in Moscow. Cohen said he had made the false statements on behalf of Trump.[527] In February 2020, Stone was sentenced to 40 months in prison for lying to Congress and witness tampering. The sentencing judge said Stone "was prosecuted for covering up for the president".[528]
First impeachment
Main articles: First impeachment of Donald Trump and Trump–Ukraine scandal
Nancy Pelosi presides over a crowded House of Representatives chamber floor during the impeachment vote
Members of House of Representatives vote on two articles of impeachment (H.Res. 755), December 18, 2019

In August 2019, a whistleblower filed a complaint with the Inspector General of the Intelligence Community about a July 25 phone call between Trump and Ukrainian president Volodymyr Zelenskyy, during which Trump had pressured Zelenskyy to investigate CrowdStrike and Democratic presidential candidate Biden and his son Hunter.[529] The whistleblower said that the White House had attempted to cover up the incident and that the call was part of a wider campaign by the Trump administration and Trump attorney Rudy Giuliani that may have included withholding financial aid from Ukraine in July 2019 and canceling Pence's May 2019 Ukraine trip.[530]

House Speaker Nancy Pelosi initiated a formal impeachment inquiry on September 24.[531] Trump then confirmed that he withheld military aid from Ukraine, offering contradictory reasons for the decision.[532][533] On September 25, his administration released a memorandum of the phone call which confirmed that, after Zelenskyy mentioned purchasing American anti-tank missiles, Trump asked him to discuss investigating Biden and his son with Giuliani and Barr.[529][534] The testimony of multiple administration officials and former officials confirmed that this was part of a broader effort to further Trump's personal interests by giving him an advantage in the upcoming presidential election.[535] In October, William B. Taylor Jr., the chargé d'affaires for Ukraine, testified before congressional committees that soon after arriving in Ukraine in June 2019, he found that Zelenskyy was being subjected to pressure directed by Trump and led by Giuliani. According to Taylor and others, the goal was to coerce Zelenskyy into making a public commitment to investigate the company that employed Hunter Biden, as well as rumors about Ukrainian involvement in the 2016 U.S. presidential election.[536] He said it was made clear that until Zelenskyy made such an announcement, the administration would not release scheduled military aid for Ukraine and not invite Zelenskyy to the White House.[537]

On December 13, the House Judiciary Committee voted along party lines to pass two articles of impeachment: one for abuse of power and one for obstruction of Congress.[538] After debate, the House of Representatives impeached Trump on both articles on December 18.[539]
Trump displaying the headline "Trump acquitted"

During the trial in January 2020, the House impeachment managers cited evidence to support charges of abuse of power and obstruction of Congress and asserted that Trump's actions were exactly what the founding fathers had in mind when they created the impeachment process.[540]

Trump's lawyers did not deny the facts as presented in the charges, but said that he had not broken any laws or obstructed Congress.[541] They argued that the impeachment was "constitutionally and legally invalid" because he was not charged with a crime and that abuse of power is not an impeachable offense.[541]

On January 31, the Senate voted against allowing subpoenas for witnesses or documents.[542] The impeachment trial was the first in U.S. history without witness testimony.[543]

Trump was acquitted of both charges by the Republican majority. Senator Mitt Romney was the only Republican who voted to convict him on one charge, the abuse of power.[544] Following his acquittal, he fired impeachment witnesses and other political appointees and career officials he deemed insufficiently loyal.[545]
Second impeachment
Main articles: Second impeachment of Donald Trump and Second impeachment trial of Donald Trump
Speaker of the House Nancy Pelosi seated at a table and surrounded by public officials. She is signing the second impeachment of Trump.
Speaker of the House Nancy Pelosi signing the second impeachment of Trump

On January 11, 2021, an article of impeachment charging Trump with incitement of insurrection against the U.S. government was introduced to the House.[546] The House voted 232–197 to impeach him on January 13, making him the first U.S. president to be impeached twice.[547] Ten Republicans voted for the impeachment—the most members of a party ever to vote to impeach a president of their own party.[548]

On February 13, following a five-day Senate trial, Trump was acquitted when the Senate vote fell ten votes short of the two-thirds majority required to convict; seven Republicans joined every Democrat in voting to convict, the most bipartisan support in any Senate impeachment trial of a president or former president.[549][550] Most Republicans voted to acquit him, although some held him responsible but felt the Senate did not have jurisdiction over former presidents (he had left office on January 20; the Senate voted 56–44 that the trial was constitutional).[551]
2020 presidential election
Defeat to Biden
Main articles: 2020 United States presidential election and Donald Trump 2020 presidential campaign

Breaking with precedent, Trump filed to run for a second term within a few hours of assuming the presidency.[552] He held his first reelection rally less than a month after taking office[553] and officially became the Republican nominee in August 2020.[554]

In his first two years in office, Trump's reelection committee reported raising $67.5 million and began 2019 with $19.3 million in cash.[555] By July 2020, his campaign and the Republican Party had raised $1.1 billion and spent $800 million, losing their cash advantage over Biden.[556] The cash shortage forced the campaign to scale back advertising spending.[557]

Trump campaign advertisements focused on crime, claiming that cities would descend into lawlessness if Biden won.[558] He repeatedly misrepresented Biden's positions[559][560] and shifted to appeals to racism.[561]

Starting in the spring of 2020, Trump began to sow doubts about the election, claiming without evidence that the election would be rigged and that the expected widespread use of mail balloting would produce massive election fraud.[562][563] When, in August, the House of Representatives voted for a $25 billion grant to the U.S. Postal Service for the expected surge in mail voting, he blocked funding, saying he wanted to prevent any increase in voting by mail, creating a crisis in the Postal Service.[564] He repeatedly refused to say whether he would accept the results if he lost and commit to a peaceful transition of power.[565][566]

Biden won the election on November 3, receiving 81.3 million votes (51.3 percent) to Trump's 74.2 million (46.8 percent)[567][568] and 306 Electoral College votes to Trump's 232.[569]
Rejection of results
Further information: Attempts to overturn the 2020 United States presidential election, 2020–21 United States election protests, and Election denial movement in the United States

At 2 a.m. the morning after the election, with the results still unclear, Trump declared victory.[570] After Biden was projected the winner days later, Trump baselessly alleged election fraud.[571] He and his allies filed many legal challenges to the results, which were rejected by at least 86 judges in both the state and federal courts, including by federal judges appointed by Trump himself, finding no factual or legal basis.[572][573] His allegations were also refuted by state election officials.[574] On December 11, the U.S. Supreme Court declined to hear a case from the Texas attorney general that asked the court to overturn the election results in four states won by Biden.[575]

Trump withdrew from public activities in the weeks following the election.[576] He initially blocked government officials from cooperating in Biden's presidential transition.[577][578] After three weeks, the administrator of the General Services Administration declared Biden the "apparent winner" of the election, allowing the disbursement of transition resources to his team.[579] Trump still did not formally concede while claiming he recommended the GSA begin transition protocols.[580][581]

The Electoral College formalized Biden's victory on December 14.[569] From November to January, Trump repeatedly sought help to overturn the results, personally pressuring Republican local and state office-holders,[582] Republican state and federal legislators,[583] the Justice Department,[584] and Vice President Pence,[585] urging various actions such as replacing presidential electors, or a request for Georgia officials to "find" votes and announce a "recalculated" result.[583] On February 10, 2021, Georgia prosecutors opened a criminal investigation into Trump's efforts to subvert the election in Georgia.[586]

Trump did not attend Biden's inauguration.[587]
January 6 Capitol attack
Main article: January 6 United States Capitol attack
For a chronological guide, see Timeline of the January 6 United States Capitol attack.
Trump speaking at the "Stop the Steal" rally on January 6

In December 2020, Newsweek reported the Pentagon was on red alert, and ranking officers had discussed what to do if Trump declared martial law. The Pentagon responded with quotes from defense leaders that the military has no role in the outcome of elections.[588]

When Trump moved supporters into positions of power at the Pentagon after the November 2020 election, Chairman of the Joint Chiefs of Staff Mark Milley and CIA director Gina Haspel became concerned about a possible coup attempt or military action against China or Iran.[589][590] Milley insisted that he should be consulted about any military orders from Trump, including the use of nuclear weapons.[591][592]

On January 6, 2021, while congressional certification of the presidential election results was taking place in the U.S. Capitol, Trump held a noon rally at the Ellipse in Washington, D.C., where he called for the election result to be overturned and urged his supporters to "fight like hell" and "take back our country" by marching to the Capitol.[593][594] Many supporters did, joining a crowd already there. The mob broke into the building, disrupting certification and causing the evacuation of Congress.[595] During the violence, Trump posted messages on Twitter without asking the rioters to disperse. At 6 p.m., he tweeted that the rioters should "go home with love & in peace", calling them "great patriots" and repeating that the election was stolen.[596] After the mob was removed, Congress reconvened and confirmed Biden's win in the early hours of the following morning.[597] According to the Department of Justice, more than 140 police officers were injured, and five people died.[598][599]

In March 2023, Trump collaborated with incarcerated rioters on a song to benefit the prisoners. In June, he said that, if reelected, he would pardon many of them.[600]
Inter-presidency (2021–present)
See also: Personal and business legal affairs of Donald Trump and Legal affairs of Donald Trump as president

Trump lives at his Mar-a-Lago club, having established an office there as provided for by the Former Presidents Act.[56][601][602] He is entitled to live there legally as a club employee.[603][604]

Trump's false claims concerning the 2020 election were commonly referred to as the "big lie" in the press and by his critics. In May 2021, he and his supporters attempted to co-opt the term, using it to refer to the election itself.[605][606] The Republican Party used his false election narrative to justify the imposition of new voting restrictions in its favor.[606][607] As late as July 2022, he was still pressuring state legislators to overturn the 2020 election.[608]

Unlike other former presidents, Trump continued to dominate his party; he has been described as a modern party boss. He continued fundraising, raising more than twice as much as the Republican Party itself, and profited from fundraisers many Republican candidates held at Mar-a-Lago. Much of his focus was on how elections are run and on ousting election officials who had resisted his attempts to overturn the 2020 election results. In the 2022 midterm elections he endorsed over 200 candidates for various offices, most of whom supported his false claim that the 2020 presidential election was stolen from him.[609][610][611]
Business activities

In February 2021, Trump registered a new company, Trump Media & Technology Group (TMTG), for providing "social networking services" to U.S. customers.[612][613] In March 2024, TMTG merged with special-purpose acquisition company Digital World Acquisition and became a public company.[614] In February 2022, TMTG launched Truth Social, a social media platform.[615] As of March 2023, Trump Media, which had taken $8 million from Russia-connected entities, was being investigated by federal prosecutors for possible money laundering.[616][617]
Investigations, criminal indictments and convictions, civil lawsuits

Trump is the only U.S. president or former president to be convicted of a crime and the first major-party candidate to run for president after a felony conviction.[618] As of May 2024, he faces numerous criminal charges and civil cases.[619][620]
FBI investigations
Main articles: FBI investigation into Donald Trump's handling of government documents, FBI search of Mar-a-Lago, and Smith special counsel investigation
Classified intelligence material found during search of Mar-a-Lago

When Trump left the White House in January 2021, he took government materials with him to Mar-a-Lago. By May 2021, the National Archives and Records Administration (NARA) realized that important documents had not been turned over to them and asked his office to locate them. In January 2022, they retrieved 15 boxes of White House records from Mar-a-Lago. NARA later informed the Department of Justice that some of the retrieved documents were classified material.[621] The Justice Department began an investigation[622] and sent Trump a subpoena for additional material.[621] Justice Department officials visited Mar-a-Lago and received some classified documents from his lawyers,[621] one of whom signed a statement affirming that all material marked as classified had been returned.[623]

On August 8, 2022, FBI agents searched Mar-a-Lago to recover government documents and material Trump had taken with him when he left office in violation of the Presidential Records Act,[624][625] reportedly including some related to nuclear weapons.[626] The search warrant indicates an investigation of potential violations of the Espionage Act and obstruction of justice laws.[627] The items taken in the search included 11 sets of classified documents, four of them tagged as "top secret" and one as "top secret/SCI", the highest level of classification.[624][625]

On November 18, 2022, U.S. attorney general Merrick Garland appointed federal prosecutor Jack Smith as a special counsel to oversee the federal criminal investigations into Trump retaining government property at Mar-a-Lago and examining Trump's role in the events leading up to the Capitol attack.[628][629]
Criminal referral by the House January 6 Committee
Main article: United States House Select Committee on the January 6 Attack

On December 19, 2022, the United States House Select Committee on the January 6 Attack recommended criminal charges against Trump for obstructing an official proceeding, conspiracy to defraud the United States, and inciting or assisting an insurrection.[630]
State criminal indictments
Main article: Georgia election racketeering prosecution

In December 2022, following a jury trial, the Trump Organization was convicted on 17 counts of criminal tax fraud, conspiracy, and falsifying business records in connection with a tax-fraud scheme stretching over 15 years. In January 2023, the organization was fined the maximum $1.6 million, and its chief financial officer Allen Weisselberg was sentenced to jail and probation after a plea deal. Trump was not personally charged in the case.[631][632]

Later in August, a Fulton County, Georgia, grand jury indicted Trump on 13 charges, including racketeering, for his efforts to subvert the election outcome in Georgia; multiple Trump campaign officials were also indicted.[633][634] He surrendered, was processed at Fulton County Jail, and was released on bail pending trial.[635] He pleaded not guilty.[636] On March 13, 2024, the judge dismissed three of the 13 charges against him.[637]
Federal criminal indictments
Main articles: Federal prosecution of Donald Trump (classified documents case) and Federal prosecution of Donald Trump (election obstruction case)

In June 2023, following a special counsel investigation, a federal grand jury in Miami indicted Trump on 31 counts of "willfully retaining national defense information" under the Espionage Act, one count of making false statements, and one count each of conspiracy to obstruct justice, withholding government documents, corruptly concealing records, concealing a document in a federal investigation and scheming to conceal their efforts.[638] He pleaded not guilty.[639] A superseding indictment the following month added three charges.[640] The judge assigned to the case, Aileen Cannon, was appointed to the bench by Trump and had previously issued rulings favorable to him in a past civil case, some of which were overturned by an appellate court.[641] She moved slowly on the case, indefinitely postponed the trial in May 2024, and dismissed it on July 15, ruling that the special counsel's appointment was unconstitutional.[642] On August 26, Special Counsel Smith appealed the dismissal.[643]

In August 2023, a Washington, D.C., federal grand jury indicted Trump for his efforts to overturn the 2020 election results. He was charged with conspiring to defraud the U.S., obstruct the certification of the Electoral College vote, and deprive voters of the civil right to have their votes counted, and obstructing an official proceeding.[644] He pleaded not guilty.[645] On November 25, the judge dismissed the case without prejudice after the prosecution filed a motion to dismiss citing Department of Justice policy.[646]

On November 25, the prosecution, citing Department of Justice policy prohibiting the prosecution of a sitting president, requested the dismissal of the D.C. case and Trump's removal as a co-defendant in the classified documents case. The D.C. case was dismissed without prejudice that same day. The next day, the U.S. Court of Appeals for the Eleventh Circuit dropped Trump from the appeal.[647]
Criminal conviction in the 2016 campaign fraud case
Main articles: Prosecution of Donald Trump in New York and Stormy Daniels–Donald Trump scandal
See also: Personal and business legal affairs of Donald Trump § Payments related to alleged affairs, and Karen McDougal § Alleged affair with Donald Trump

During the 2016 presidential election campaign, American Media, Inc. (AMI), publisher of the National Enquirer,[648] and a company set up by Cohen paid Playboy model Karen McDougal and adult film actress Stormy Daniels for keeping silent about their alleged affairs with Trump between 2006 and 2007.[649] Cohen pleaded guilty in 2018 to breaking campaign finance laws, saying he had arranged both payments at Trump's direction to influence the presidential election.[650] Trump denied the affairs and said he was not aware of Cohen's payment to Daniels, but he reimbursed him in 2017.[651][652] Federal prosecutors asserted that Trump had been involved in discussions regarding nondisclosure payments as early as 2014.[653] Court documents showed that the FBI believed he was directly involved in the payment to Daniels, based on calls he had with Cohen in October 2016.[654][655] Federal prosecutors closed the investigation in 2019,[656] but in 2021, the New York State Attorney General's Office and Manhattan District Attorney's Office opened a criminal investigations into his business activities.[657] The Manhattan DA's Office subpoenaed the Trump Organization and AMI for records related to the payments[658] and Trump and the Trump Organization for eight years of tax returns.[659]

In March 2023, a New York grand jury indicted Trump on 34 felony counts of falsifying business records to book the hush money payments to Daniels as business expenses, in an attempt to influence the 2016 election.[660][661][662] The trial began in April 2024, and in May a jury convicted him on all 34 counts.[663] He continues to fight the conviction and the sentencing remains unresolved.[664]
Civil judgments
Main articles: New York business fraud lawsuit against the Trump Organization and E. Jean Carroll v. Donald J. Trump

In September 2022, the attorney general of New York filed a civil fraud case against Trump, his three oldest children, and the Trump Organization.[665] During the investigation leading up to the lawsuit, Trump was fined $110,000 for failing to turn over records subpoenaed by the attorney general.[666] In an August 2022 deposition, he invoked his Fifth Amendment right against self-incrimination more than 400 times.[667] The presiding judge ruled in September 2023 that Trump, his adult sons, and the Trump Organization repeatedly committed fraud and ordered their New York business certificates canceled and their business entities sent into receivership for dissolution.[668] In February 2024, the court found him liable, ordered him to pay a penalty of more than $350 million plus interest, for a total exceeding $450 million, and barred him from serving as an officer or director of any New York corporation or legal entity for three years. He said he would appeal the verdict. The judge also ordered the company to be overseen by the monitor appointed by the court in 2023 and an independent director of compliance, and that any "restructuring and potential dissolution" would be the decision of the monitor.[669]

In May 2023, a New York jury in a federal lawsuit brought by journalist E. Jean Carroll in 2022 ("Carroll II") found Trump liable for sexual abuse and defamation and ordered him to pay her $5 million.[670] He asked for a new trial or a reduction of the award, arguing that the jury had not found him liable for rape. He also separately countersued Carroll for defamation. The judge for the two lawsuits ruled against him,[671][672] writing that Carroll's accusation of "rape" is "substantially true".[673] He appealed both decisions.[671][674] In January 2024, the jury in the defamation case brought by Carroll in 2019 ("Carroll I") ordered him to pay Carroll $83.3 million in damages. In March, he posted a $91.6 million bond and appealed.[675]
2024 presidential election
Main article: Donald Trump 2024 presidential campaign
Further information: 2024 Republican Party presidential primaries, 2024 United States presidential election, and Second presidential transition of Donald Trump
See also: Attempted assassination of Donald Trump in Pennsylvania and Attempted assassination of Donald Trump in Florida
Trump at a rally in Arizona, 2024

On November 15, 2022, Trump announced his candidacy for the 2024 presidential election and set up a fundraising account.[676][677] In March 2023, the campaign began diverting 10 percent of the donations to his leadership PAC. His campaign had paid $100 million towards his legal bills by March 2024.[678][679]

In December 2023, the Colorado Supreme Court ruled Trump disqualified for the Colorado Republican primary for his role in inciting the January 6, 2021, attack on Congress. In March 2024, the U.S. Supreme Court restored his name to the ballot in a unanimous decision, ruling that Colorado lacks the authority to enforce Section 3 of the 14th Amendment, which bars insurrectionists from holding federal office.[680]
Trump's escalation of election rigging claims before the 2024 election[681]

During the campaign, Trump made increasingly violent and authoritarian statements.[682][683][684][685] He also said that he would weaponize the FBI and the Justice Department against his political opponents[686][687] and use the military to go after Democratic politicians and those that do not support his candidacy.[688][689] He used harsher, more dehumanizing anti-immigrant rhetoric than during his presidency.[690][691][692][693] His harsher rhetoric against his political enemies has been described by some historians and scholars as authoritarian, fascist,[d] and unlike anything a political candidate has ever said in American history.[698][689][699] Age and health concerns also arose during the campaign, with several medical experts highlighting an increase in rambling, tangential speech and behavioral disinhibition.[700]

Trump mentioned "rigged election" and "election interference" earlier and more frequently than in the 2016 and 2020 campaigns and refused to commit to accepting the 2024 election results.[701][681] Analysts for The New York Times described this as an intensification of his "heads I win; tails you cheated" rhetorical strategy; the paper said the claim of a rigged election had become the backbone of the campaign.[681]
President-elect Trump with Ukrainian president Volodymyr Zelenskyy in Paris on December 7, 2024

On July 13, 2024, Trump's ear was grazed by a bullet[702] in an assassination attempt at a campaign rally in Butler Township, Pennsylvania.[703][704] Two days later, the 2024 Republican National Convention nominated him as their presidential candidate, with Senator JD Vance as his running mate.[705] On September 15, 2024, he was targeted in another assassination attempt in Florida.[706]

Trump was elected the 47th president of the United States in November 2024, defeating incumbent vice president Kamala Harris,[707] and making him the second president in U.S. history after Grover Cleveland in 1892 to be elected to a nonconsecutive second term.[708] The Associated Press and BBC News described it as an extraordinary comeback.[709][710]
Assessments
Public image
Main article: Public image of Donald Trump
See also: Donald Trump in popular culture

A Gallup poll in 134 countries comparing the approval ratings of U.S. leadership between 2016 and 2017 found that Trump led Obama in job approval in only 29 countries, most of them non-democracies;[711] approval of U.S. leadership plummeted among allies and G7 countries.[712] By mid-2020, only 16 percent of international respondents to a 13-nation Pew Research poll expressed confidence in him, lower than China's Xi Jinping and Russia's Vladimir Putin.[713]

Trump was the only president never to reach a 50 percent approval rating in the Gallup poll, which dates to 1938, partially due to a record-high partisan gap in his approval ratings: 88 percent among Republicans and 7 percent among Democrats.[714] His early ratings were unusually stable, ranging between 35 and 49 percent.[715] He finished his term with a rating between 29 and 34 percent—the lowest of any president since modern polling began—and a record-low average of 41 percent throughout his presidency.[714][716] In Gallup's annual poll asking Americans to name the man they admire the most, he placed second to Obama in 2017 and 2018, tied with Obama for first in 2019, and placed first in 2020.[717][718] Since Gallup started conducting the poll in 1946, he was the first elected president not to be named most admired in his first year in office.[719] In late 2024, Time named Trump its Person of the Year.[720]
Scholarly
Further information: Historical rankings of presidents of the United States § 2021 C-SPAN

In the C-SPAN "Presidential Historians Survey 2021",[721] historians ranked Trump as the fourth-worst president. He rated lowest in the leadership characteristics categories for moral authority and administrative skills.[722][723] The Siena College Research Institute's 2022 survey ranked him 43rd out of 45 presidents. He was ranked near the bottom in all categories except for luck, willingness to take risks, and party leadership, and he ranked last in several categories.[724] In 2018 and 2024, surveys of members of the American Political Science Association ranked him the worst president.[725][726]
Political practice and rhetoric
Further information: Trumpism, Political positions of Donald Trump, and Rhetoric of Donald Trump

Beginning with his 2016 campaign, Trump's politics and rhetoric led to the creation of a political movement known as Trumpism.[727] Professor Mary Stuckey identified his rhetoric as using vitriol: "demeaning language, false equivalency, and exclusion".[728] Professor Gary Jacobson found he used fearmongering about immigrants and minorities[729] that other researchers found were essential to his support.[730] His political base has been compared to a cult of personality.[e]

Trump's political positions have been described as right-wing populist.[738][739][740] Politico described them as "eclectic, improvisational and often contradictory".[741] He helped bring far-right fringe ideas and organizations into the mainstream.[742] The alt-right movement coalesced around and supported his candidacy, due in part to its opposition to multiculturalism and immigration.[743][744][745] He has a strong appeal to evangelical Christian voters. He appeals to Christian nationalists,[746] and his rallies take on the symbols, rhetoric and agenda of Christian nationalism.[747]
Link to hate crimes
Further information: Rhetoric of Donald Trump § Violence and dehumanization

Research suggests Trump's rhetoric is associated with an increased incidence of hate crimes.[748][749] During his 2016 campaign, he urged or praised physical attacks against protesters or reporters.[750][751] Numerous defendants investigated or prosecuted for violent acts and hate crimes, including participants in the storming of the U.S. Capitol, cited his rhetoric in arguing that they were not culpable or should receive leniency.[752][753] A nationwide review by ABC News in May 2020 identified at least 54 criminal cases from August 2015 to April 2020 in which he was invoked in direct connection with violence or threats of violence mostly by white men and primarily against minorities.[754]
Truthfulness
Main article: False or misleading statements by Donald Trump
Chart depicting false or misleading claims made by Trump
Fact-checkers from The Washington Post,[755] the Toronto Star,[756] and CNN[757] compiled data on "false or misleading claims" (orange background), and "false claims" (violet foreground), respectively.

As a candidate and as president, Trump frequently made false statements in public remarks[758][149] to an extent unprecedented in American politics.[758][759][760] His falsehoods are a distinctive part of his political identity[759] and have been described as firehosing.[761] His false and misleading statements were documented by fact-checkers, including at The Washington Post, which tallied 30,573 false or misleading statements made by him during his first presidency,[755] increasing in frequency over time.[762]

Some of Trump's falsehoods were inconsequential, such as his repeated claim of the "biggest inaugural crowd ever".[763][764] Others had more far-reaching effects, such as his promotion of antimalarial drugs as an unproven treatment for COVID-19,[765][766] causing a U.S. shortage of these drugs and panic-buying in Africa and South Asia.[767][768] Other misinformation, such as misattributing a rise in crime in England and Wales to the "spread of radical Islamic terror", served his domestic political purposes.[769] His attacks on mail-in ballots and other election practices weakened public faith in the integrity of the 2020 presidential election,[770][771] while his disinformation about the pandemic delayed and weakened the national response to it.[772][773][774] Trump habitually does not apologize for his falsehoods.[775]

Until 2018, the media rarely referred to Trump's falsehoods as lies, including when he repeated demonstrably false statements.[776][777][778]
Conspiracy theories
Main article: List of conspiracy theories promoted by Donald Trump

Before and throughout his presidency, Trump promoted numerous conspiracy theories, including Obama birtherism, the Clinton body count conspiracy theory, the conspiracy theory movement QAnon, the Global warming hoax theory, Trump Tower wiretapping allegations, that Osama bin Laden was alive and Obama and Biden had members of Navy SEAL Team 6 killed, and alleged Ukrainian interference in U.S. elections.[779][780][781][782][783] In at least two instances, Trump clarified to press that he believed the conspiracy theory in question.[781]

During and since the 2020 presidential election, Trump promoted various conspiracy theories for his defeat.[784][785][786]
Social media
Main article: Social media use by Donald Trump

Trump's social media presence attracted worldwide attention after he joined Twitter in 2009. He tweeted frequently during his 2016 campaign and as president until Twitter banned him after the January 6 attack.[787] He often used Twitter to communicate directly with the public and sideline the press.[788] In June 2017, the White House press secretary said that his tweets were official presidential statements.[789]

After years of criticism for allowing Trump to post misinformation and falsehoods, Twitter began to tag some of his tweets with fact-checks in May 2020.[790] In response, he tweeted that social media platforms "totally silence" conservatives and that he would "strongly regulate, or close them down".[791] In the days after the storming of the Capitol, he was banned from Facebook, Instagram, Twitter and other platforms.[792] The loss of his social media presence diminished his ability to shape events[793][794] and prompted a dramatic decrease in the volume of misinformation shared on Twitter.[795] In February 2022, he launched social media platform Truth Social where he only attracted a fraction of his Twitter following.[796] Elon Musk, after acquiring Twitter, reinstated his Twitter account in November 2022.[797][798] Meta Platforms' two-year ban lapsed in January 2023, allowing him to return to Facebook and Instagram,[799] although in 2024, he continued to call the company an "enemy of the people".[800]
Relationship with the press
Further information: First presidency of Donald Trump § Relationship with the news media
Trump, seated at the Resolute Desk in the White House, speaking to a crowd of reporters with boom microphones in front of him and public officials behind him
Trump talking to the press, March 2017

Trump sought media attention throughout his career, sustaining a "love-hate" relationship with the press.[801] In the 2016 campaign, he benefited from a record amount of free media coverage, elevating his standing in the Republican primaries.[146] The New York Times writer Amy Chozick wrote in 2018 that his media dominance enthralled the public and created "must-see TV".[802]

As a candidate and as president, Trump frequently accused the press of bias, calling it the "fake news media" and "the enemy of the people".[803] In 2018, journalist Lesley Stahl said that he had privately told her that he intentionally discredited the media "so when you write negative stories about me no one will believe you".[804]

As president, Trump mused about revoking the press credentials of journalists he viewed as critical.[805] His administration moved to revoke the press passes of two White House reporters, which were restored by the courts.[806] The Trump White House held about a hundred formal press briefings in 2017, declining by half during 2018 and to two in 2019.[806]

Trump also deployed the legal system to intimidate the press.[807] The Trump campaign sued The New York Times, The Washington Post, and CNN for defamation in opinion pieces about Russian election interference. All the suits were dismissed.[808] By 2024, he repeatedly voiced support for outlawing political dissent and criticism he considers misleading or challenges his claims to power,[809] and that media companies should be investigated and prosecuted for treason for displaying "bad stories" about him and possibly lose their broadcast licenses if they refuse to name confidential sources.[810]
Notes

To pass on his wealth, Fred Trump gave each of his children $6,000 every year, the maximum allowed without incurring a gift tax. To avoid taxes, after making them landlords of two of his housing developments, he paid them each $13,928 rent every year. Three years old when this began, Donald Trump was a millionaire in 2024 dollars by age eight.[110]
Trump acknowledged a negative net worth in 1990 of minus $900 million in his book The Art of the Comeback.[113] Timothy L. O'Brien explains in his book TrumpNation that Forbes dropped Trump from its list of wealthiest Americans from 1990–1995. In 1997, Forbes acknowledged Trump's negative net worth of minus $900 million.[114]
Presidential elections in the U.S. are decided by the Electoral College. Each state names a number of electors equal to its representation in Congress and (in most states) all electors vote for the winner of their state's popular vote.
Attributed to multiple references:[694][695][696][697]

    Attributed to multiple sources:[731][732][733][734][735][736][737]

References

Kranish & Fisher 2017, pp. 30, 37.
Kranish & Fisher 2017, p. v.
Horowitz, Jason (September 22, 2015). "Donald Trump's Old Queens Neighborhood Contrasts With the Diverse Area Around It". The New York Times. Retrieved November 7, 2018.
Kranish & Fisher 2017, pp. 33, 38.
D'Antonio 2015, pp. 40–41.
Kranish & Fisher 2017, p. 38.
D'Antonio 2015, p. 43.
Kranish & Fisher 2017, pp. 45, 47.
"Two Hundred and Twelfth Commencement for the Conferring of Degrees" (PDF). University of Pennsylvania. May 20, 1968. pp. 19–21. Retrieved March 31, 2023.
D'Antonio 2015, pp. 69–71.
Ashford, Grace (February 27, 2019). "Michael Cohen Says Trump Told Him to Threaten Schools Not to Release Grades". The New York Times.
Blair 2015, p. 300.
Baron, James (December 12, 1990). "Trumps Get Divorce; Next, Who Gets What?". The New York Times. Retrieved March 5, 2023.
Hafner, Josh (July 19, 2016). "Get to know Donald's other daughter: Tiffany Trump". USA Today. Retrieved July 10, 2022.
Kranish & Fisher 2017, p. 266.
"Donald Trump Fast Facts". CNN. July 2, 2021. Retrieved September 29, 2021.
Nagourney, Adam (October 30, 2020). "In Trump and Biden, a Choice of Teetotalers for President". The New York Times. Retrieved February 5, 2021.
Parker, Ashley; Rucker, Philip (October 2, 2018). "Kavanaugh likes beer — but Trump is a teetotaler: 'He doesn't like drinkers.'". The Washington Post. Retrieved February 5, 2021.
Dangerfield, Katie (January 17, 2018). "Donald Trump sleeps 4–5 hours each night; he's not the only famous 'short sleeper'". Global News. Retrieved February 5, 2021.
Almond, Douglas; Du, Xinming (December 2020). "Later bedtimes predict President Trump's performance". Economics Letters. 197. doi:10.1016/j.econlet.2020.109590. ISSN 0165-1765. PMC 7518119. PMID 33012904.
Ballengee, Ryan (July 14, 2018). "Donald Trump says he gets most of his exercise from golf, then uses cart at Turnberry". Golf News Net. Retrieved July 4, 2019.
Rettner, Rachael (May 14, 2017). "Trump thinks that exercising too much uses up the body's 'finite' energy". The Washington Post. Retrieved September 29, 2021.
O'Donnell & Rutherford 1991, p. 133.
Marquardt, Alex; Crook, Lawrence III (May 1, 2018). "Exclusive: Bornstein claims Trump dictated the glowing health letter". CNN. Retrieved May 20, 2018.
Schecter, Anna (May 1, 2018). "Trump doctor Harold Bornstein says bodyguard, lawyer 'raided' his office, took medical files". NBC News. Retrieved June 6, 2019.
Multiple sources:

    Lopez, German (February 14, 2019). "Donald Trump's long history of racism, from the 1970s to 2019". Vox. Retrieved June 15, 2019.
    Desjardins, Lisa (January 12, 2018). "Every moment in Trump's charged relationship with race". PBS NewsHour. Retrieved January 13, 2018.
    Dawsey, Josh (January 11, 2018). "Trump's history of making offensive comments about nonwhite immigrants". The Washington Post. Retrieved January 11, 2018.
    Weaver, Aubree Eliza (January 12, 2018). "Trump's 'shithole' comment denounced across the globe". Politico. Retrieved January 13, 2018.
    Stoddard, Ed; Mfula, Chris (January 12, 2018). "Africa calls Trump racist after 'shithole' remark". Reuters. Retrieved October 1, 2019.

"Harsh Words For U.S. Family Separation Policy, Quinnipiac University National Poll Finds; Voters Have Dim View Of Trump, Dems On Immigration". Quinnipiac University Polling Institute. July 3, 2018. Retrieved July 5, 2018.
Lopez, German (December 15, 2017). "The past year of research has made it very clear: Trump won because of racial resentment". Vox. Retrieved January 14, 2018.
Lajevardi & Oskooii 2018.
Diaz, Jaclyn (October 21, 2024). "The Central Park 5 are suing Trump over Philly debate comments". NPR. Retrieved November 27, 2024.
John, Arit (June 23, 2020). "From birtherism to 'treason': Trump's false allegations against Obama". Los Angeles Times. Retrieved February 17, 2023.
Keneally, Meghan (September 18, 2015). "Donald Trump's History of Raising Birther Questions About President Obama". ABC News. Retrieved August 27, 2016.
Haberman, Maggie; Rappeport, Alan (September 16, 2016). "Trump Drops False 'Birther' Theory, but Floats a New One: Clinton Started It". The New York Times. Retrieved October 12, 2021.
Haberman, Maggie; Martin, Jonathan (November 28, 2017). "Trump Once Said the 'Access Hollywood' Tape Was Real. Now He's Not Sure". The New York Times. Retrieved June 11, 2020.
Rothe & Collins 2019.
Shear, Michael D.; Sullivan, Eileen (October 16, 2018). "'Horseface,' 'Lowlife,' 'Fat, Ugly': How the President Demeans Women". The New York Times. Retrieved August 5, 2020.
Osborne, Lucy (September 17, 2020). "'It felt like tentacles': the women who accuse Trump of sexual misconduct". The Guardian. Retrieved June 6, 2024.
Timm, Jane C. (October 7, 2016). "Trump caught on hot mic making lewd comments about women in 2005". NBC News. Retrieved June 10, 2018.
Penington, Bill. "What Exactly Is 'Locker-Room Talk'? Let an Expert Explain". The New York Times. Retrieved December 14, 2024.
Fahrenthold, David. "Trump recorded having extremely lewd conversation about women in 2005". The Washington Post. Retrieved December 14, 2024.
Burns, Alexander; Haberman, Maggie; Martin, Jonathan (October 7, 2016). "Donald Trump Apology Caps Day of Outrage Over Lewd Tape". The New York Times. Retrieved October 8, 2016.
Handy, Bruce (April 1, 2019). "Trump Once Proposed Building a Castle on Madison Avenue". The Atlantic. Retrieved July 28, 2024.
Mahler, Jonathan; Eder, Steve (August 27, 2016). "'No Vacancies' for Blacks: How Donald Trump Got His Start, and Was First Accused of Bias". The New York Times. Retrieved January 13, 2018.
Rich, Frank (April 30, 2018). "The Original Donald Trump". New York. Retrieved May 8, 2018.
Blair 2015, p. 250.
Mahler, Jonathan; Flegenheimer, Matt (June 20, 2016). "What Donald Trump Learned From Joseph McCarthy's Right-Hand Man". The New York Times. Retrieved May 26, 2020.
1634–1699: McCusker, J. J. (1997). How Much Is That in Real Money? A Historical Price Index for Use as a Deflator of Money Values in the Economy of the United States: Addenda et Corrigenda (PDF). American Antiquarian Society. 1700–1799: McCusker, J. J. (1992). How Much Is That in Real Money? A Historical Price Index for Use as a Deflator of Money Values in the Economy of the United States (PDF). American Antiquarian Society. 1800–present: Federal Reserve Bank of Minneapolis. "Consumer Price Index (estimate) 1800–". Retrieved February 29, 2024.
Kranish, Michael; O'Harrow, Robert Jr. (January 23, 2016). "Inside the government's racial bias case against Donald Trump's company, and how he fought it". The Washington Post. Retrieved January 7, 2021.
Johnston 2016, pp. 45–46.
Brenner, Marie (June 28, 2017). "How Donald Trump and Roy Cohn's Ruthless Symbiosis Changed America". Vanity Fair. Retrieved May 26, 2020.
Qiu, Linda (June 21, 2016). "Yep, Donald Trump's companies have declared bankruptcy...more than four times". PolitiFact. Retrieved May 25, 2023.
Nevius, James (April 3, 2019). "The winding history of Donald Trump's first major Manhattan real estate project". Curbed.
Kessler, Glenn (March 3, 2016). "Trump's false claim he built his empire with a 'small loan' from his father". The Washington Post. Retrieved September 29, 2021.
Kranish & Fisher 2017, p. 84.
Geist, William E. (April 8, 1984). "The Expanding Empire of Donald Trump". The New York Times Magazine. Retrieved September 29, 2021.
Haberman, Maggie (October 31, 2019). "Trump, Lifelong New Yorker, Declares Himself a Resident of Florida". The New York Times. Retrieved January 24, 2020.
"Trump Revises Plaza Loan". The New York Times. November 4, 1992. Retrieved May 23, 2023.
"Trump's Plaza Hotel Bankruptcy Plan Approved". The New York Times. Reuters. December 12, 1992. Retrieved May 24, 2023.
Segal, David (January 16, 2016). "What Donald Trump's Plaza Deal Reveals About His White House Bid". The New York Times. Retrieved May 3, 2022.
Stout, David; Gilpin, Kenneth N. (April 12, 1995). "Trump Is Selling Plaza Hotel To Saudi and Asian Investors". The New York Times. Retrieved July 18, 2019.
Kranish & Fisher 2017, p. 298.
Bagli, Charles V. (June 1, 2005). "Trump Group Selling West Side Parcel for $1.8 billion". The New York Times. Retrieved May 17, 2016.
Kiel, Paul; Buettner, Russ (May 11, 2024). "IRS Audit of Trump Could Cost Former President More Than $100 Million". ProPublica. Retrieved August 26, 2024.
McQuade, Dan (August 16, 2015). "The Truth About the Rise and Fall of Donald Trump's Atlantic City Empire". Philadelphia. Retrieved March 21, 2016.
Kranish & Fisher 2017, p. 128.
Saxon, Wolfgang (April 28, 1986). "Trump Buys Hilton's Hotel in Atlantic City". The New York Times. Retrieved May 25, 2023.
"Trump's Castle and Plaza file for bankruptcy". United Press International. March 9, 1992. Retrieved May 25, 2023.
"Company News; Taj Mahal is out of Bankruptcy". The New York Times. October 5, 1991. Retrieved May 22, 2008.
O'Connor, Claire (May 29, 2011). "Fourth Time's A Charm: How Donald Trump Made Bankruptcy Work For Him". Forbes. Retrieved January 27, 2022.
Norris, Floyd (June 7, 1995). "Trump Plaza casino stock trades today on Big Board". The New York Times. Retrieved December 14, 2014.
Tully, Shawn (March 10, 2016). "How Donald Trump Made Millions Off His Biggest Business Failure". Fortune. Retrieved May 6, 2018.
Peterson-Withorn, Chase (April 23, 2018). "Donald Trump Has Gained More Than $100 Million On Mar-a-Lago". Forbes. Retrieved July 4, 2018.
Dangremond, Sam; Kim, Leena (December 22, 2017). "A History of Mar-a-Lago, Donald Trump's American Castle". Town & Country. Retrieved July 3, 2018.
Garcia, Ahiza (December 29, 2016). "Trump's 17 golf courses teed up: Everything you need to know". CNN Money. Retrieved January 21, 2018.
"Take a look at the golf courses owned by Donald Trump". Golfweek. July 24, 2020. Retrieved July 7, 2021.
Anthony, Zane; Sanders, Kathryn; Fahrenthold, David A. (April 13, 2018). "Whatever happened to Trump neckties? They're over. So is most of Trump's merchandising empire". The Washington Post. Retrieved September 29, 2021.
Williams, Aaron; Narayanswamy, Anu (January 25, 2017). "How Trump has made millions by selling his name". The Washington Post. Retrieved December 12, 2017.
Markazi, Arash (July 14, 2015). "5 things to know about Donald Trump's foray into doomed USFL". ESPN. Retrieved September 30, 2021.
O'Donnell & Rutherford 1991, p. 137–143.
Hogan, Kevin (April 10, 2016). "The Strange Tale of Donald Trump's 1989 Biking Extravaganza". Politico. Retrieved April 12, 2016.
Buettner, Russ; Craig, Susanne (May 7, 2019). "Decade in the Red: Trump Tax Figures Show Over $1 Billion in Business Losses". The New York Times. Retrieved May 8, 2019.
Mattingly, Phil; Jorgensen, Sarah (August 23, 2016). "The Gordon Gekko era: Donald Trump's lucrative and controversial time as an activist investor". CNN. Retrieved September 14, 2022.
Peterson, Barbara (April 13, 2017). "The Crash of Trump Air". The Daily Beast. Retrieved May 17, 2023.
"10 Donald Trump Business Failures". Time. October 11, 2016. Retrieved May 17, 2023.
Barstow, David; Craig, Susanne; Buettner, Russ (October 2, 2018). "Trump Engaged in Suspect Tax Schemes as He Reaped Riches From His Father". The New York Times. Retrieved October 2, 2018.
Haberman 2022, pp. 129–130.
Rutenberg, Jim (June 22, 2002). "Three Beauty Pageants Leaving CBS for NBC". The New York Times. Retrieved August 14, 2016.
de Moraes, Lisa (June 22, 2002). "There She Goes: Pageants Move to NBC". The Washington Post. Retrieved August 14, 2016.
Zara, Christopher (October 26, 2016). "Why the heck does Donald Trump have a Walk of Fame star, anyway? It's not the reason you think". Fast Company. Retrieved June 16, 2018.
Puente, Maria (June 29, 2015). "NBC to Donald Trump: You're fired". USA Today. Retrieved July 28, 2015.
D'Antonio 2015, pp. 281–282.
D'Antonio 2015, pp. 282–283.
Eder, Steve (November 18, 2016). "Donald Trump Agrees to Pay $25 Million in Trump University Settlement". The New York Times. Retrieved November 18, 2016.
Tigas, Mike; Wei, Sisi (May 9, 2013). "Nonprofit Explorer". ProPublica. Retrieved September 9, 2016.
Fahrenthold, David A. (September 10, 2016). "How Donald Trump retooled his charity to spend other people's money". The Washington Post. Retrieved March 19, 2024.
Pallotta, Frank (August 18, 2022). "Investigation into Vince McMahon's hush money payments reportedly turns up Trump charity donations". CNN. Retrieved March 19, 2024.
Solnik, Claude (September 15, 2016). "Taking a peek at Trump's (foundation) tax returns". Long Island Business News. Retrieved September 30, 2021.
Cillizza, Chris; Fahrenthold, David A. (September 15, 2016). "Meet the reporter who's giving Donald Trump fits". The Washington Post. Retrieved June 26, 2021.
Fahrenthold, David A. (October 3, 2016). "Trump Foundation ordered to stop fundraising by N.Y. attorney general's office". The Washington Post. Retrieved May 17, 2023.
Jacobs, Ben (December 24, 2016). "Donald Trump to dissolve his charitable foundation after mounting complaints". The Guardian. Retrieved December 25, 2016.
Thomsen, Jacqueline (June 14, 2018). "Five things to know about the lawsuit against the Trump Foundation". The Hill. Retrieved June 15, 2018.
Goldmacher, Shane (December 18, 2018). "Trump Foundation Will Dissolve, Accused of 'Shocking Pattern of Illegality'". The New York Times. Retrieved May 9, 2019.
Katersky, Aaron (November 7, 2019). "President Donald Trump ordered to pay $2M to collection of nonprofits as part of civil lawsuit". ABC News. Retrieved November 7, 2019.
"Donald Trump: Three decades, 4,095 lawsuits". USA Today. Retrieved April 17, 2018.
Winter, Tom (June 24, 2016). "Trump Bankruptcy Math Doesn't Add Up". NBC News. Retrieved February 26, 2020.
Flitter, Emily (July 17, 2016). "Art of the spin: Trump bankers question his portrayal of financial comeback". Reuters. Retrieved October 14, 2018.
Smith, Allan (December 8, 2017). "Trump's long and winding history with Deutsche Bank could now be at the center of Robert Mueller's investigation". Business Insider. Retrieved October 14, 2018.
Riley, Charles; Egan, Matt (January 12, 2021). "Deutsche Bank won't do any more business with Trump". CNN. Retrieved September 14, 2022.
Stump, Scott (October 26, 2015). "Donald Trump: My dad gave me 'a small loan' of $1 million to get started". CNBC. Retrieved November 13, 2016.
Buettner & Craig 2024, pp. 30–31.
Barstow, David; Craig, Susanne; Buettner, Russ (October 2, 2018). "11 Takeaways From The Times's Investigation into Trump's Wealth". The New York Times. Retrieved October 3, 2018.
Greenberg, Jonathan (April 20, 2018). "Trump lied to me about his wealth to get onto the Forbes 400. Here are the tapes". The Washington Post. Retrieved September 29, 2021.
Boyer, Dave (October 3, 2016). "Donald Trump revealed $900 million business loss in '97 book". The Washington Times. Retrieved December 18, 2024.
O'Brien 2005, p. 79.
Johnston 2021, p. 20.
Alexander, Dan (November 4, 2024) [September 27, 2024]. "Here's How Much Donald Trump Is Worth". Forbes. Retrieved December 15, 2024.
"Profile: Donald Trump". Forbes. December 16, 2024. Retrieved December 16, 2024.
Buncombe, Andrew (July 4, 2018). "Trump boasted about writing many books – his ghostwriter says otherwise". The Independent. Retrieved October 11, 2020.
Mayer, Jane (July 18, 2016). "Donald Trump's Ghostwriter Tells All". The New Yorker. Retrieved June 19, 2017.
LaFrance, Adrienne (December 21, 2015). "Three Decades of Donald Trump Film and TV Cameos". The Atlantic.
Kranish & Fisher 2017, p. 166.
Massie, Christopher; Kaczynski, Andrew (March 16, 2016). "There Are Hours Of Audio Of Donald Trump's Nationally Syndicated Radio Show In The 2000s". BuzzFeed. Retrieved December 6, 2024.
Grossmann, Matt; Hopkins, David A. (September 9, 2016). "How the conservative media is taking over the Republican Party". The Washington Post. Retrieved October 19, 2018.
Grynbaum, Michael M.; Parker, Ashley (July 16, 2016). "Donald Trump the Political Showman, Born on 'The Apprentice'". The New York Times. Retrieved July 8, 2018.
Kranish & Fisher 2017, pp. 213–217.
Poniewozik, James (September 28, 2020). "Donald Trump Was the Real Winner of 'The Apprentice'". The New York Times. Retrieved October 18, 2023.
Rao, Sonia (February 4, 2021). "Facing expulsion, Trump resigns from the Screen Actors Guild: 'You have done nothing for me'". The Washington Post. Retrieved February 5, 2021.
Harmata, Claudia (February 7, 2021). "Donald Trump Banned from Future Re-Admission to SAG-AFTRA: It's 'More Than a Symbolic Step'". People. Retrieved February 8, 2021.
Gillin, Joshua (August 24, 2015). "Bush says Trump was a Democrat longer than a Republican 'in the last decade'". PolitiFact. Retrieved March 18, 2017.
"Trump Officially Joins Reform Party". CNN. October 25, 1999. Retrieved December 26, 2020.
Oreskes, Michael (September 2, 1987). "Trump Gives a Vague Hint of Candidacy". The New York Times. Retrieved February 17, 2016.
Butterfield, Fox (November 18, 1987). "Trump Urged To Head Gala Of Democrats". The New York Times. Retrieved October 1, 2021.
Meacham 2016, p. 326.
Politico. "George W. Bush ‘surprised’ by dad’s criticism, author says". [1]
Winger, Richard (December 25, 2011). "Donald Trump Ran For President in 2000 in Several Reform Party Presidential Primaries". Ballot Access News. Retrieved October 1, 2021.
Clift, Eleanor (July 18, 2016). "The Last Time Trump Wrecked a Party". The Daily Beast. Retrieved October 14, 2021.
Nagourney, Adam (February 14, 2000). "Reform Bid Said to Be a No-Go for Trump". The New York Times. Retrieved December 26, 2020.
MacAskill, Ewen (May 16, 2011). "Donald Trump bows out of 2012 US presidential election race". The Guardian. Retrieved February 28, 2020.
Bobic, Igor; Stein, Sam (February 22, 2017). "How CPAC Helped Launch Donald Trump's Political Career". HuffPost. Retrieved February 28, 2020.
Linkins, Jason (February 11, 2011). "Donald Trump Brings His 'Pretend To Run For President' Act To CPAC". HuffPost. Retrieved September 14, 2022.
Lerner, Adam B. (June 16, 2015). "The 10 best lines from Donald Trump's announcement speech". Politico. Retrieved June 7, 2018.
Graham, David A. (May 13, 2016). "The Lie of Trump's 'Self-Funding' Campaign". The Atlantic. Retrieved June 7, 2018.
Reeve, Elspeth (October 27, 2015). "How Donald Trump Evolved From a Joke to an Almost Serious Candidate". The New Republic. Retrieved July 23, 2018.
Bump, Philip (March 23, 2016). "Why Donald Trump is poised to win the nomination and lose the general election, in one poll". The Washington Post. Retrieved October 1, 2021.
Nussbaum, Matthew (May 3, 2016). "RNC Chairman: Trump is our nominee". Politico. Retrieved May 4, 2016.
Cillizza, Chris (June 14, 2016). "This Harvard study is a powerful indictment of the media's role in Donald Trump's rise". The Washington Post. Retrieved October 1, 2021.
Flitter, Emily; Oliphant, James (August 28, 2015). "Best president ever! How Trump's love of hyperbole could backfire". Reuters. Retrieved October 1, 2021.
McCammon, Sarah (August 10, 2016). "Donald Trump's controversial speech often walks the line". NPR. Retrieved October 1, 2021.
"The 'King of Whoppers': Donald Trump". FactCheck.org. December 21, 2015. Retrieved March 4, 2019.
Holan, Angie Drobnic; Qiu, Linda (December 21, 2015). "2015 Lie of the Year: the campaign misstatements of Donald Trump". PolitiFact. Retrieved October 1, 2021.
Farhi, Paul (February 26, 2016). "Think Trump's wrong? Fact checkers can tell you how often. (Hint: A lot.)". The Washington Post. Retrieved October 1, 2021.
Walsh, Kenneth T. (August 15, 2016). "Trump: Media Is 'Dishonest and Corrupt'". U.S. News & World Report. Retrieved October 1, 2021.
Blake, Aaron (July 6, 2016). "Donald Trump is waging war on political correctness. And he's losing". The Washington Post. Retrieved October 1, 2021.
Hartig, Hannah; Lapinski, John; Psyllos, Stephanie (July 19, 2016). "Poll: Clinton and Trump Now Tied as GOP Convention Kicks Off". NBC News. Retrieved October 1, 2021.
Levingston, Ivan (July 15, 2016). "Donald Trump officially names Mike Pence for VP". CNBC. Retrieved October 1, 2021.
"Trump closes the deal, becomes Republican nominee for president". Fox News. July 19, 2016. Retrieved October 1, 2021.
"US presidential debate: Trump won't commit to accept election result". BBC News. October 20, 2016. Retrieved October 27, 2016.
Johnson, Jenna (April 12, 2017). "Trump on NATO: 'I said it was obsolete. It's no longer obsolete.'". The Washington Post. Retrieved November 26, 2019.
Edwards 2018, "On the campaign trail, Trump repeatedly called North Atlantic Treaty Organization (NATO) 'obsolete'".
Rucker, Philip; Costa, Robert (March 21, 2016). "Trump questions need for NATO, outlines noninterventionist foreign policy". The Washington Post. Retrieved August 24, 2021.
"Trump's promises before and after the election". BBC. September 19, 2017. Retrieved October 1, 2021.
"Donald Trump's Mexico wall: Who is going to pay for it?". BBC. February 6, 2017. Retrieved December 9, 2017.
"Donald Trump emphasizes plans to build 'real' wall at Mexico border". Canadian Broadcasting Corporation. August 19, 2015. Retrieved September 29, 2015.
Oh, Inae (August 19, 2015). "Donald Trump: The 14th Amendment is Unconstitutional". Mother Jones. Retrieved November 22, 2015.
Schaffner, Brian F.; Macwilliams, Matthew; Nteta, Tatishe (March 2018). "Understanding White Polarization in the 2016 Vote for President: The Sobering Role of Racism and Sexism". Political Science Quarterly. 133 (1): 9–34. doi:10.1002/polq.12737.
Wolf, Z. Byron (April 6, 2018). "Trump basically called Mexicans rapists again". CNN. Retrieved June 28, 2022.
"NBC Officially Fires Trump From 'Celebrity Apprentice'". NBC News. Associated Press. August 13, 2015. Retrieved November 9, 2024.
Diamond, Jeremy; Frates, Chris (July 22, 2015). "Donald Trump's 92-page financial disclosure released". CNN. Retrieved September 14, 2022.
Executive Branch Personnel Public Financial Disclosure Report (U.S. OGE Form 278e) (PDF). U.S. Office of Government Ethics (Report). July 15, 2015. Archived from the original (PDF) on July 23, 2015. Retrieved December 21, 2023 – via Bloomberg Businessweek.
Rappeport, Alan (May 11, 2016). "Donald Trump Breaks With Recent History by Not Releasing Tax Returns". The New York Times. Retrieved July 19, 2016.
Qiu, Linda (October 5, 2016). "Pence's False claim that Trump 'hasn't broken' tax return promise". PolitiFact. Retrieved April 29, 2020.
Isidore, Chris; Sahadi, Jeanne (February 26, 2016). "Trump says he can't release tax returns because of audits". CNN. Retrieved March 1, 2023.
de Vogue, Ariane (February 22, 2021). "Supreme Court allows release of Trump tax returns to NY prosecutor". CNN. Retrieved September 14, 2022.
Gresko, Jessica (February 22, 2021). "Supreme Court won't halt turnover of Trump's tax records". AP News. Retrieved October 2, 2021.
Eder, Steve; Twohey, Megan (October 10, 2016). "Donald Trump Acknowledges Not Paying Federal Income Taxes for Years". The New York Times. Retrieved October 2, 2021.
Schmidt, Kiersten; Andrews, Wilson (December 19, 2016). "A Historic Number of Electors Defected, and Most Were Supposed to Vote for Clinton". The New York Times. Retrieved January 31, 2017.
Desilver, Drew (December 20, 2016). "Trump's victory another example of how Electoral College wins are bigger than popular vote ones". Pew Research Center. Retrieved October 2, 2021.
Crockett, Zachary (November 11, 2016). "Donald Trump will be the only US president ever with no political or military experience". Vox. Retrieved January 3, 2017.
Phillips, Amber (November 9, 2016). "Republicans are poised to grasp the holy grail of governance". The Washington Post. Retrieved October 2, 2021.
Halpert, Madeline (December 13, 2024). "Donald Trump named Time Person of the Year again". BBC.
Logan, Brian; Sanchez, Chris (November 10, 2016). "Protests against Donald Trump break out nationwide". Business Insider. Retrieved September 16, 2022.
Mele, Christopher; Correal, Annie (November 9, 2016). "'Not Our President': Protests Spread After Donald Trump's Election". The New York Times. Retrieved May 10, 2024.
Przybyla, Heidi M.; Schouten, Fredreka (January 21, 2017). "At 2.6 million strong, Women's Marches crush expectations". USA Today. Retrieved January 22, 2017.
Quigley, Aidan (January 25, 2017). "All of Trump's executive actions so far". Politico. Retrieved January 28, 2017.
V.V.B (March 31, 2017). "Ivanka Trump's new job". The Economist. Retrieved April 3, 2017.
Schmidt, Michael S.; Lipton, Eric; Savage, Charlie (January 21, 2017). "Jared Kushner, Trump's Son-in-Law, Is Cleared to Serve as Adviser". The New York Times. Retrieved May 7, 2017.
Geewax, Marilyn (January 20, 2018). "Trump Has Revealed Assumptions About Handling Presidential Wealth, Businesses". NPR. Retrieved October 2, 2021.
"Donald Trump: A list of potential conflicts of interest". BBC. April 18, 2017. Retrieved October 2, 2021.
Yourish, Karen; Buchanan, Larry (January 12, 2017). "It 'Falls Short in Every Respect': Ethics Experts Pan Trump's Conflicts Plan". The New York Times. Retrieved September 3, 2024.
Venook, Jeremy (August 9, 2017). "Trump's Interests vs. America's, Dubai Edition". The Atlantic. Retrieved October 2, 2021.
Stone, Peter (July 19, 2019). "How Trump's businesses are booming with lobbyists, donors and governments". The Guardian. Retrieved November 20, 2024.
In Focus: The Emoluments Clauses of the U.S. Constitution (PDF). Congressional Research Service (Report). August 19, 2020. Retrieved October 2, 2021.
LaFraniere, Sharon (January 25, 2018). "Lawsuit on Trump Emoluments Violations Gains Traction in Court". The New York Times. Retrieved January 25, 2018.
de Vogue, Ariane; Cole, Devan (January 25, 2021). "Supreme Court dismisses emoluments cases against Trump". CNN. Retrieved September 14, 2022.
Van Dam, Andrew (January 8, 2021). "Trump will have the worst jobs record in modern U.S. history. It's not just the pandemic". The Washington Post. Retrieved October 2, 2021.
Smialek, Jeanna (June 8, 2020). "The U.S. Entered a Recession in February". The New York Times. Retrieved June 10, 2020.
Long, Heather (December 15, 2017). "The final GOP tax bill is complete. Here's what is in it". The Washington Post. Retrieved July 31, 2021.
Andrews, Wilson; Parlapiano, Alicia (December 15, 2017). "What's in the Final Republican Tax Bill". The New York Times. Retrieved December 22, 2017.
Gale, William G. (February 14, 2020). "Did the 2017 tax cut—the Tax Cuts and Jobs Act—pay for itself?". Brookings Institution. Retrieved July 31, 2021.
Long, Heather; Stein, Jeff (October 25, 2019). "The U.S. deficit hit $984 billion in 2019, soaring during Trump era". The Washington Post. Retrieved June 10, 2020.
Sloan, Allan; Podkul, Cezary (January 14, 2021). "Donald Trump Built a National Debt So Big (Even Before the Pandemic) That It'll Weigh Down the Economy for Years". ProPublica. Retrieved October 3, 2021.
Bliss, Laura (November 16, 2020). "How Trump's $1 Trillion Infrastructure Pledge Added Up". Bloomberg News. Retrieved December 29, 2021.
Burns, Dan (January 8, 2021). "Trump ends his term like a growing number of Americans: out of a job". Reuters. Retrieved May 10, 2024.
Parker, Ashley; Davenport, Coral (May 26, 2016). "Donald Trump's Energy Plan: More Fossil Fuels and Fewer Rules". The New York Times. Retrieved October 3, 2021.
Samenow, Jason (March 22, 2016). "Donald Trump's unsettling nonsense on weather and climate". The Washington Post. Retrieved October 3, 2021.
Lemire, Jonathan; Madhani, Aamer; Weissert, Will; Knickmeyer, Ellen (September 15, 2020). "Trump spurns science on climate: 'Don't think science knows'". AP News. Retrieved May 11, 2024.
Plumer, Brad; Davenport, Coral (December 28, 2019). "Science Under Attack: How Trump Is Sidelining Researchers and Their Work". The New York Times. Retrieved May 11, 2024.
"Trump proposes cuts to climate and clean-energy programs". National Geographic Society. May 3, 2019. Retrieved November 24, 2023.
Dennis, Brady (November 7, 2017). "As Syria embraces Paris climate deal, it's the United States against the world". The Washington Post. Retrieved May 28, 2018.
Gardner, Timothy (December 3, 2019). "Senate confirms Brouillette, former Ford lobbyist, as energy secretary". Reuters. Retrieved December 15, 2019.
Brown, Matthew (September 15, 2020). "Trump's fossil fuel agenda gets pushback from federal judges". AP News. Retrieved October 3, 2021.
Lipton, Eric (October 5, 2020). "'The Coal Industry Is Back,' Trump Proclaimed. It Wasn't". The New York Times. Retrieved October 3, 2021.
Subramaniam, Tara (January 30, 2021). "From building the wall to bringing back coal: Some of Trump's more notable broken promises". CNN. Retrieved October 3, 2021.
Popovich, Nadja; Albeck-Ripka, Livia; Pierre-Louis, Kendra (January 20, 2021). "The Trump Administration Rolled Back More Than 100 Environmental Rules. Here's the Full List". The New York Times. Retrieved December 21, 2023.
Plumer, Brad (January 30, 2017). "Trump wants to kill two old regulations for every new one issued. Sort of". Vox. Retrieved March 11, 2023.
Thompson, Frank W. (October 9, 2020). "Six ways Trump has sabotaged the Affordable Care Act". Brookings Institution. Retrieved January 3, 2022.
Arnsdorf, Isaac; DePillis, Lydia; Lind, Dara; Song, Lisa; Syed, Moiz; Osei, Zipporah (November 25, 2020). "Tracking the Trump Administration's "Midnight Regulations"". ProPublica. Retrieved January 3, 2022.
Poydock, Margaret (September 17, 2020). "President Trump has attacked workers' safety, wages, and rights since Day One". Economic Policy Institute. Retrieved January 3, 2022.
Baker, Cayli (December 15, 2020). "The Trump administration's major environmental deregulations". Brookings Institution. Retrieved January 29, 2022.
Grunwald, Michael (April 10, 2017). "Trump's Secret Weapon Against Obama's Legacy". Politico Magazine. Retrieved January 29, 2022.
Lipton, Eric; Appelbaum, Binyamin (March 5, 2017). "Leashes Come Off Wall Street, Gun Sellers, Polluters and More". The New York Times. Retrieved January 29, 2022.
"Trump-Era Trend: Industries Protest. Regulations Rolled Back. A Dozen Examples". The New York Times. March 5, 2017. Retrieved January 29, 2022 – via DocumentCloud.
"Roundup: Trump-Era Agency Policy in the Courts". Institute for Policy Integrity. April 25, 2022. Retrieved January 8, 2022.
Kodjak, Alison (November 9, 2016). "Trump Can Kill Obamacare With Or Without Help From Congress". NPR. Retrieved January 12, 2017.
Davis, Julie Hirschfeld; Pear, Robert (January 20, 2017). "Trump Issues Executive Order Scaling Back Parts of Obamacare". The New York Times. Retrieved January 23, 2017.
Luhby, Tami (October 13, 2017). "What's in Trump's health care executive order?". CNN. Retrieved October 14, 2017.
Nelson, Louis (July 18, 2017). "Trump says he plans to 'let Obamacare fail'". Politico. Retrieved September 29, 2017.
Young, Jeffrey (August 31, 2017). "Trump Ramps Up Obamacare Sabotage With Huge Cuts To Enrollment Programs". HuffPost. Retrieved September 29, 2017.
Stolberg, Sheryl Gay (June 26, 2020). "Trump Administration Asks Supreme Court to Strike Down Affordable Care Act". The New York Times. Retrieved October 3, 2021.
Katkov, Mark (June 26, 2020). "Obamacare Must 'Fall,' Trump Administration Tells Supreme Court". NPR. Retrieved September 29, 2021.
Rappeport, Alan; Haberman, Maggie (January 22, 2020). "Trump Opens Door to Cuts to Medicare and Other Entitlement Programs". The New York Times. Retrieved January 24, 2020.
Mann, Brian (October 29, 2020). "Opioid Crisis: Critics Say Trump Fumbled Response To Another Deadly Epidemic". NPR. Retrieved December 13, 2020.
"Abortion: How do Trump and Biden's policies compare?". BBC News. September 9, 2020. Retrieved July 17, 2023.
de Vogue, Ariane (November 15, 2016). "Trump: Same-sex marriage is 'settled', but Roe v Wade can be changed". CNN. Retrieved November 30, 2016.
O'Hara, Mary Emily (March 30, 2017). "LGBTQ Advocates Say Trump's New Executive Order Makes Them Vulnerable to Discrimination". NBC News. Retrieved July 30, 2017.
Luthi, Susannah (August 17, 2020). "Judge halts Trump's rollback of transgender health protections". Politico. Retrieved November 8, 2023.
Krieg, Gregory (June 20, 2016). "The times Trump changed his positions on guns". CNN. Retrieved October 3, 2021.
Dawsey, Josh (November 1, 2019). "Trump abandons proposing ideas to curb gun violence after saying he would following mass shootings". The Washington Post. Retrieved October 3, 2021.
Bures, Brendan (February 21, 2020). "Trump administration doubles down on anti-marijuana position". Chicago Tribune. Retrieved October 3, 2021.
Wolf, Zachary B. (July 27, 2019). "Trump returns to the death penalty as Democrats turn against it". CNN. Retrieved September 18, 2022.
Honderich, Holly (January 16, 2021). "In Trump's final days, a rush of federal executions". BBC. Retrieved September 18, 2022.
Tarm, Michael; Kunzelman, Michael (January 15, 2021). "Trump administration carries out 13th and final execution". AP News. Retrieved January 30, 2022.
McCarthy, Tom (February 7, 2016). "Donald Trump: I'd bring back 'a hell of a lot worse than waterboarding'". The Guardian. Retrieved February 8, 2016.
"Ted Cruz, Donald Trump Advocate Bringing Back Waterboarding". ABC News. February 6, 2016. Retrieved February 9, 2016.
Kessler, Glenn (May 8, 2020). "The 'very fine people' at Charlottesville: Who were they?". The Washington Post. Retrieved October 23, 2021.
Beauchamp, Zack (January 11, 2018). "Trump's "shithole countries" comment exposes the core of Trumpism". Vox. Retrieved January 11, 2018.
Wintour, Patrick; Burke, Jason; Livsey, Anna (January 13, 2018). "'There's no other word but racist': Trump's global rebuke for 'shithole' remark". The Guardian. Retrieved January 13, 2018.
Rogers, Katie; Fandos, Nicholas (July 14, 2019). "Trump Tells Congresswomen to 'Go Back' to the Countries They Came From". The New York Times. Retrieved September 30, 2021.
Mak, Tim (July 16, 2019). "House Votes To Condemn Trump's 'Racist Comments'". NPR. Retrieved July 17, 2019.
Simon, Mallory; Sidner, Sara (July 16, 2019). "Trump said 'many people agree' with his racist tweets. These white supremacists certainly do". CNN. Retrieved July 20, 2019.
Choi, Matthew (September 22, 2020). "'She's telling us how to run our country': Trump again goes after Ilhan Omar's Somali roots". Politico. Retrieved October 12, 2021.
Leonnig, Carol D.; Zapotosky, Matt; Dawsey, Josh; Tan, Rebecca (June 2, 2020). "Barr personally ordered removal of protesters near White House, leading to use of force against largely peaceful crowd". The Washington Post. Retrieved June 3, 2020.
Bump, Philip (June 2, 2020). "Timeline: The clearing of Lafayette Square". The Washington Post. Retrieved June 6, 2020.
Gittleson, Ben; Phelps, Jordyn (June 3, 2020). "Police use munitions to forcibly push back peaceful protesters for Trump church visit". ABC News. Retrieved June 29, 2021.
O'Neil, Luke (June 2, 2020). "What do we know about Trump's love for the Bible?". The Guardian. Retrieved June 11, 2020.
Stableford, Dylan; Wilson, Christopher (June 3, 2020). "Religious leaders condemn teargassing protesters to clear street for Trump". Yahoo! News. Retrieved June 8, 2020.
"Scores of retired military leaders publicly denounce Trump". AP News. June 6, 2020. Retrieved June 8, 2020.
Gramlich, John (January 22, 2021). "Trump used his clemency power sparingly despite a raft of late pardons and commutations". Pew Research Center. Retrieved July 23, 2023.
Vogel, Kenneth P. (March 21, 2021). "The Road to Clemency From Trump Was Closed to Most Who Sought It". The New York Times. Retrieved July 23, 2023.
Olorunnipa, Toluse; Dawsey, Josh (December 24, 2020). "Trump wields pardon power as political weapon, rewarding loyalists and undermining prosecutors". The Washington Post. Retrieved October 3, 2021.
Johnson, Kevin; Jackson, David; Wagner, Dennis (January 19, 2021). "Donald Trump grants clemency to 144 people (not himself or family members) in final hours". USA Today. Retrieved July 23, 2023.
Phillips, Dave (November 22, 2019). "Trump Clears Three Service Members in War Crimes Cases". The New York Times. Retrieved April 18, 2024.
Fritze, John (August 8, 2019). "A USA Today analysis found Trump used words like 'invasion' and 'killer' at rallies more than 500 times since 2017". USA Today. Retrieved August 9, 2019.
Johnson 2017a.
Johnson & Cuison-Villazor 2019.
Mitchell, Ellen (January 29, 2019). "Pentagon to send a 'few thousand' more troops to southern border". The Hill. Retrieved June 4, 2020.
Snow, Anita (February 25, 2020). "Crackdown on immigrants who use public benefits takes effect". AP News. Retrieved June 4, 2020.
"Donald Trump has cut refugee admissions to America to a record low". The Economist. November 4, 2019. Retrieved June 25, 2020.
Kanno-Youngs, Zolan; Shear, Michael D. (October 1, 2020). "Trump Virtually Cuts Off Refugees as He Unleashes a Tirade on Immigrants". The New York Times. Retrieved September 30, 2021.
Hesson, Ted (October 11, 2019). "Trump ending U.S. role as worldwide leader on refugees". Politico. Retrieved June 25, 2020.
Pilkington, Ed (December 8, 2015). "Donald Trump: ban all Muslims entering US". The Guardian. Retrieved October 10, 2020.
Johnson, Jenna (June 25, 2016). "Trump now proposes only Muslims from terrorism-heavy countries would be banned from U.S." The Washington Post. Retrieved October 3, 2021.
Walters, Joanna; Helmore, Edward; Dehghan, Saeed Kamali (January 28, 2017). "US airports on frontline as Donald Trump's travel ban causes chaos and protests". The Guardian. Retrieved July 19, 2017.
"Protests erupt at airports nationwide over immigration action". CBS News. January 28, 2017. Retrieved March 22, 2021.
Barrett, Devlin; Frosch, Dan (February 4, 2017). "Federal Judge Temporarily Halts Trump Order on Immigration, Refugees". The Wall Street Journal. Retrieved October 3, 2021.
Levine, Dan; Rosenberg, Mica (March 15, 2017). "Hawaii judge halts Trump's new travel ban before it can go into effect". Reuters. Retrieved October 3, 2021.
"Trump signs new travel ban directive". BBC News. March 6, 2017. Retrieved March 18, 2017.
Sherman, Mark (June 26, 2017). "Limited version of Trump's travel ban to take effect Thursday". Chicago Tribune. Associated Press. Retrieved August 5, 2017.
Laughland, Oliver (September 25, 2017). "Trump travel ban extended to blocks on North Korea, Venezuela and Chad". The Guardian. Retrieved October 13, 2017.
Hurley, Lawrence (December 4, 2017). "Supreme Court lets Trump's latest travel ban go into full effect". Reuters. Retrieved October 3, 2021.
Wagner, Meg; Ries, Brian; Rocha, Veronica (June 26, 2018). "Supreme Court upholds travel ban". CNN. Retrieved June 26, 2018.
Pearle, Lauren (February 5, 2019). "Trump administration admits thousands more migrant families may have been separated than estimated". ABC News. Retrieved May 30, 2020.
Spagat, Elliot (October 25, 2019). "Tally of children split at border tops 5,400 in new count". AP News. Retrieved May 30, 2020.
Davis, Julie Hirschfeld; Shear, Michael D. (June 16, 2018). "How Trump Came to Enforce a Practice of Separating Migrant Families". The New York Times. Retrieved May 30, 2020.
Savage, Charlie (June 20, 2018). "Explaining Trump's Executive Order on Family Separation". The New York Times. Retrieved May 30, 2020.
Domonoske, Camila; Gonzales, Richard (June 19, 2018). "What We Know: Family Separation And 'Zero Tolerance' At The Border". NPR. Retrieved May 30, 2020.
Epstein, Jennifer (June 18, 2018). "Donald Trump's family separations bedevil GOP as public outrage grows". Bloomberg News. Retrieved May 30, 2020 – via The Sydney Morning Herald.
Sarlin, Benjy (June 15, 2018). "Despite claims, GOP immigration bill would not end family separation, experts say". NBC News. Retrieved June 18, 2018.
Davis, Julie Hirschfeld; Nixon, Ron (May 29, 2018). "Trump Officials, Moving to Break Up Migrant Families, Blame Democrats". The New York Times. Retrieved December 29, 2020.
Beckwith, Ryan Teague (June 20, 2018). "Here's What President Trump's Immigration Order Actually Does". Time. Retrieved May 30, 2020.
Shear, Michael D.; Goodnough, Abby; Haberman, Maggie (June 20, 2018). "Trump Retreats on Separating Families, but Thousands May Remain Apart". The New York Times. Retrieved June 20, 2018.
Hansler, Jennifer (June 27, 2018). "Judge says government does a better job of tracking 'personal property' than separated kids". CNN. Retrieved May 30, 2020.
Walters, Joanna (June 27, 2018). "Judge orders US to reunite families separated at border within 30 days". The Guardian. Retrieved May 30, 2020.
Timm, Jane C. (January 13, 2021). "Fact check: Mexico never paid for it. But what about Trump's other border wall promises?". NBC News. Retrieved December 21, 2021.
Farley, Robert (February 16, 2021). "Trump's Border Wall: Where Does It Stand?". FactCheck.org. Retrieved December 21, 2021.
Davis, Julie Hirschfeld; Tackett, Michael (January 2, 2019). "Trump and Democrats Dig in After Talks to Reopen Government Go Nowhere". The New York Times. Retrieved January 3, 2019.
Gambino, Lauren; Walters, Joanna (January 26, 2019). "Trump signs bill to end $6bn shutdown and temporarily reopen government". The Guardian. Reuters. Retrieved May 31, 2020.
Pramuk, Jacob (January 25, 2019). "Trump signs bill to temporarily reopen government after longest shutdown in history". CNBC. Retrieved May 31, 2020.
Fritze, John (January 24, 2019). "By the numbers: How the government shutdown is affecting the US". USA Today. Retrieved May 31, 2020.
Mui, Ylan (January 28, 2019). "The government shutdown cost the economy $11 billion, including a permanent $3 billion loss, Congressional Budget Office says". CNBC. Retrieved May 31, 2020.
Bacon, Perry Jr. (January 25, 2019). "Why Trump Blinked". FiveThirtyEight. Retrieved October 3, 2021.
Pramuk, Jacob; Wilkie, Christina (February 15, 2019). "Trump declares national emergency to build border wall, setting up massive legal fight". CNBC. Retrieved May 31, 2020.
Carney, Jordain (October 17, 2019). "Senate fails to override Trump veto over emergency declaration". The Hill. Retrieved May 31, 2020.
Quinn, Melissa (December 11, 2019). "Supreme Court allows Trump to use military funds for border wall construction". CBS News. Retrieved September 19, 2022.
Trump v. Sierra Club, No. 19A60, 588 U.S. ___ (2019)
Allyn, Bobby (January 9, 2020). "Appeals Court Allows Trump To Divert $3.6 Billion In Military Funds For Border Wall". NPR. Retrieved September 19, 2022.
El Paso Cty. v. Trump, 982 F.3d 332 (5th Cir. December 4, 2020).
Cummings, William (October 24, 2018). "'I am a nationalist': Trump's embrace of controversial label sparks uproar". USA Today. Retrieved August 24, 2021.
Bennhold, Katrin (June 6, 2020). "Has 'America First' Become 'Trump First'? Germans Wonder". The New York Times. Retrieved August 24, 2021.
Carothers, Thomas; Brown, Frances Z. (October 1, 2018). "Can U.S. Democracy Policy Survive Trump?". Carnegie Endowment for International Peace. Retrieved October 19, 2019.
McGurk 2020.
Swanson, Ana (March 12, 2020). "Trump Administration Escalates Tensions With Europe as Crisis Looms". The New York Times. Retrieved October 4, 2021.
Baker, Peter (May 26, 2017). "Trump Says NATO Allies Don't Pay Their Share. Is That True?". The New York Times. Retrieved October 4, 2021.
Barnes, Julian E.; Cooper, Helene (January 14, 2019). "Trump Discussed Pulling U.S. From NATO, Aides Say Amid New Concerns Over Russia". The New York Times. Retrieved April 5, 2021.
Bradner, Eric (January 23, 2017). "Trump's TPP withdrawal: 5 things to know". CNN. Retrieved March 12, 2018.
Inman, Phillip (March 10, 2018). "The war over steel: Trump tips global trade into new turmoil". The Guardian. Retrieved March 15, 2018.
Lawder, David; Blanchard, Ben (June 15, 2018). "Trump sets tariffs on $50 billion in Chinese goods; Beijing strikes back". Reuters. Retrieved October 3, 2021.
Singh, Rajesh Kumar (August 2, 2019). "Explainer: Trump's China tariffs – Paid by U.S. importers, not by China". Reuters. Retrieved November 27, 2022.
Palmer, Doug (February 5, 2021). "America's trade gap soared under Trump, final figures show". Politico. Retrieved June 1, 2024.
Rodriguez, Sabrina (April 24, 2020). "North American trade deal to take effect on July 1". Politico. Retrieved January 31, 2022.
Zengerle, Patricia (January 16, 2019). "Bid to keep U.S. sanctions on Russia's Rusal fails in Senate". Reuters. Retrieved October 5, 2021.
Whalen, Jeanne (January 15, 2019). "In rare rebuke of Trump administration, some GOP lawmakers advance measure to oppose lifting Russian sanctions". The Washington Post. Retrieved October 5, 2021.
Bugos, Shannon (September 2019). "U.S. Completes INF Treaty Withdrawal". Arms Control Association. Retrieved October 5, 2021.
Panetta, Grace (June 14, 2018). "Trump reportedly claimed to leaders at the G7 that Crimea is part of Russia because everyone there speaks Russian". Business Insider. Retrieved February 13, 2020.
Baker, Peter (August 10, 2017). "Trump Praises Putin Instead of Critiquing Cuts to U.S. Embassy Staff". The New York Times. Retrieved June 7, 2020.
Nussbaum, Matthew (April 8, 2018). "Trump blames Putin for backing 'Animal Assad'". Politico. Retrieved October 5, 2021.
"Nord Stream 2: Trump approves sanctions on Russia gas pipeline". BBC News. December 21, 2019. Retrieved October 5, 2021.
Diamond, Jeremy; Malloy, Allie; Dewan, Angela (March 26, 2018). "Trump expelling 60 Russian diplomats in wake of UK nerve agent attack". CNN. Retrieved October 5, 2021.
Zurcher, Anthony (July 16, 2018). "Trump-Putin summit: After Helsinki, the fallout at home". BBC. Retrieved July 18, 2018.
Calamur, Krishnadev (July 16, 2018). "Trump Sides With the Kremlin, Against the U.S. Government". The Atlantic. Retrieved July 18, 2018.
Fox, Lauren (July 16, 2018). "Top Republicans in Congress break with Trump over Putin comments". CNN. Retrieved July 18, 2018.
Bose, Nandita; Shalal, Andrea (August 7, 2019). "Trump says China is 'killing us with unfair trade deals'". Reuters. Retrieved August 24, 2019.
Hass, Ryan; Denmark, Abraham (August 7, 2020). "More pain than gain: How the US-China trade war hurt America". Brookings Institution. Retrieved October 4, 2021.
"How China Won Trump's Trade War and Got Americans to Foot the Bill". Bloomberg News. January 11, 2021. Retrieved October 4, 2021.
Bajak, Frank; Liedtke, Michael (May 21, 2019). "Huawei sanctions: Who gets hurt in dispute?". USA Today. Retrieved August 24, 2019.
"Trump's Trade War Targets Chinese Students at Elite U.S. Schools". Time. June 3, 2019. Retrieved August 24, 2019.
Meredith, Sam (August 6, 2019). "China responds to US after Treasury designates Beijing a 'currency manipulator'". CNBC. Retrieved August 6, 2019.
Sink, Justin (April 11, 2018). "Trump Praises China's Xi's Trade Speech, Easing Tariff Tensions". IndustryWeek. Retrieved October 5, 2021.
Nakamura, David (August 23, 2019). "Amid trade war, Trump drops pretense of friendship with China's Xi Jinping, calls him an 'enemy'". The Washington Post. Retrieved October 25, 2020.
Mason, Jeff; Spetalnick, Matt; Alper, Alexandra (March 18, 2020). "Trump ratchets up criticism of China over coronavirus". Reuters. Retrieved October 25, 2020.
"Trump held off sanctioning Chinese over Uighurs to pursue trade deal". BBC News. June 22, 2020. Retrieved October 5, 2021.
Verma, Pranshu; Wong, Edward (July 9, 2020). "U.S. Imposes Sanctions on Chinese Officials Over Mass Detention of Muslims". The New York Times. Retrieved October 5, 2021.
Taylor, Adam; Meko, Tim (December 21, 2017). "What made North Korea's weapons programs so much scarier in 2017". The Washington Post. Retrieved July 5, 2019.
Windrem, Robert; Siemaszko, Corky; Arkin, Daniel (May 2, 2017). "North Korea crisis: How events have unfolded under Trump". NBC News. Retrieved June 8, 2020.
Borger, Julian (September 19, 2017). "Donald Trump threatens to 'totally destroy' North Korea in UN speech". The Guardian. Retrieved June 8, 2020.
McCausland, Phil (September 22, 2017). "Kim Jong Un Calls President Trump 'Dotard' and 'Frightened Dog'". NBC News. Retrieved June 8, 2020.
"Transcript: Kim Jong Un's letters to President Trump". CNN. September 9, 2020. Retrieved October 5, 2021.
Gangel, Jamie; Herb, Jeremy (September 9, 2020). "'A magical force': New Trump-Kim letters provide window into their 'special friendship'". CNN. Retrieved October 5, 2021.
Rappeport, Alan (March 22, 2019). "Trump Overrules Own Experts on Sanctions, in Favor to North Korea". The New York Times. Retrieved September 30, 2021.
Baker, Peter; Crowley, Michael (June 30, 2019). "Trump Steps Into North Korea and Agrees With Kim Jong-un to Resume Talks". The New York Times. Retrieved October 5, 2021.
Sanger, David E.; Sang-Hun, Choe (June 12, 2020). "Two Years After Trump-Kim Meeting, Little to Show for Personal Diplomacy". The New York Times. Retrieved October 5, 2021.
Tanner, Jari; Lee, Matthew (October 5, 2019). "North Korea Says Nuclear Talks Break Down While U.S. Says They Were 'Good'". AP News. Retrieved July 21, 2021.
Herskovitz, Jon (December 28, 2020). "Kim Jong Un's Nuclear Weapons Got More Dangerous Under Trump". Bloomberg News. Retrieved October 5, 2021.
Warrick, Joby; Denyer, Simon (September 30, 2020). "As Kim wooed Trump with 'love letters', he kept building his nuclear capability, intelligence shows". The Washington Post. Retrieved October 5, 2021.
Jaffe, Greg; Ryan, Missy (January 21, 2018). "Up to 1,000 more U.S. troops could be headed to Afghanistan this spring". The Washington Post. Retrieved October 4, 2021.
Gordon, Michael R.; Schmitt, Eric; Haberman, Maggie (August 20, 2017). "Trump Settles on Afghan Strategy Expected to Raise Troop Levels". The New York Times. Retrieved October 4, 2021.
George, Susannah; Dadouch, Sarah; Lamothe, Dan (February 29, 2020). "U.S. signs peace deal with Taliban agreeing to full withdrawal of American troops from Afghanistan". The Washington Post. Retrieved October 4, 2021.
Mashal, Mujib (February 29, 2020). "Taliban and U.S. Strike Deal to Withdraw American Troops From Afghanistan". The New York Times. Retrieved December 29, 2020.
Kiely, Eugene; Farley, Robert (August 17, 2021). "Timeline of U.S. Withdrawal from Afghanistan". FactCheck.org. Retrieved August 31, 2021.
Sommer, Allison Kaplan (July 25, 2019). "How Trump and Netanyahu Became Each Other's Most Effective Political Weapon". Haaretz. Retrieved August 2, 2019.
Nelson, Louis; Nussbaum, Matthew (December 6, 2017). "Trump says U.S. recognizes Jerusalem as Israel's capital, despite global condemnation". Politico. Retrieved December 6, 2017.
Romo, Vanessa (March 25, 2019). "Trump Formally Recognizes Israeli Sovereignty Over Golan Heights". NPR. Retrieved April 5, 2021.
Gladstone, Rick; Landler, Mark (December 21, 2017). "Defying Trump, U.N. General Assembly Condemns U.S. Decree on Jerusalem". The New York Times. Retrieved December 21, 2017.
Huet, Natalie (March 22, 2019). "Outcry as Trump backs Israeli sovereignty over Golan Heights". Euronews. Reuters. Retrieved October 4, 2021.
Crowley, Michael (September 15, 2020). "Israel, U.A.E. and Bahrain Sign Accords, With an Eager Trump Playing Host". The New York Times. Retrieved February 9, 2024.
Phelps, Jordyn; Struyk, Ryan (May 20, 2017). "Trump signs $110 billion arms deal with Saudi Arabia on 'a tremendous day'". ABC News. Retrieved July 6, 2018.
Holland, Steve; Bayoumy, Yara (March 20, 2018). "Trump praises U.S. military sales to Saudi as he welcomes crown prince". Reuters. Retrieved June 2, 2021.
Chiacu, Doina; Ali, Idrees (March 21, 2018). "Trump, Saudi leader discuss Houthi 'threat' in Yemen: White House". Reuters. Retrieved June 2, 2021.
Stewart, Phil; Ali, Idrees (October 11, 2019). "U.S. says deploying more forces to Saudi Arabia to counter Iran threat". Reuters. Retrieved October 4, 2021.
"Syria war: Trump's missile strike attracts US praise – and barbs". BBC News. April 7, 2017. Retrieved April 8, 2017.
Joyce, Kathleen (April 14, 2018). "US strikes Syria after suspected chemical attack by Assad regime". Fox News. Retrieved April 14, 2018.
Landler, Mark; Cooper, Helene; Schmitt, Eric (December 19, 2018). "Trump withdraws U.S. Forces From Syria, Declaring 'We Have Won Against ISIS'". The New York Times. Retrieved December 31, 2018.
Borger, Julian; Chulov, Martin (December 20, 2018). "Trump shocks allies and advisers with plan to pull US troops out of Syria". The Guardian. Retrieved December 20, 2018.
Cooper, Helene (December 20, 2018). "Jim Mattis, Defense Secretary, Resigns in Rebuke of Trump's Worldview". The New York Times. Retrieved December 21, 2018.
McKernan, Bethan; Borger, Julian; Sabbagh, Dan (October 9, 2019). "Turkey launches military operation in northern Syria". The Guardian. Retrieved September 28, 2021.
O'Brien, Connor (October 16, 2019). "House condemns Trump's Syria withdrawal". Politico. Retrieved October 17, 2019.
Edmondson, Catie (October 16, 2019). "In Bipartisan Rebuke, House Majority Condemns Trump for Syria Withdrawal". The New York Times. Retrieved October 17, 2019.
Lederman, Josh; Lucey, Catherine (May 8, 2018). "Trump declares US leaving 'horrible' Iran nuclear accord". AP News. Retrieved May 8, 2018.
Landler, Mark (May 8, 2018). "Trump Abandons Iran Nuclear Deal He Long Scorned". The New York Times. Retrieved October 4, 2021.
Nichols, Michelle (February 18, 2021). "U.S. rescinds Trump White House claim that all U.N. sanctions had been reimposed on Iran". Reuters. Retrieved December 14, 2021.
Hennigan, W.J. (November 24, 2021). "'They're Very Close.' U.S. General Says Iran Is Nearly Able to Build a Nuclear Weapon". Time. Retrieved December 18, 2021.
Donevan, Connor; Dorning, Courtney; Kelly, Mary Louise (May 30, 2023). "5 years after U.S. left Iran nuclear deal, more enriched Uranium and much less trust". NPR. Retrieved November 3, 2024.
Crowley, Michael; Hassan, Falih; Schmitt, Eric (January 2, 2020). "U.S. Strike in Iraq Kills Qassim Suleimani, Commander of Iranian Forces". The New York Times. Retrieved January 3, 2020.
Baker, Peter; Bergman, Ronen; Kirkpatrick, David D.; Barnes, Julian E.; Rubin, Alissa J. (January 11, 2020). "Seven Days in January: How Trump Pushed U.S. and Iran to the Brink of War". The New York Times. Retrieved November 8, 2022.
Horton, Alex; Lamothe, Dan (December 8, 2021). "Army awards more Purple Hearts for troops hurt in Iranian attack that Trump downplayed". The Washington Post. Retrieved November 8, 2021.
Trimble, Megan (December 28, 2017). "Trump White House Has Highest Turnover in 40 Years". U.S. News & World Report. Retrieved March 16, 2018.
Wise, Justin (July 2, 2018). "AP: Trump admin sets record for White House turnover". The Hill. Retrieved July 3, 2018.
"Trump White House sets turnover records, analysis shows". NBC News. Associated Press. July 2, 2018. Retrieved July 3, 2018.
Keith, Tamara (March 7, 2018). "White House Staff Turnover Was Already Record-Setting. Then More Advisers Left". NPR. Retrieved March 16, 2018.
Tenpas, Kathryn Dunn; Kamarck, Elaine; Zeppos, Nicholas W. (March 16, 2018). "Tracking Turnover in the Trump Administration". Brookings Institution. Retrieved March 16, 2018.
Rogers, Katie; Karni, Annie (April 23, 2020). "Home Alone at the White House: A Sour President, With TV His Constant Companion". The New York Times. Retrieved May 5, 2020.
Cillizza, Chris (June 19, 2020). "Donald Trump makes terrible hires, according to Donald Trump". CNN. Retrieved June 24, 2020.
Keith, Tamara (March 6, 2020). "Mick Mulvaney Out, Mark Meadows in As White House Chief Of Staff". NPR. Retrieved October 5, 2021.
Baker, Peter; Haberman, Maggie (July 28, 2017). "Reince Priebus Pushed Out After Rocky Tenure as Trump Chief of Staff". The New York Times. Retrieved October 6, 2021.
Fritze, John; Subramanian, Courtney; Collins, Michael (September 4, 2020). "Trump says former chief of staff Gen. John Kelly couldn't 'handle the pressure' of the job". USA Today. Retrieved October 6, 2021.
Stanek, Becca (May 11, 2017). "President Trump just completely contradicted the official White House account of the Comey firing". The Week. Retrieved May 11, 2017.
Schmidt, Michael S.; Apuzzo, Matt (June 7, 2017). "Comey Says Trump Pressured Him to 'Lift the Cloud' of Inquiry". The New York Times. Retrieved November 2, 2021.
"Statement for the Record Senate Select Committee on Intelligence James B. Comey" (PDF). United States Senate Select Committee on Intelligence. June 8, 2017. p. 7. Retrieved November 2, 2021.
Jones-Rooy, Andrea (November 29, 2017). "The Incredibly And Historically Unstable First Year Of Trump's Cabinet". FiveThirtyEight. Retrieved March 16, 2018.
Hersher, Rebecca; Neely, Brett (July 5, 2018). "Scott Pruitt Out at EPA". NPR. Retrieved July 5, 2018.
Eilperin, Juliet; Dawsey, Josh; Fears, Darryl (December 15, 2018). "Interior Secretary Zinke resigns amid investigations". The Washington Post. Retrieved August 7, 2024.
Keith, Tamara (October 12, 2017). "Trump Leaves Top Administration Positions Unfilled, Says Hollow Government By Design". NPR. Retrieved March 16, 2018.
"Tracking how many key positions Trump has filled so far". The Washington Post. January 8, 2019. Retrieved October 6, 2021.
Gramlich, John (January 13, 2021). "How Trump compares with other recent presidents in appointing federal judges". Pew Research Center. Retrieved May 30, 2021.
Kumar, Anita (September 26, 2020). "Trump's legacy is now the Supreme Court". Politico.
Farivar, Masood (December 24, 2020). "Trump's Lasting Legacy: Conservative Supermajority on Supreme Court". Voice of America. Retrieved December 21, 2023.
Biskupic, Joan (June 2, 2023). "Nine Black Robes: Inside the Supreme Court's Drive to the Right and Its Historic Consequences". WBUR-FM. Retrieved December 21, 2023.
Quay, Grayson (June 25, 2022). "Trump takes credit for Dobbs decision but worries it 'won't help him in the future'". The Week. Retrieved October 2, 2023.
Kapur, Sahil (May 17, 2023). "Trump: 'I was able to kill Roe v. Wade'". NBC News. Retrieved December 21, 2023.
Phillip, Abby; Barnes, Robert; O'Keefe, Ed (February 8, 2017). "Supreme Court nominee Gorsuch says Trump's attacks on judiciary are 'demoralizing'". The Washington Post. Retrieved October 6, 2021.
Shepherd, Katie (November 8, 2019). "Trump 'violates all recognized democratic norms,' federal judge says in biting speech on judicial independence". The Washington Post. Retrieved October 6, 2021.
Holshue et al. 2020.
Hein, Alexandria (January 31, 2020). "Coronavirus declared public health emergency in US". Fox News. Retrieved October 2, 2020.
Cloud, David S.; Pringle, Paul; Stokols, Eli (April 19, 2020). "How Trump let the U.S. fall behind the curve on coronavirus threat". Los Angeles Times. Retrieved November 27, 2024.
Kelly, Caroline (March 21, 2020). "Washington Post: US intelligence warned Trump in January and February as he dismissed coronavirus threat". CNN. Retrieved April 21, 2020.
Watson, Kathryn (April 3, 2020). "A timeline of what Trump has said on coronavirus". CBS News. Retrieved January 27, 2021.
Gangel, Jamie; Herb, Jeremy; Stuart, Elizabeth (September 9, 2020). "'Play it down': Trump admits to concealing the true threat of coronavirus in new Woodward book". CNN. Retrieved September 14, 2022.
Partington, Richard; Wearden, Graeme (March 9, 2020). "Global stock markets post biggest falls since 2008 financial crisis". The Guardian. Retrieved March 15, 2020.
Heeb, Gina (March 6, 2020). "Trump signs emergency coronavirus package, injecting $8.3 billion into efforts to fight the outbreak". Business Insider. Retrieved October 6, 2021.
"WHO Director-General's opening remarks at the media briefing on COVID-19 – 11 March 2020". World Health Organization. March 11, 2020. Retrieved March 11, 2020.
"Coronavirus: What you need to know about Trump's Europe travel ban". The Local. March 12, 2020. Retrieved October 6, 2021.
Karni, Annie; Haberman, Maggie (March 12, 2020). "In Rare Oval Office Speech, Trump Voices New Concerns and Old Themes". The New York Times. Retrieved March 18, 2020.
Liptak, Kevin (March 13, 2020). "Trump declares national emergency – and denies responsibility for coronavirus testing failures". CNN. Retrieved March 18, 2020.
Valverde, Miriam (March 12, 2020). "Donald Trump's Wrong Claim That 'Anybody' Can Get Tested For Coronavirus". Kaiser Health News. Retrieved March 18, 2020.
Hulse, Carl; Cochrane, Emily (March 26, 2020). "As Coronavirus Spread, Largest Stimulus in History United a Polarized Senate". The New York Times. Retrieved November 27, 2024.
Taylor, Andrew; Fram, Alan; Kellman, Laurie; Superville, Darlene (March 28, 2020). "Trump signs $2.2T stimulus after swift congressional votes". AP News. Retrieved November 27, 2024.
"Trump's immigration executive order: What you need to know". Al Jazeera. April 23, 2020. Retrieved October 6, 2021.
Shear, Michael D.; Weiland, Noah; Lipton, Eric; Haberman, Maggie; Sanger, David E. (July 18, 2020). "Inside Trump's Failure: The Rush to Abandon Leadership Role on the Virus". The New York Times. Retrieved July 19, 2020.
"Trump creates task force to lead U.S. coronavirus response". CBS News. January 30, 2020. Retrieved October 10, 2020.
Karni, Annie (March 23, 2020). "In Daily Coronavirus Briefing, Trump Tries to Redefine Himself". The New York Times. Retrieved April 8, 2020.
Baker, Peter; Rogers, Katie; Enrich, David; Haberman, Maggie (April 6, 2020). "Trump's Aggressive Advocacy of Malaria Drug for Treating Coronavirus Divides Medical Community". The New York Times. Retrieved April 8, 2020.
Dale, Daniel (March 17, 2020). "Fact check: Trump tries to erase the memory of him downplaying the coronavirus". CNN. Retrieved March 19, 2020.
Georgiou, Aristos (March 19, 2020). "WHO expert condemns language stigmatizing coronavirus after Trump repeatedly calls it the 'Chinese virus'". Newsweek. Retrieved March 19, 2020.
Beavers, Olivia (March 19, 2020). "US-China relationship worsens over coronavirus". The Hill. Retrieved March 19, 2020.
Lemire, Jonathan (April 9, 2020). "As pandemic deepens, Trump cycles through targets to blame". AP News. Retrieved May 5, 2020.
"Coronavirus: Outcry after Trump suggests injecting disinfectant as treatment". BBC News. April 24, 2020. Retrieved August 11, 2020.
Aratani, Lauren (May 5, 2020). "Why is the White House winding down the coronavirus taskforce?". The Guardian. Retrieved June 8, 2020.
"Coronavirus: Trump says virus task force to focus on reopening economy". BBC News. May 6, 2020. Retrieved June 8, 2020.
Liptak, Kevin (May 6, 2020). "In reversal, Trump says task force will continue 'indefinitely' – eyes vaccine czar". CNN. Retrieved June 8, 2020.
Acosta, Jim; Liptak, Kevin; Westwood, Sarah (May 29, 2020). "As US deaths top 100,000, Trump's coronavirus task force is curtailed". CNN. Retrieved June 8, 2020.
Ollstein, Alice Miranda (April 14, 2020). "Trump halts funding to World Health Organization". Politico. Retrieved September 7, 2020.
Cohen, Zachary; Hansler, Jennifer; Atwood, Kylie; Salama, Vivian; Murray, Sara (July 7, 2020). "Trump administration begins formal withdrawal from World Health Organization". CNN. Retrieved July 19, 2020.
"Coronavirus: Trump moves to pull US out of World Health Organization". BBC News. July 7, 2020. Retrieved August 11, 2020.
Wood, Graeme (April 15, 2020). "The WHO Defunding Move Isn't What It Seems". The Atlantic. Retrieved September 7, 2020.
Phillips, Amber (April 8, 2020). "Why exactly is Trump lashing out at the World Health Organization?". The Washington Post. Retrieved September 8, 2020.
Wilson, Jason (April 17, 2020). "The rightwing groups behind wave of protests against Covid-19 restrictions". The Guardian. Retrieved April 18, 2020.
Andone, Dakin (April 16, 2020). "Protests Are Popping Up Across the US over Stay-at-Home Restrictions". CNN. Retrieved October 7, 2021.
Shear, Michael D.; Mervosh, Sarah (April 17, 2020). "Trump Encourages Protest Against Governors Who Have Imposed Virus Restrictions". The New York Times. Retrieved April 19, 2020.
Chalfant, Morgan; Samuels, Brett (April 20, 2020). "Trump support for protests threatens to undermine social distancing rules". The Hill. Retrieved July 10, 2020.
Lemire, Jonathan; Nadler, Ben (April 24, 2020). "Trump approved of Georgia's plan to reopen before bashing it". AP News. Retrieved April 28, 2020.
Kumar, Anita (April 18, 2020). "Trump's unspoken factor on reopening the economy: Politics". Politico. Retrieved July 10, 2020.
Danner, Chas (July 11, 2020). "99 Days Later, Trump Finally Wears a Face Mask in Public". New York. Retrieved July 12, 2020.
Blake, Aaron (June 25, 2020). "Trump's dumbfounding refusal to encourage wearing masks". The Washington Post. Retrieved July 10, 2020.
Higgins-Dunn, Noah (July 14, 2020). "Trump says U.S. would have half the number of coronavirus cases if it did half the testing". CNBC. Retrieved August 26, 2020.
Bump, Philip (July 23, 2020). "Trump is right that with lower testing, we record fewer cases. That's already happening". The Washington Post. Retrieved August 26, 2020.
Feuer, Will (August 26, 2020). "CDC quietly revises coronavirus guidance to downplay importance of testing for asymptomatic people". CNBC. Retrieved August 26, 2020.
"The C.D.C. changes testing guidelines to exclude those exposed to virus who don't exhibit symptoms". The New York Times. August 26, 2020. Retrieved August 26, 2020.
Valencia, Nick; Murray, Sara; Holmes, Kristen (August 26, 2020). "CDC was pressured 'from the top down' to change coronavirus testing guidance, official says". CNN. Retrieved August 26, 2020.
Gumbrecht, Jamie; Gupta, Sanjay; Valencia, Nick (September 18, 2020). "Controversial coronavirus testing guidance came from HHS and didn't go through CDC scientific review, sources say". CNN. Retrieved September 18, 2020.
Blake, Aaron (July 6, 2020). "President Trump, coronavirus truther". The Washington Post. Retrieved July 11, 2020.
Rabin, Roni Caryn; Cameron, Chris (July 5, 2020). "Trump Falsely Claims '99 Percent' of Virus Cases Are 'Totally Harmless'". The New York Times. Retrieved October 7, 2021.
Sprunt, Barbara (July 7, 2020). "Trump Pledges To 'Pressure' Governors To Reopen Schools Despite Health Concerns". NPR. Retrieved July 10, 2020.
McGinley, Laurie; Johnson, Carolyn Y. (June 15, 2020). "FDA pulls emergency approval for antimalarial drugs touted by Trump as covid-19 treatment". The Washington Post. Retrieved October 7, 2021.
LaFraniere, Sharon; Weiland, Noah; Shear, Michael D. (September 12, 2020). "Trump Pressed for Plasma Therapy. Officials Worry, Is an Unvetted Vaccine Next?". The New York Times. Retrieved September 13, 2020.
Diamond, Dan (September 11, 2020). "Trump officials interfered with CDC reports on Covid-19". Politico. Retrieved September 14, 2020.
Sun, Lena H. (September 12, 2020). "Trump officials seek greater control over CDC reports on coronavirus". The Washington Post. Retrieved September 14, 2020.
McGinley, Laurie; Johnson, Carolyn Y.; Dawsey, Josh (August 22, 2020). "Trump without evidence accuses 'deep state' at FDA of slow-walking coronavirus vaccines and treatments". The Washington Post. Retrieved October 7, 2021.
Liptak, Kevin; Klein, Betsy (October 5, 2020). "A timeline of Trump and those in his orbit during a week of coronavirus developments". CNN. Retrieved October 3, 2020.
Olorunnipa, Toluse; Dawsey, Josh (October 5, 2020). "Trump returns to White House, downplaying virus that hospitalized him and turned West Wing into a 'ghost town'". The Washington Post. Retrieved October 5, 2020.
Weiland, Noah; Haberman, Maggie; Mazzetti, Mark; Karni, Annie (February 11, 2021). "Trump Was Sicker Than Acknowledged With Covid-19". The New York Times. Retrieved February 16, 2021.
Acosta, J.; Kelly, C. (March 1, 2021). "Donald and Melania Trump received Covid vaccine at the White House in January". CNN. Retrieved November 8, 2024.
Edelman, Adam (July 5, 2020). "Warning signs flash for Trump in Wisconsin as pandemic response fuels disapproval". NBC News. Retrieved September 14, 2020.
Strauss, Daniel (September 7, 2020). "Biden aims to make election about Covid-19 as Trump steers focus elsewhere". The Guardian. Retrieved November 4, 2021.
Karson, Kendall (September 13, 2020). "Deep skepticism for Trump's coronavirus response endures: POLL". ABC News. Retrieved September 14, 2020.
Impelli, Matthew (October 26, 2020). "Fact Check: Is U.S. 'Rounding the Turn' On COVID, as Trump Claims?". Newsweek. Retrieved October 31, 2020.
Maan, Anurag (October 31, 2020). "U.S. reports world record of more than 100,000 COVID-19 cases in single day". Reuters. Retrieved October 31, 2020.
Woodward, Calvin; Pace, Julie (December 16, 2018). "Scope of investigations into Trump has shaped his presidency". AP News. Retrieved December 19, 2018.
Buchanan, Larry; Yourish, Karen (September 25, 2019). "Tracking 30 Investigations Related to Trump". The New York Times. Retrieved October 4, 2020.
Fahrenthold, David A.; Bade, Rachael; Wagner, John (April 22, 2019). "Trump sues in bid to block congressional subpoena of financial records". The Washington Post. Retrieved May 1, 2019.
Savage, Charlie (May 20, 2019). "Accountants Must Turn Over Trump's Financial Records, Lower-Court Judge Rules". The New York Times. Retrieved September 30, 2021.
Merle, Renae; Kranish, Michael; Sonmez, Felicia (May 22, 2019). "Judge rejects Trump's request to halt congressional subpoenas for his banking records". The Washington Post. Retrieved September 30, 2021.
Flitter, Emily; McKinley, Jesse; Enrich, David; Fandos, Nicholas (May 22, 2019). "Trump's Financial Secrets Move Closer to Disclosure". The New York Times. Retrieved September 30, 2021.
Hutzler, Alexandra (May 21, 2019). "Donald Trump's Subpoena Appeals Now Head to Merrick Garland's Court". Newsweek. Retrieved August 24, 2021.
Broadwater, Luke (September 17, 2022). "Trump's Former Accounting Firm Begins Turning Over Documents to Congress". The New York Times. Retrieved January 28, 2023.
Rosenberg, Matthew (July 6, 2017). "Trump Misleads on Russian Meddling: Why 17 Intelligence Agencies Don't Need to Agree". The New York Times. Retrieved October 7, 2021.
Sanger, David E. (January 6, 2017). "Putin Ordered 'Influence Campaign' Aimed at U.S. Election, Report Says". The New York Times. Retrieved October 4, 2021.
Berman, Russell (March 20, 2017). "It's Official: The FBI Is Investigating Trump's Links to Russia". The Atlantic. Retrieved June 7, 2017.
Harding, Luke (November 15, 2017). "How Trump walked into Putin's web". The Guardian. Retrieved May 22, 2019.
McCarthy, Tom (December 13, 2016). "Trump's relationship with Russia – what we know and what comes next". The Guardian. Retrieved March 11, 2017.
Bump, Philip (March 3, 2017). "The web of relationships between Team Trump and Russia". The Washington Post. Retrieved March 11, 2017.
Nesbit, Jeff (August 2, 2016). "Donald Trump's Many, Many, Many, Many Ties to Russia". Time. Retrieved February 28, 2017.
Phillips, Amber (August 19, 2016). "Paul Manafort's complicated ties to Ukraine, explained". The Washington Post. Retrieved June 14, 2017.
Graham, David A. (November 15, 2019). "We Still Don't Know What Happened Between Trump and Russia". The Atlantic. Retrieved October 7, 2021.
Parker, Ned; Landay, Jonathan; Strobel, Warren (May 18, 2017). "Exclusive: Trump campaign had at least 18 undisclosed contacts with Russians: sources". Reuters. Retrieved May 19, 2017.
Murray, Sara; Borger, Gloria; Diamond, Jeremy (February 14, 2017). "Flynn resigns amid controversy over Russia contacts". CNN. Retrieved March 2, 2017.
Harris, Shane; Dawsey, Josh; Nakashima, Ellen (September 27, 2019). "Trump told Russian officials in 2017 he wasn't concerned about Moscow's interference in U.S. election". The Washington Post. Retrieved October 8, 2021.
Barnes, Julian E.; Rosenberg, Matthew (November 22, 2019). "Charges of Ukrainian Meddling? A Russian Operation, U.S. Intelligence Says". The New York Times. Retrieved October 8, 2021.
Apuzzo, Matt; Goldman, Adam; Fandos, Nicholas (May 16, 2018). "Code Name Crossfire Hurricane: The Secret Origins of the Trump Investigation". The New York Times. Retrieved December 21, 2023.
Dilanian, Ken (September 7, 2020). "FBI agent who helped launch Russia investigation says Trump was 'compromised'". NBC News. Retrieved December 21, 2023.
Pearson, Nick (May 17, 2018). "Crossfire Hurricane: Trump Russia investigation started with Alexander Downer interview". Nine News. Retrieved December 21, 2023.
Schmidt, Michael S. (August 30, 2020). "Justice Dept. Never Fully Examined Trump's Ties to Russia, Ex-Officials Say". The New York Times. Retrieved October 8, 2021.
"Rosenstein to testify in Senate on Trump-Russia probe". Reuters. May 27, 2020. Retrieved October 19, 2021.
Vitkovskaya, Julie (June 16, 2017). "Trump Is Officially under Investigation. How Did We Get Here?". The Washington Post. Retrieved June 16, 2017.
Keating, Joshua (March 8, 2018). "It's Not Just a "Russia" Investigation Anymore". Slate. Retrieved October 8, 2021.
Haberman, Maggie; Schmidt, Michael S. (April 10, 2018). "Trump Sought to Fire Mueller in December". The New York Times. Retrieved October 8, 2021.
Breuninger, Kevin (March 22, 2019). "Mueller probe ends: Special counsel submits Russia report to Attorney General William Barr". CNBC. Retrieved March 22, 2019.
Barrett, Devlin; Zapotosky, Matt (April 30, 2019). "Mueller complained that Barr's letter did not capture 'context' of Trump probe". The Washington Post. Retrieved May 30, 2019.
Hsu, Spencer S.; Barrett, Devlin (March 5, 2020). "Judge cites Barr's 'misleading' statements in ordering review of Mueller report redactions". The Washington Post. Retrieved October 8, 2021.
Savage, Charlie (March 5, 2020). "Judge Calls Barr's Handling of Mueller Report 'Distorted' and 'Misleading'". The New York Times. Retrieved October 8, 2021.
Yen, Hope; Woodward, Calvin (July 24, 2019). "AP FACT CHECK: Trump falsely claims Mueller exonerated him". AP News. Retrieved October 8, 2021.
"Main points of Mueller report". Agence France-Presse. January 16, 2012. Archived from the original on April 20, 2019. Retrieved April 20, 2019.
Ostriker, Rebecca; Puzzanghera, Jim; Finucane, Martin; Datar, Saurabh; Uraizee, Irfan; Garvin, Patrick (April 18, 2019). "What the Mueller report says about Trump and more". The Boston Globe. Retrieved April 22, 2019.
Law, Tara (April 18, 2019). "Here Are the Biggest Takeaways From the Mueller Report". Time. Retrieved April 22, 2019.
Lynch, Sarah N.; Sullivan, Andy (April 18, 2018). "In unflattering detail, Mueller report reveals Trump actions to impede inquiry". Reuters. Retrieved July 10, 2022.
Mazzetti, Mark (July 24, 2019). "Mueller Warns of Russian Sabotage and Rejects Trump's 'Witch Hunt' Claims". The New York Times. Retrieved March 4, 2020.
Bump, Philip (May 30, 2019). "Trump briefly acknowledges that Russia aided his election – and falsely says he didn't help the effort". The Washington Post. Retrieved March 5, 2020.
Polantz, Katelyn; Kaufman, Ellie; Murray, Sara (June 19, 2020). "Mueller raised possibility Trump lied to him, newly unsealed report reveals". CNN. Retrieved October 30, 2022.
Barrett, Devlin; Zapotosky, Matt (April 17, 2019). "Mueller report lays out obstruction evidence against the president". The Washington Post. Retrieved April 20, 2019.
Farley, Robert; Robertson, Lori; Gore, D'Angelo; Spencer, Saranac Hale; Fichera, Angelo; McDonald, Jessica (April 18, 2019). "What the Mueller Report Says About Obstruction". FactCheck.org. Retrieved April 22, 2019.
Mascaro, Lisa (April 18, 2019). "Mueller drops obstruction dilemma on Congress". AP News. Retrieved April 20, 2019.
Segers, Grace (May 29, 2019). "Mueller: If it were clear president committed no crime, "we would have said so"". CBS News. Retrieved June 2, 2019.
Cheney, Kyle; Caygle, Heather; Bresnahan, John (December 10, 2019). "Why Democrats sidelined Mueller in impeachment articles". Politico. Retrieved October 8, 2021.
Blake, Aaron (December 10, 2019). "Democrats ditch 'bribery' and Mueller in Trump impeachment articles. But is that the smart play?". The Washington Post. Retrieved October 8, 2021.
Zapotosky, Matt; Bui, Lynh; Jackman, Tom; Barrett, Devlin (August 21, 2018). "Manafort convicted on 8 counts; mistrial declared on 10 others". The Washington Post. Retrieved August 21, 2018.
Mangan, Dan (July 30, 2018). "Trump and Giuliani are right that 'collusion is not a crime.' But that doesn't matter for Mueller's probe". CNBC. Retrieved October 8, 2021.
"Mueller investigation: No jail time sought for Trump ex-adviser Michael Flynn". BBC. December 5, 2018. Retrieved October 8, 2021.
Barrett, Devlin; Zapotosky, Matt; Helderman, Rosalind S. (November 29, 2018). "Michael Cohen, Trump's former lawyer, pleads guilty to lying to Congress about Moscow project". The Washington Post. Retrieved December 12, 2018.
Weiner, Rachel; Zapotosky, Matt; Jackman, Tom; Barrett, Devlin (February 20, 2020). "Roger Stone sentenced to three years and four months in prison, as Trump predicts 'exoneration' for his friend". The Washington Post. Retrieved March 3, 2020.
Bump, Philip (September 25, 2019). "Trump wanted Russia's main geopolitical adversary to help undermine the Russian interference story". The Washington Post. Retrieved October 1, 2019.
Cohen, Marshall; Polantz, Katelyn; Shortell, David; Kupperman, Tammy; Callahan, Michael (September 26, 2019). "Whistleblower says White House tried to cover up Trump's abuse of power". CNN. Retrieved October 4, 2022.
Fandos, Nicholas (September 24, 2019). "Nancy Pelosi Announces Formal Impeachment Inquiry of Trump". The New York Times. Retrieved October 8, 2021.
Forgey, Quint (September 24, 2019). "Trump changes story on withholding Ukraine aid". Politico. Retrieved October 1, 2019.
Graham, David A. (September 25, 2019). "Trump's Incriminating Conversation With the Ukrainian President". The Atlantic. Retrieved July 7, 2021.
Santucci, John; Mallin, Alexander; Thomas, Pierre; Faulders, Katherine (September 25, 2019). "Trump urged Ukraine to work with Barr and Giuliani to probe Biden: Call transcript". ABC News. Retrieved October 1, 2019.
"Document: Read the Whistle-Blower Complaint". The New York Times. September 24, 2019. Retrieved October 2, 2019.
Shear, Michael D.; Fandos, Nicholas (October 22, 2019). "Ukraine Envoy Testifies Trump Linked Military Aid to Investigations, Lawmaker Says". The New York Times. Retrieved October 22, 2019.
LaFraniere, Sharon (October 22, 2019). "6 Key Revelations of Taylor's Opening Statement to Impeachment Investigators". The New York Times. Retrieved October 23, 2019.
Siegel, Benjamin; Faulders, Katherine; Pecorin, Allison (December 13, 2019). "House Judiciary Committee passes articles of impeachment against President Trump". ABC News. Retrieved December 13, 2019.
Gregorian, Dareh (December 18, 2019). "Trump impeached by the House for abuse of power, obstruction of Congress". NBC News. Retrieved December 18, 2019.
Kim, Seung Min; Wagner, John; Demirjian, Karoun (January 23, 2020). "Democrats detail abuse-of-power charge against Trump as Republicans complain of repetitive arguments". The Washington Post. Retrieved January 27, 2020.
Shear, Michael D.; Fandos, Nicholas (January 18, 2020). "Trump's Defense Team Calls Impeachment Charges 'Brazen' as Democrats Make Legal Case". The New York Times. Retrieved January 30, 2020.
Herb, Jeremy; Mattingly, Phil; Raju, Manu; Fox, Lauren (January 31, 2020). "Senate impeachment trial: Wednesday acquittal vote scheduled after effort to have witnesses fails". CNN. Retrieved February 2, 2020.
Bookbinder, Noah (January 9, 2020). "The Senate has conducted 15 impeachment trials. It heard witnesses in every one". The Washington Post. Retrieved February 8, 2020.
Wilkie, Christina; Breuninger, Kevin (February 5, 2020). "Trump acquitted of both charges in Senate impeachment trial". CNBC. Retrieved February 2, 2021.
Baker, Peter (February 22, 2020). "Trump's Efforts to Remove the Disloyal Heightens Unease Across His Administration". The New York Times. Retrieved February 22, 2020.
Naylor, Brian (January 11, 2021). "Impeachment Resolution Cites Trump's 'Incitement' of Capitol Insurrection". NPR. Retrieved January 11, 2021.
Fandos, Nicholas (January 13, 2021). "Trump Impeached for Inciting Insurrection". The New York Times. Retrieved January 14, 2021.
Blake, Aaron (January 13, 2021). "Trump's second impeachment is the most bipartisan one in history". The Washington Post. Retrieved January 19, 2021.
Levine, Sam; Gambino, Lauren (February 13, 2021). "Donald Trump acquitted in impeachment trial". The Guardian. Retrieved February 13, 2021.
Fandos, Nicholas (February 13, 2021). "Trump Acquitted of Inciting Insurrection, Even as Bipartisan Majority Votes 'Guilty'". The New York Times. Retrieved February 14, 2021.
Watson, Kathryn; Quinn, Melissa; Segers, Grace; Becket, Stefan (February 10, 2021). "Senate finds Trump impeachment trial constitutional on first day of proceedings". CBS News. Retrieved February 18, 2021.
Morehouse, Lee (January 31, 2017). "Trump breaks precedent, files as candidate for re-election on first day". KTVK. Archived from the original on February 2, 2017. Retrieved February 19, 2017.
Graham, David A. (February 15, 2017). "Trump Kicks Off His 2020 Reelection Campaign on Saturday". The Atlantic. Retrieved February 19, 2017.
Martin, Jonathan; Burns, Alexander; Karni, Annie (August 24, 2020). "Nominating Trump, Republicans Rewrite His Record". The New York Times. Retrieved August 25, 2020.
Balcerzak, Ashley; Levinthal, Dave; Levine, Carrie; Kleiner, Sarah; Beachum, Lateshia (February 1, 2019). "Donald Trump's campaign cash machine: big, brawny and burning money". Center for Public Integrity. Retrieved October 8, 2021.
Goldmacher, Shane; Haberman, Maggie (September 7, 2020). "How Trump's Billion-Dollar Campaign Lost Its Cash Advantage". The New York Times. Retrieved October 8, 2021.
Egkolfopoulou, Misyrlena; Allison, Bill; Korte, Gregory (September 14, 2020). "Trump Campaign Slashes Ad Spending in Key States in Cash Crunch". Bloomberg News. Retrieved October 8, 2021.
Haberman, Maggie; Corasaniti, Nick; Karni, Annie (July 21, 2020). "As Trump Pushes into Portland, His Campaign Ads Turn Darker". The New York Times. Retrieved July 25, 2020.
Bump, Philip (August 28, 2020). "Nearly every claim Trump made about Biden's positions was false". The Washington Post. Retrieved October 9, 2021.
Dale, Daniel; Subramaniam, Tara; Lybrand, Holmes (August 31, 2020). "Fact check: Trump makes more false claims about Biden and protests". CNN. Retrieved October 9, 2021.
Hopkins, Dan (August 27, 2020). "Why Trump's Racist Appeals Might Be Less Effective In 2020 Than They Were In 2016". FiveThirtyEight. Retrieved May 28, 2021.
Kumar, Anita (August 8, 2020). "Trump aides exploring executive actions to curb voting by mail". Politico. Retrieved August 15, 2020.
Saul, Stephanie; Epstein, Reid J. (August 31, 2020). "Trump Is Pushing a False Argument on Vote-by-Mail Fraud. Here Are the Facts". The New York Times. Retrieved October 8, 2021.
Bogage, Jacob (August 12, 2020). "Trump says Postal Service needs money for mail-in voting, but he'll keep blocking funding". The Washington Post. Retrieved August 14, 2020.
Sonmez, Felicia (July 19, 2020). "Trump declines to say whether he will accept November election results". The Washington Post. Retrieved October 8, 2021.
Browne, Ryan; Starr, Barbara (September 25, 2020). "As Trump refuses to commit to a peaceful transition, Pentagon stresses it will play no role in the election". CNN. Retrieved October 8, 2021.
"Presidential Election Results: Biden Wins". The New York Times. December 11, 2020. Retrieved December 11, 2020.
"2020 US Presidential Election Results: Live Map". ABC News. December 10, 2020. Retrieved December 11, 2020.
Holder, Josh; Gabriel, Trip; Paz, Isabella Grullón (December 14, 2020). "Biden's 306 Electoral College Votes Make His Victory Official". The New York Times. Retrieved October 9, 2021.
"With results from key states unclear, Trump declares victory". Reuters. November 4, 2020. Retrieved November 10, 2020.
King, Ledyard (November 7, 2020). "Trump revives baseless claims of election fraud after Biden wins presidential race". USA Today. Retrieved November 7, 2020.
Helderman, Rosalind S.; Viebeck, Elise (December 12, 2020). "'The last wall': How dozens of judges across the political spectrum rejected Trump's efforts to overturn the election". The Washington Post. Retrieved October 9, 2021.
Blake, Aaron (December 14, 2020). "The most remarkable rebukes of Trump's legal case: From the judges he hand-picked". The Washington Post. Retrieved October 9, 2021.
Woodward, Calvin (November 16, 2020). "AP Fact Check: Trump conclusively lost, denies the evidence". AP News. Retrieved November 17, 2020.
Liptak, Adam (December 11, 2020). "Supreme Court Rejects Texas Suit Seeking to Subvert Election". The New York Times. Retrieved October 9, 2021.
Smith, David (November 21, 2020). "Trump's monumental sulk: president retreats from public eye as Covid ravages US". The Guardian. Retrieved October 9, 2021.
Lamire, Jonathan; Miller, Zeke (November 9, 2020). "Refusing to concede, Trump blocks cooperation on transition". AP News. Retrieved November 10, 2020.
Timm, Jane C.; Smith, Allan (November 14, 2020). "Trump is stonewalling Biden's transition. Here's why it matters". NBC News. Retrieved November 26, 2020.
Rein, Lisa (November 23, 2020). "Under pressure, Trump appointee Emily Murphy approves transition in unusually personal letter to Biden". The Washington Post. Retrieved November 24, 2020.
Naylor, Brian; Wise, Alana (November 23, 2020). "President-Elect Biden To Begin Formal Transition Process After Agency OK". NPR. Retrieved December 11, 2020.
Ordoñez, Franco; Rampton, Roberta (November 26, 2020). "Trump Is In No Mood To Concede, But Says Will Leave White House". NPR. Retrieved December 11, 2020.
Gardner, Amy (January 3, 2021). "'I just want to find 11,780 votes': In extraordinary hour-long call, Trump pressures Georgia secretary of state to recalculate the vote in his favor". The Washington Post. Retrieved January 20, 2021.
Kumar, Anita; Orr, Gabby; McGraw, Meridith (December 21, 2020). "Inside Trump's pressure campaign to overturn the election". Politico. Retrieved December 22, 2020.
Cohen, Marshall (November 5, 2021). "Timeline of the coup: How Trump tried to weaponize the Justice Department to overturn the 2020 election". CNN. Retrieved November 6, 2021.
Haberman, Maggie; Karni, Annie (January 5, 2021). "Pence Said to Have Told Trump He Lacks Power to Change Election Result". The New York Times. Retrieved January 7, 2021.
Fausset, Richard; Hakim, Danny (February 10, 2021). "Georgia Prosecutors Open Criminal Inquiry Into Trump's Efforts to Subvert Election". The New York Times. Retrieved February 11, 2021.
Haberman, Maggie (January 20, 2021). "Trump Departs Vowing, 'We Will Be Back in Some Form'". The New York Times. Retrieved January 25, 2021.
Arkin, William M. (December 24, 2020). "Exclusive: Donald Trump's martial-law talk has military on red alert". Newsweek. Retrieved September 15, 2021.
Gangel, Jamie; Herb, Jeremy; Cohen, Marshall; Stuart, Elizabeth; Starr, Barbara (July 14, 2021). "'They're not going to f**king succeed': Top generals feared Trump would attempt a coup after election, according to new book". CNN. Retrieved September 15, 2021.
Breuninger, Kevin (July 15, 2021). "Top U.S. Gen. Mark Milley feared Trump would attempt a coup after his loss to Biden, new book says". CNBC. Retrieved September 15, 2021.
Gangel, Jamie; Herb, Jeremy; Stuart, Elizabeth (September 14, 2021). "Woodward/Costa book: Worried Trump could 'go rogue,' Milley took top-secret action to protect nuclear weapons". CNN. Retrieved September 15, 2021.
Schmidt, Michael S. (September 14, 2021). "Fears That Trump Might Launch a Strike Prompted General to Reassure China, Book Says". The New York Times. Retrieved September 15, 2021.
Savage, Charlie (January 10, 2021). "Incitement to Riot? What Trump Told Supporters Before Mob Stormed Capitol". The New York Times. Retrieved January 11, 2021.
"Donald Trump Speech 'Save America' Rally Transcript January 6". Rev. January 6, 2021. Retrieved January 8, 2021.
Tan, Shelley; Shin, Youjin; Rindler, Danielle (January 9, 2021). "How one of America's ugliest days unraveled inside and outside the Capitol". The Washington Post. Retrieved May 2, 2021.
Panetta, Grace; Lahut, Jake; Zavarise, Isabella; Frias, Lauren (December 21, 2022). "A timeline of what Trump was doing as his MAGA mob attacked the US Capitol on Jan. 6". Business Insider. Retrieved June 1, 2023.
Gregorian, Dareh; Gibson, Ginger; Kapur, Sahil; Helsel, Phil (January 6, 2021). "Congress confirms Biden's win after pro-Trump mob's assault on Capitol". NBC News. Retrieved January 8, 2021.
Rubin, Olivia; Mallin, Alexander; Steakin, Will (January 4, 2022). "By the numbers: How the Jan. 6 investigation is shaping up 1 year later". ABC News. Retrieved June 4, 2023.
Cameron, Chris (January 5, 2022). "These Are the People Who Died in Connection With the Capitol Riot". The New York Times. Retrieved January 29, 2022.
Terkel, Amanda (May 11, 2023). "Trump says he would pardon a 'large portion' of Jan. 6 rioters". NBC. Retrieved June 3, 2023.
Wolfe, Jan (January 27, 2021). "Explainer: Why Trump's post-presidency perks, like a pension and office, are safe for the rest of his life". Reuters. Retrieved February 2, 2021.
Quinn, Melissa (January 27, 2021). "Trump opens 'Office of the Former President' in Florida". CBS News. Retrieved February 2, 2021.
Spencer, Terry (January 28, 2021). "Palm Beach considers options as Trump remains at Mar-a-Lago". AP News. Retrieved February 2, 2021.
Durkee, Allison (May 7, 2021). "Trump Can Legally Live At Mar-A-Lago, Palm Beach Says". Forbes. Retrieved March 7, 2024.
Solender, Andrew (May 3, 2021). "Trump Says He'll Appropriate 'The Big Lie' To Refer To His Election Loss". Forbes. Retrieved October 10, 2021.
Wolf, Zachary B. (May 19, 2021). "The 5 key elements of Trump's Big Lie and how it came to be". CNN. Retrieved October 10, 2021.
Balz, Dan (May 29, 2021). "The GOP push to revisit 2020 has worrisome implications for future elections". The Washington Post. Retrieved June 18, 2021.
Bender, Michael C.; Epstein, Reid J. (July 20, 2022). "Trump Recently Urged a Powerful Legislator to Overturn His 2020 Defeat in Wisconsin". The New York Times. Retrieved August 13, 2022.
Goldmacher, Shane (April 17, 2022). "Mar-a-Lago Machine: Trump as a Modern-Day Party Boss". The New York Times. Retrieved July 31, 2022.
Paybarah, Azi (August 2, 2022). "Where Trump's Endorsement Record Stands Halfway through Primary Season". The New York Times. Retrieved August 3, 2022.
Castleman, Terry; Mason, Melanie (August 5, 2022). "Tracking Trump's endorsement record in the 2022 primary elections". Los Angeles Times. Retrieved August 6, 2022.
Lyons, Kim (December 6, 2021). "SEC investigating Trump SPAC deal to take his social media platform public". The Verge. Retrieved December 30, 2021.
"Trump Media & Technology Group Corp". Bloomberg News. Retrieved December 30, 2021.
Harwell, Drew (March 26, 2024). "Trump Media soars in first day of public tradings". The Washington Post. Retrieved March 28, 2024.
Bhuyian, Johana (February 21, 2022). "Donald Trump's social media app launches on Apple store". The Guardian. Retrieved May 7, 2023.
Lowell, Hugo (March 15, 2023). "Federal investigators examined Trump Media for possible money laundering, sources say". The Guardian. Retrieved April 5, 2023.
Durkee, Alison (March 15, 2023). "Trump's Media Company Reportedly Under Federal Investigation For Money Laundering Linked To Russia". Forbes. Retrieved March 15, 2023.
Roebuck, Jeremy (May 30, 2024). "Donald Trump conviction: Will he go to prison? Can he still run for president? What happens now?". Philadelphia Inquirer. Retrieved June 1, 2024.
Sisak, Michael R. (May 30, 2024). "Trump Investigations". AP News. Retrieved June 1, 2024.
"Keeping Track of the Trump Criminal Cases". The New York Times. May 30, 2024. Retrieved June 1, 2024.
Lybrand, Holmes; Cohen, Marshall; Rabinowitz, Hannah (August 12, 2022). "Timeline: The Justice Department criminal inquiry into Trump taking classified documents to Mar-a-Lago". CNN. Retrieved August 14, 2022.
Montague, Zach; McCarthy, Lauren (August 9, 2022). "The Timeline Related to the F.B.I.'s Search of Mar-a-Lago". The New York Times. Retrieved August 14, 2022.
Haberman, Maggie; Thrush, Glenn (August 13, 2022). "Trump Lawyer Told Justice Dept. That Classified Material Had Been Returned". The New York Times. Retrieved August 14, 2022.
Barrett, Devlin; Dawsey, Josh (August 12, 2022). "Agents at Trump's Mar-a-Lago seized 11 sets of classified documents, court filing shows". The Washington Post. Retrieved August 12, 2022.
Haberman, Maggie; Thrush, Glenn; Savage, Charlie (August 12, 2022). "Files Seized From Trump Are Part of Espionage Act Inquiry". The New York Times. Retrieved August 13, 2022.
Barrett, Devlin; Dawsey, Josh; Stein, Perry; Harris, Shane (August 12, 2022). "FBI searched Trump's home to look for nuclear documents and other items, sources say". The Washington Post. Retrieved August 12, 2022.
Swan, Betsy; Cheney, Kyle; Wu, Nicholas (August 12, 2022). "FBI search warrant shows Trump under investigation for potential obstruction of justice, Espionage Act violations". Politico. Retrieved August 12, 2022.
Thrush, Glenn; Savage, Charlie; Haberman, Maggie; Feuer, Alan (November 18, 2022). "Garland Names Special Counsel for Trump Inquiries". The New York Times. Retrieved November 19, 2022.
Tucker, Eric; Balsamo, Michael (November 18, 2022). "Garland names special counsel to lead Trump-related probes". AP News. Retrieved November 19, 2022.
Feuer, Alan (December 19, 2022). "It's Unclear Whether the Justice Dept. Will Take Up the Jan. 6 Panel's Charges". The New York Times. Retrieved March 25, 2023.
Scannell, Kara; del Valle, Lauren (December 6, 2022). "Trump Organization found guilty on all counts of criminal tax fraud". CNN. Retrieved September 3, 2024.
Sisek, Michael R. (January 13, 2023). "Trump Organization fined $1.6 million for tax fraud". AP News. Retrieved September 3, 2024.
Lowell, Hugo; Wicker, Jewel (August 15, 2023). "Donald Trump and allies indicted in Georgia over bid to reverse 2020 election loss". The Guardian. Retrieved December 22, 2023.
Drenon, Brandon (August 25, 2023). "What are the charges in Trump's Georgia indictment?". BBC News. Retrieved December 22, 2023.
Pereira, Ivan; Barr, Luke (August 25, 2023). "Trump mug shot released by Fulton County Sheriff's Office". ABC News. Retrieved August 25, 2023.
Rabinowitz, Hannah (August 31, 2023). "Trump pleads not guilty in Georgia election subversion case". CNN. Retrieved August 31, 2023.
Bailey, Holly (March 13, 2024). "Georgia judge dismisses six charges in Trump election interference case". The Washington Post. Retrieved March 14, 2024.
Barrett, Devlin; Dawsey, Josh; Stein, Perry; Alemany, Jacqueline (June 9, 2023). "Trump Put National Secrets at Risk, Prosecutors Say in Historic Indictment". The Washington Post. Retrieved June 10, 2023.
Greve, Joan E.; Lowell, Hugo (June 14, 2023). "Trump pleads not guilty to 37 federal criminal counts in Mar-a-Lago case". The Guardian. Retrieved June 14, 2023.
Schonfeld, Zach (July 28, 2023). "5 revelations from new Trump charges". The Hill. Retrieved August 4, 2023.
Savage, Charlie (June 9, 2023). "A Trump-Appointed Judge Who Showed Him Favor Gets the Documents Case". The New York Times.
Tucker, Eric (July 15, 2024). "Federal judge dismisses Trump classified documents case over concerns with prosecutor's appointment". AP News. Retrieved July 15, 2024.
Mallin, Alexander (August 26, 2024). "Prosecutors Appeal Dismissal of Trump Documents Case". The New York Times. Retrieved August 27, 2024.
Barrett, Devlin; Hsu, Spencer S.; Stein, Perry; Dawsey, Josh; Alemany, Jacqueline (August 2, 2023). "Trump charged in probe of Jan. 6, efforts to overturn 2020 election". The Washington Post. Retrieved August 2, 2023.
Sneed, Tierney; Rabinowitz, Hannah; Polantz, Katelyn; Lybrand, Holmes (August 3, 2023). "Donald Trump pleads not guilty to January 6-related charges". CNN. Retrieved August 3, 2023.
Stein, Perry; Hsu, Spencer S. (November 25, 2024). "With D.C. case dismissed, Trump is no longer under federal indictment". The Washington Post. Retrieved November 26, 2024.
Halpert, Madeline (November 26, 2024). "Special counsel's last criminal case against Trump dismissed". Retrieved November 30, 2024.
Ellison, Sarah; Farhi, Paul (December 12, 2018). "Publisher of the National Enquirer admits to hush-money payments made on Trump's behalf". The Washington Post. Retrieved January 17, 2021.
Bump, Philip (August 21, 2018). "How the campaign finance charges against Michael Cohen implicate Trump". The Washington Post. Retrieved July 25, 2019.
Neumeister, Larry; Hays, Tom (August 22, 2018). "Cohen pleads guilty, implicates Trump in hush-money scheme". AP News. Retrieved October 7, 2021.
Nelson, Louis (March 7, 2018). "White House on Stormy Daniels: Trump 'denied all these allegations'". Politico. Retrieved March 16, 2018.
Singman, Brooke (August 22, 2018). "Trump insists he learned of Michael Cohen payments 'later on', in 'Fox & Friends' exclusive". Fox News. Retrieved August 23, 2018.
Barrett, Devlin; Zapotosky, Matt (December 7, 2018). "Court filings directly implicate Trump in efforts to buy women's silence, reveal new contact between inner circle and Russian". The Washington Post. Retrieved December 7, 2018.
Allen, Jonathan; Stempel, Jonathan (July 18, 2019). "FBI documents point to Trump role in hush money for porn star Daniels". Reuters. Retrieved July 22, 2019.
Mustian, Jim (July 19, 2019). "Records detail frenetic effort to bury stories about Trump". AP News. Retrieved July 22, 2019.
Mustian, Jim (July 19, 2019). "Why no hush-money charges against Trump? Feds are silent". AP News. Retrieved October 7, 2021.
Harding, Luke; Holpuch, Amanda (May 19, 2021). "New York attorney general opens criminal investigation into Trump Organization". The Guardian. Retrieved May 19, 2021.
Protess, Ben; Rashbaum, William K. (August 1, 2019). "Manhattan D.A. Subpoenas Trump Organization Over Stormy Daniels Hush Money". The New York Times. Retrieved August 2, 2019.
Rashbaum, William K.; Protess, Ben (September 16, 2019). "8 Years of Trump Tax Returns Are Subpoenaed by Manhattan D.A." The New York Times. Retrieved October 7, 2021.
Barrett, Devlin (May 29, 2024). "Jurors must be unanimous to convict Trump, can disagree on underlying crimes". The Washington Post. Retrieved June 15, 2024.
Scannell, Kara; Miller, John; Herb, Jeremy; Cole, Devan (March 31, 2023). "Donald Trump indicted by Manhattan grand jury on 34 counts related to fraud". CNN. Retrieved April 1, 2023.
Marimow, Ann E. (April 4, 2023). "Here are the 34 charges against Trump and what they mean". The Washington Post. Retrieved April 5, 2023.
Reiss, Adam; Grumbach, Gary; Gregorian, Dareh; Winter, Tom; Frankel, Jillian (May 30, 2024). "Donald Trump found guilty in historic New York hush money case". NBC News. Retrieved May 31, 2024.
Bagchi, Aysha (December 18, 2024). "Trump lost on immunity. What's next for his New York hush money conviction?". USA Today. Retrieved December 19, 2024.
Scannell, Kara (September 21, 2022). "New York attorney general files civil fraud lawsuit against Trump, some of his children and his business". CNN. Retrieved September 21, 2022.
Katersky, Aaron (February 14, 2023). "Court upholds fine imposed on Trump over his failure to comply with subpoena". ABC News. Retrieved April 8, 2024.
Bromwich, Jonah E.; Protess, Ben; Rashbaum, William K. (August 10, 2022). "Trump Invokes Fifth Amendment, Attacking Legal System as Troubles Mount". The New York Times. Retrieved August 11, 2011.
Kates, Graham (September 26, 2023). "Donald Trump and his company "repeatedly" violated fraud law, New York judge rules". CBS News.
Bromwich, Jonah E.; Protess, Ben (February 17, 2024). "Trump Fraud Trial Penalty Will Exceed $450 Million". The New York Times. Retrieved February 17, 2024.
Sullivan, Becky; Bernstein, Andrea; Marritz, Ilya; Lawrence, Quil (May 9, 2023). "A jury finds Trump liable for battery and defamation in E. Jean Carroll trial". NPR. Retrieved May 10, 2023.
Orden, Erica (July 19, 2023). "Trump loses bid for new trial in E. Jean Carroll case". Politico. Retrieved August 13, 2023.
Scannell, Kara (August 7, 2023). "Judge dismisses Trump's defamation lawsuit against Carroll for statements she made on CNN". CNN. Retrieved August 7, 2023.
Reiss, Adam; Gregorian, Dareh (August 7, 2023). "Judge tosses Trump's counterclaim against E. Jean Carroll, finding rape claim is 'substantially true'". NBC News. Retrieved August 13, 2023.
Stempel, Jonathan (August 10, 2023). "Trump appeals dismissal of defamation claim against E. Jean Carroll". Reuters. Retrieved August 17, 2023.
Kates, Graham (March 8, 2024). "Trump posts $91 million bond to appeal E. Jean Carroll defamation verdict". CBS News. Retrieved April 8, 2024.
Arnsdorf, Isaac; Scherer, Michael (November 15, 2022). "Trump, who as president fomented an insurrection, says he is running again". The Washington Post. Retrieved December 5, 2022.
Schouten, Fredreka (November 16, 2022). "Questions about Donald Trump's campaign money, answered". CNN. Retrieved December 5, 2022.
Goldmacher, Shane; Haberman, Maggie (June 25, 2023). "As Legal Fees Mount, Trump Steers Donations Into PAC That Has Covered Them". The New York Times. Retrieved June 25, 2023.
Escobar, Molly Cook; Sun, Albert; Goldmacher, Shane (March 27, 2024). "How Trump Moved Money to Pay $100 Million in Legal Bills". The New York Times. Retrieved April 3, 2024.
Levine, Sam (March 4, 2024). "Trump was wrongly removed from Colorado ballot, US supreme court rules". The Guardian. Retrieved June 23, 2024.
Yourish, Karen; Smart, Charlie (May 24, 2024). "Trump's Pattern of Sowing Election Doubt Intensifies in 2024". The New York Times. Retrieved August 30, 2024.
Bender, Michael C.; Gold, Michael (November 20, 2023). "Trump's Dire Words Raise New Fears About His Authoritarian Bent". The New York Times.
Stone, Peter (November 22, 2023). "'Openly authoritarian campaign': Trump's threats of revenge fuel alarm". The Guardian.
Colvin, Jill; Barrow, Bill (December 7, 2023). "Trump's vow to only be a dictator on 'day one' follows growing worry over his authoritarian rhetoric". AP News.
LeVine, Marianne (November 12, 2023). "Trump calls political enemies 'vermin', echoing dictators Hitler, Mussolini". The Washington Post.
Levine, Sam (November 10, 2023). "Trump suggests he would use FBI to go after political rivals if elected in 2024". The Guardian.
Vazquez, Maegan (November 10, 2023). "Trump says on Univision he could weaponize FBI, DOJ against his enemies". The Washington Post.
Stracqualursi, Veronica (October 14, 2024). "Trump suggests using military against 'enemy from within' on Election Day". CNN.
Lerer, Lisa; Gold, Michael (October 15, 2024). "Trump Escalates Threats to Political Opponents He Deems the 'Enemy'". The New York Times.
Gold, Michael; Huynh, Anjali (April 2, 2024). "Trump Again Invokes 'Blood Bath' and Dehumanizes Migrants in Border Remarks". The New York Times. Retrieved April 3, 2024.
Savage, Charlie; Haberman, Maggie; Swan, Jonathan (November 11, 2023). "Sweeping Raids, Giant Camps and Mass Deportations: Inside Trump's 2025 Immigration Plans". The New York Times.
Layne, Nathan; Slattery, Gram; Reid, Tim (April 3, 2024). "Trump calls migrants 'animals', intensifying focus on illegal immigration". Reuters. Retrieved April 3, 2024.
Philbrick, Ian Prasad; Bentahar, Lyna (December 5, 2023). "Donald Trump's 2024 Campaign, in His Own Menacing Words". The New York Times. Retrieved May 10, 2024.
Basu, Zachary (November 13, 2023). "Trump campaign defends "vermin" speech amid fascist comparisons". Axios (website). Retrieved December 8, 2023.
Browning, Christopher R. (July 25, 2023). "A New Kind of Fascism". The Atlantic. Retrieved December 8, 2023.
Kim, Soo Rin; Ibssa, Lalee (November 13, 2023). "Trump compares political opponents to 'vermin' who he will 'root out', alarming historians". ABC News. Retrieved December 8, 2023.
Ward, Myah (October 12, 2024). "We watched 20 Trump rallies. His racist, anti-immigrant messaging is getting darker". Politico. Retrieved October 12, 2024.
Homans, Charles (April 27, 2024). "Donald Trump Has Never Sounded Like This". The New York Times Magazine. Retrieved April 27, 2024.
Applebaum, Anne (October 18, 2024). "Trump Is Speaking Like Hitler, Stalin, and Mussolini". The Atlantic. Retrieved October 18, 2024.
Baker, Peter; Freedman, Dylan (October 6, 2024). "Trump's Speeches, Increasingly Angry and Rambling, Reignite the Question of Age". The New York Times. Retrieved November 6, 2024.
Lane, Nathan; Ulmer, Alexandra (May 16, 2024). "Trump, allies are laying the groundwork to contest potential election loss". Reuters. Retrieved September 12, 2024.
Browne, Malachy; Lum, Devon; Cardia, Alexander (July 26, 2024). "Speculation Swirls About What Hit Trump. An Analysis Suggests It was a Bullet". The New York Times. Retrieved July 29, 2024.
Hutchinson, Bill; Cohen, Miles (July 16, 2024). "Gunman opened fire at Trump rally as witnesses say they tried to alert police". ABC News. Retrieved July 17, 2024.
"AP PHOTOS: Shooting at Trump rally in Pennsylvania". AP News. July 14, 2024. Retrieved July 23, 2024.
Astor, Maggie (July 15, 2024). "What to Know About J.D. Vance, Trump's Running Mate". The New York Times. Retrieved July 15, 2024.
Debusmann, Bernd Jr; Yousif, Nadine (September 23, 2024). "Suspect described Trump 'assassination attempt' in pre-written note". BBC News. Retrieved November 21, 2024.
"2024 Presidential Election Results". AP News. November 25, 2024. Retrieved November 25, 2024.
Treisman, Rachel (November 4, 2024). "Trump is hoping to win non-consecutive terms. Only one president has done it". NPR. Retrieved November 9, 2024.
Sheerin, Jude; Murphy, Matt (November 6, 2024). "Trump pulls off historic White House comeback". BBC. Retrieved November 9, 2024. "Projections suggest he is likely to win the overall popular vote nationally"
Miller, Zeke; Price, Michelle L.; Weissert, Will; Colvin, Jill (November 6, 2024). "Trump wins the White House in political comeback rooted in appeals to frustrated voters". AP News. Retrieved November 9, 2024. "an extraordinary comeback for a former president"
Datta, Monti (September 16, 2019). "3 countries where Trump is popular". The Conversation. Retrieved October 3, 2021.
"Rating World Leaders: 2018 The U.S. vs. Germany, China and Russia". Gallup. Retrieved October 3, 2021. Page 9
Wike, Richard; Fetterolf, Janell; Mordecai, Mara (September 15, 2020). "U.S. Image Plummets Internationally as Most Say Country Has Handled Coronavirus Badly". Pew Research Center. Retrieved December 24, 2020.
Jones, Jeffrey M. (January 18, 2021). "Last Trump Job Approval 34%; Average Is Record-Low 41%". Gallup. Retrieved October 3, 2021.
Klein, Ezra (September 2, 2020). "Can anything change Americans' minds about Donald Trump? The eerie stability of Trump's approval rating, explained". Vox. Retrieved October 10, 2021.
Enten, Harry (January 16, 2021). "Trump finishes with worst first term approval rating ever". CNN. Retrieved October 3, 2021.
"Most Admired Man and Woman". Gallup. December 28, 2006. Retrieved October 3, 2021.
Budryk, Zack (December 29, 2020). "Trump ends Obama's 12-year run as most admired man: Gallup". The Hill. Retrieved December 31, 2020.
Bach, Natash (December 28, 2017). "Trump Is the Only Elected U.S. President Not to Be Named America's Most Admired Man In His First Year". Fortune (magazine). Retrieved November 19, 2024.
Balk, Tim (December 12, 2024). "Donald Trump Is Time Magazine's Person of the Year". The New York Times. Retrieved December 12, 2024.
"Presidential Historians Survey 2021". C-SPAN. Retrieved June 30, 2021.
Sheehey, Maeve (June 30, 2021). "Trump debuts at 41st in C-SPAN presidential rankings". Politico. Retrieved March 31, 2023.
Brockell, Gillian (June 30, 2021). "Historians just ranked the presidents. Trump wasn't last". The Washington Post. Retrieved July 1, 2021.
"American Presidents: Greatest and Worst". Siena College Research Institute. June 22, 2022. Retrieved July 11, 2022.
Rottinghaus, Brandon; Vaughn, Justin S. (February 19, 2018). "Opinion: How Does Trump Stack Up Against the Best—and Worst—Presidents?". The New York Times. Retrieved July 13, 2024.
Chappell, Bill (February 19, 2024). "In historians' Presidents Day survey, Biden vs. Trump is not a close call". NPR.
O'Brien, Timothy L. (November 1, 2024). "The Peculiarly American Roots of Trumpism". Bloomberg News. Retrieved November 26, 2024.
Stuckey 2020, p. 366.
Jacobson 2020, p. 25.
Mason, Wronski & Kane 2021.
Sundahl 2022, "[In] a model for distinguishing between popularity and personality cults based on three parameters covering a representational and social practice dimension... Trump and Putin belong in the domain of personality cults".
Franks & Hesami 2021, "Results of the current study... may lend credence to accusations that some Trump supporters have a cult-like loyalty to the 45th president".
Adams 2021, p. 256.
Reyes 2020, p. 869.
Diamond 2023, p. 96, "The cult of Trumpism fosters and exploits paranoia and allegiance to an all-powerful, charismatic figure, contributing to a social milieu at risk for the erosion of democratic principles and the rise of fascism".
Hassan 2019, p. xviii, "...Trump employs many of the same techniques as prominent cult leaders".
Ben-Ghiat, Ruth (December 19, 2020). "Op-Ed: Trump's formula for building a lasting personality cult". Los Angeles Times. Retrieved October 4, 2023.
"The Republican Party has lurched towards populism and illiberalism". The Economist. October 31, 2020. Retrieved October 14, 2021.
Borger, Julian (October 26, 2021). "Republicans closely resemble autocratic parties in Hungary and Turkey – study". The Guardian. Retrieved October 14, 2021.
Chotiner, Isaac (July 29, 2021). "Redefining Populism". The New Yorker. Retrieved October 14, 2021.
Noah, Timothy (July 26, 2015). "Will the real Donald Trump please stand up?". Politico. Retrieved October 1, 2021.
Bierman, Noah (August 22, 2016). "Donald Trump helps bring far-right media's edgier elements into the mainstream". Los Angeles Times. Retrieved October 7, 2021.
Weigel, David (August 20, 2016). "'Racialists' are cheered by Trump's latest strategy". The Washington Post. Retrieved June 23, 2018.
Krieg, Gregory (August 25, 2016). "Clinton is attacking the 'Alt-Right' – What is it?". CNN. Retrieved August 25, 2016.
Pierce, Matt (September 20, 2020). "Q&A: What is President Trump's relationship with far-right and white supremacist groups?". Los Angeles Times. Retrieved October 7, 2021.
Perry, Whitehead & Grubbs 2021, p. 229.
Peter, Smith (May 18, 2024). "Jesus is their savior, Trump is their candidate. Ex-president's backers say he shares faith, values". AP News. Retrieved November 23, 2024.
Kunzelman, Michael; Galvan, Astrid (August 7, 2019). "Trump words linked to more hate crime? Some experts think so". AP News. Retrieved October 7, 2021.
Feinberg, Ayal; Branton, Regina; Martinez-Ebers, Valerie (March 22, 2019). "Analysis | Counties that hosted a 2016 Trump rally saw a 226 percent increase in hate crimes". The Washington Post. Retrieved October 7, 2021.
White, Daniel (February 1, 2016). "Donald Trump Tells Crowd To 'Knock the Crap Out Of' Hecklers". Time. Retrieved August 9, 2019.
Koerner, Claudia (October 18, 2018). "Trump Thinks It's Totally Cool That A Congressman Assaulted A Journalist For Asking A Question". BuzzFeed News. Retrieved October 19, 2018.
Tracy, Abigail (August 8, 2019). ""The President of the United States Says It's Okay": The Rise of the Trump Defense". Vanity Fair. Retrieved October 7, 2021.
Helderman, Rosalind S.; Hsu, Spencer S.; Weiner, Rachel (January 16, 2021). "'Trump said to do so': Accounts of rioters who say the president spurred them to rush the Capitol could be pivotal testimony". The Washington Post. Retrieved September 27, 2021.
Levine, Mike (May 30, 2020). "'No Blame?' ABC News finds 54 cases invoking 'Trump' in connection with violence, threats, alleged assaults". ABC News. Retrieved February 4, 2021.
Kessler, Glenn; Kelly, Meg; Rizzo, Salvador; Lee, Michelle Ye Hee (January 20, 2021). "In four years, President Trump made 30,573 false or misleading claims". The Washington Post. Retrieved October 11, 2021.
Dale, Daniel (June 5, 2019). "Donald Trump has now said more than 5,000 false things as president". Toronto Star. Retrieved October 11, 2021.
Dale, Daniel; Subramiam, Tara (March 9, 2020). "Fact check: Donald Trump made 115 false claims in the last two weeks of February". CNN. Retrieved November 1, 2023.
Finnegan, Michael (September 25, 2016). "Scope of Trump's falsehoods unprecedented for a modern presidential candidate". Los Angeles Times. Retrieved October 10, 2021.
Glasser, Susan B. (August 3, 2018). "It's True: Trump Is Lying More, and He's Doing It on Purpose". The New Yorker. Retrieved January 10, 2019.
Konnikova, Maria (January 20, 2017). "Trump's Lies vs. Your Brain". Politico. Retrieved March 31, 2018.
Kakutani, Michiko (2018). "The Firehose of Falsehood: Propaganda and Fake News". The Death of Truth: Notes on Falsehood in the Age of Trump. Crown/Archetype. pp. 94–104. ISBN 9780525574842.
Kessler, Glenn; Kelly, Meg; Rizzo, Salvador; Shapiro, Leslie; Dominguez, Leo (January 23, 2021). "A term of untruths: The longer Trump was president, the more frequently he made false or misleading claims". The Washington Post. Retrieved October 11, 2021.
Qiu, Linda (January 21, 2017). "Donald Trump had biggest inaugural crowd ever? Metrics don't show it". PolitiFact. Retrieved March 30, 2018.
Rein, Lisa (March 6, 2017). "Here are the photos that show Obama's inauguration crowd was bigger than Trump's". The Washington Post. Retrieved March 8, 2017.
Wong, Julia Carrie (April 7, 2020). "Hydroxychloroquine: how an unproven drug became Trump's coronavirus 'miracle cure'". The Guardian. Retrieved June 25, 2021.
Spring, Marianna (May 27, 2020). "Coronavirus: The human cost of virus misinformation". BBC News. Retrieved June 13, 2020.
Rowland, Christopher (March 23, 2020). "As Trump touts an unproven coronavirus treatment, supplies evaporate for patients who need those drugs". The Washington Post. Retrieved March 24, 2020.
Parkinson, Joe; Gauthier-Villars, David (March 23, 2020). "Trump Claim That Malaria Drugs Treat Coronavirus Sparks Warnings, Shortages". The Wall Street Journal. Retrieved March 26, 2020.
Zurcher, Anthony (November 29, 2017). "Trump's anti-Muslim retweet fits a pattern". BBC News. Retrieved June 13, 2020.
Siders, David (May 25, 2020). "Trump sees a 'rigged election' ahead. Democrats see a constitutional crisis in the making". Politico. Retrieved October 9, 2021.
Riccardi, Nicholas (September 17, 2020). "AP Fact Check: Trump's big distortions on mail-in voting". AP News. Retrieved October 7, 2020.
Lipton, Eric; Sanger, David E.; Haberman, Maggie; Shear, Michael D.; Mazzetti, Mark; Barnes, Julian E. (April 11, 2020). "He Could Have Seen What Was Coming: Behind Trump's Failure on the Virus". The New York Times. Retrieved April 11, 2020.
Guynn, Jessica (October 5, 2020). "From COVID-19 to voting: Trump is nation's single largest spreader of disinformation, studies say". USA Today. Retrieved October 7, 2020.
Bergengruen, Vera; Hennigan, W.J. (October 6, 2020). "'You're Gonna Beat It.' How Donald Trump's COVID-19 Battle Has Only Fueled Misinformation". Time. Retrieved October 7, 2020.
Allen, Jonathan (December 31, 2018). "Does being President Trump still mean never having to say you're sorry?". NBC News. Retrieved June 14, 2020.
Greenberg, David (January 28, 2017). "The Perils of Calling Trump a Liar". Politico. Retrieved June 13, 2020.
Bauder, David (August 29, 2018). "News media hesitate to use 'lie' for Trump's misstatements". AP News. Retrieved September 27, 2023.
Farhi, Paul (June 5, 2019). "Lies? The news media is starting to describe Trump's 'falsehoods' that way". The Washington Post. Retrieved April 11, 2024.
Fichera, Angelo; Spencer, Saranac Hale (October 20, 2020). "Trump's Long History With Conspiracy Theories". FactCheck.org. Retrieved September 15, 2021.
Subramaniam, Tara; Lybrand, Holmes (October 15, 2020). "Fact-checking the dangerous bin Laden conspiracy theory that Trump touted". CNN. Retrieved October 11, 2021.
Haberman, Maggie (February 29, 2016). "Even as He Rises, Donald Trump Entertains Conspiracy Theories". The New York Times. Retrieved October 11, 2021.
Bump, Philip (November 26, 2019). "President Trump loves conspiracy theories. Has he ever been right?". The Washington Post. Retrieved October 11, 2021.
Reston, Maeve (July 2, 2020). "The Conspiracy-Theorist-in-Chief clears the way for fringe candidates to become mainstream". CNN. Retrieved October 11, 2021.
Perkins, Tom (November 18, 2020). "The dead voter conspiracy theory peddled by Trump voters, debunked". The Guardian. Retrieved October 11, 2021.
Cohen, Li (January 15, 2021). "6 conspiracy theories about the 2020 election – debunked". CBS News. Retrieved September 13, 2021.
McEvoy, Jemima (December 17, 2020). "These Are The Voter Fraud Claims Trump Tried (And Failed) To Overturn The Election With". Forbes. Retrieved September 13, 2021.
Conger, Kate; Isaac, Mike (January 16, 2021). "Inside Twitter's Decision to Cut Off Trump". The New York Times. Retrieved October 10, 2021.
Madhani, Aamer; Colvin, Jill (January 9, 2021). "A farewell to @realDonaldTrump, gone after 57,000 tweets". AP News. Retrieved October 10, 2021.
Landers, Elizabeth (June 6, 2017). "White House: Trump's tweets are 'official statements'". CNN. Retrieved October 10, 2021.
Dwoskin, Elizabeth (May 27, 2020). "Twitter labels Trump's tweets with a fact check for the first time". The Washington Post. Retrieved July 7, 2020.
Dwoskin, Elizabeth (May 27, 2020). "Trump lashes out at social media companies after Twitter labels tweets with fact checks". The Washington Post. Retrieved May 28, 2020.
Fischer, Sara; Gold, Ashley (January 11, 2021). "All the platforms that have banned or restricted Trump so far". Axios. Retrieved January 16, 2021.
Timberg, Craig (January 14, 2021). "Twitter ban reveals that tech companies held keys to Trump's power all along". The Washington Post. Retrieved February 17, 2021.
Alba, Davey; Koeze, Ella; Silver, Jacob (June 7, 2021). "What Happened When Trump Was Banned on Social Media". The New York Times. Retrieved December 21, 2023.
Dwoskin, Elizabeth; Timberg, Craig (January 16, 2021). "Misinformation dropped dramatically the week after Twitter banned Trump and some allies". The Washington Post. Retrieved February 17, 2021.
Harwell, Drew; Dawsey, Josh (November 7, 2022). "Trump once reconsidered sticking with Truth Social. Now he's stuck". The Washington Post. Retrieved May 7, 2023.
Mac, Ryan; Browning, Kellen (November 19, 2022). "Elon Musk Reinstates Trump's Twitter Account". The New York Times. Retrieved November 21, 2022.
Dang, Sheila; Coster, Helen (November 20, 2022). "Trump snubs Twitter after Musk announces reactivation of ex-president's account". Reuters. Retrieved May 10, 2024.
Bond, Shannon (January 23, 2023). "Meta allows Donald Trump back on Facebook and Instagram". NPR.
Egan, Matt (March 11, 2024). "Trump calls Facebook the enemy of the people. Meta's stock sinks". CNN.
Parnes, Amie (April 28, 2018). "Trump's love-hate relationship with the press". The Hill. Retrieved July 4, 2018.
Chozick, Amy (September 29, 2018). "Why Trump Will Win a Second Term". The New York Times. Retrieved September 22, 2019.
Hetherington, Marc; Ladd, Jonathan M. (May 1, 2020). "Destroying trust in the media, science, and government has left America vulnerable to disaster". Brookings Institution. Retrieved October 11, 2021.
Thomsen, Jacqueline (May 22, 2018). "'60 Minutes' correspondent: Trump said he attacks the press so no one believes negative coverage". The Hill. Retrieved May 23, 2018.
Stelter, Brian; Collins, Kaitlan (May 9, 2018). "Trump's latest shot at the press corps: 'Take away credentials?'". CNN Money. Retrieved November 22, 2024.
Grynbaum, Michael M. (December 30, 2019). "After Another Year of Trump Attacks, 'Ominous Signs' for the American Press". The New York Times. Retrieved October 11, 2021.
Geltzer, Joshua A.; Katyal, Neal K. (March 11, 2020). "The True Danger of the Trump Campaign's Defamation Lawsuits". The Atlantic. Retrieved October 1, 2020.
"US judge throws out Donald Trump's lawsuit against New York Times". The Guardian. May 3, 2023. Retrieved November 25, 2024.
Kapur, Sahil (October 13, 2024). "'Totally illegal': Trump escalates rhetoric on outlawing political dissent and criticism". NBC News. Retrieved November 23, 2024.

    Folkenflik, David (October 21, 2024). "Could Trump's threats against news outlets carry weight if he wins the presidency?". NPR. Retrieved November 23, 2024.

Works cited
Books

    Blair, Gwenda (2015) [2001]. The Trumps: Three Generations That Built an Empire. Simon & Schuster. ISBN 978-1-5011-3936-9.
    Buettner, Russ; Craig, Susanne (2024). Lucky Loser: How Donald Trump Squandered His Father's Fortune And Created The Illusion of Success. Penguin Press. ISBN 978-0-593-29864-0.
    D'Antonio, Michael (2015). Never enough : Donald Trump and the pursuit of success. Thomas Dunne Books, St. Martin's Press. ISBN 978-1-250-04238-5.
    Haberman, Maggie (2022). Confidence man: the making of Donald Trump and the breaking of America. Penguin Press. ISBN 978-0-593-29734-6.
    Hassan, Steven (2019). The Cult of Trump. Simon & Schuster. ISBN 978-1-9821-2733-6.
    Johnston, David Cay (2016). The Making of Donald Trump. Melville House Publishing. ISBN 978-1-61219-658-9.
    Johnston, David Cay (2021). The Big Cheat: How Donald Trump Fleeced America And Enriched Himself And His Family. Simon & Schuster. ISBN 978-1-9821-7804-8.
    Kranish, Michael; Fisher, Marc (2017) [2016]. Trump Revealed: The Definitive Biography of the 45th President. Simon & Schuster. ISBN 978-1-5011-5652-6.
    O'Brien, Timothy L. (2005). TrumpNation: The Art of Being The Donald. Warner Books. ISBN 978-0-446-57854-7.
    Meacham, Jon (2016). Destiny and Power: The American Odyssey of George Herbert Walker Bush. Random House. p. 326. ISBN 978-0-8129-7947-3.
    O'Donnell, John R.; Rutherford, James (1991). Trumped!. Crossroad Press Trade Edition. ISBN 978-1-946025-26-5.

​​​​​


Journals

    Adams, Kenneth Alan (Spring 2021). "The Trump Death Cult". Journal of Psychohistory. 48 (4): 256–276. ISSN 0145-3378. Retrieved November 6, 2024.
    Diamond, Michael J. (February 22, 2023). "Perverted Containment: Trumpism, Cult Creation, and the Rise of Destructive American Populism". Psychoanalytic Inquiry. 43 (2): 96–109. doi:10.1080/07351690.2023.2163147.
    Edwards, Jason A. (2018). "Make America Great Again: Donald Trump and Redefining the U.S. Role in the World". Communication Quarterly. 66 (2): 176. doi:10.1080/01463373.2018.1438485.
    Franks, Andrew S.; Hesami, Farhang (September 18, 2021). "Seeking Evidence of The MAGA Cult and Trump Derangement Syndrome: An Examination of (A)symmetric Political Bias". Societies. 11 (3): 113. doi:10.3390/soc11030113.
    Holshue, Michelle L.; DeBolt, Chas; Lindquist, Scott; Lofy, Kathy H.; Wiesman, John; Bruce, Hollianne; Spitters, Christopher; Ericson, Keith; Wilkerson, Sara; Tural, Ahmet; Diaz, George (March 5, 2020). "First Case of 2019 Novel Coronavirus in the United States". The New England Journal of Medicine. 382 (10): 929–936. doi:10.1056/NEJMoa2001191. PMC 7092802. PMID 32004427.
    Jacobson, Gary C. (October 24, 2020). "Donald Trump and the Parties: Impeachment, Pandemic, Protest, and Electoral Politics in 2020". Presidential Studies Quarterly. 50 (4): 762–795. doi:10.1111/psq.12682. ISSN 0360-4918.
    Johnson, Kevin R. (2017a). "Immigration and civil rights in the Trump administration: Law and policy making by executive order". Santa Clara Law Review. 57 (3): 611–665.
    Johnson, Kevin R.; Cuison-Villazor, Rose (May 2, 2019). "The Trump Administration and the War on Immigration Diversity". Wake Forest Law Review. 54 (2): 575–616.
    Mason, Liliana; Wronski, Julie; Kane, John V. (2021). "Activating Animus: The Uniquely Social Roots of Trump Support". American Political Science Review. 115 (4): 1508–1516. doi:10.1017/S0003055421000563.
    McGurk, Brett (January 22, 2020). "The Cost of an Incoherent Foreign Policy: Trump's Iran Imbroglio Undermines U.S. Priorities Everywhere Else". Foreign Affairs.
    Lajevardi, Nazita; Oskooii, Kassra A. R. (2018). "Old-Fashioned Racism, Contemporary Islamophobia, and the Isolation of Muslim Americans in the Age of Trump". Journal of Race, Ethnicity, and Politics. 3 (1): 112–152. doi:10.1017/rep.2017.37.
    Perry, Samuel L.; Whitehead, Andrew L.; Grubbs, Joshua B. (April 21, 2021). "The Devil That You Know: Christian Nationalism and Intent to Change One's Voting Behavior For or Against Trump in 2020". Politics and Religion. 15 (2): 229–246. doi:10.1017/S175504832100002X. hdl:11244/334967.
    Reyes, Antonio (May 4, 2020). "I, Trump The cult of personality, anti-intellectualism and the Post-Truth era". Journal of Language and Politics. 19 (6): 869–892. doi:10.1075/jlp.20002.rey. ISSN 1569-2159.
    Rothe, Dawn L.; Collins, Victoria E. (November 17, 2019). "Turning Back the Clock? Violence against Women and the Trump Administration". Victims & Offenders. 14 (8): 965–978. doi:10.1080/15564886.2019.1671284.
    Stuckey, Mary E. (February 20, 2020). "'The Power of the Presidency to Hurt': The Indecorous Rhetoric of Donald J. Trump and the Rhetorical Norms of Democracy". Presidential Studies Quarterly. 50 (2): 366–391. doi:10.1111/psq.12641.
    Sundahl, Anne-Mette Holmgård (May 4, 2022). "Personality Cult or a Mere Matter of Popularity?". International Journal of Politics, Culture, and Society. 36 (4): 431–458. doi:10.1007/s10767-022-09423-0. PMC 9066393. PMID 35528318.

External links
Library resources about
Donald Trump

    Resources in your library
    Resources in other libraries

By Donald Trump

    Resources in your library
    Resources in other libraries

    Archive of Donald Trump's tweets
    Appearances on C-SPAN Edit this at Wikidata
    Donald Trump at IMDb Edit this at Wikidata
    Donald Trump on the Internet Archive
    Biography of Donald Trump from The White House website (courtesy of the White House Historical Association)

    vte

Donald Trump

    vte

Presidents of the United States
Offices and distinctions

    vte

(← 2012) 2016 United States presidential election (2020 →)

    vte

(← 2016) 2020 United States presidential election (2024 →)

    vte

(← 2020) 2024 United States presidential election (2028 →)

    vte

Republican Party

    vte

Time Persons of the Year

    vte

Businesses of Donald Trump

    vte

Media by and about Donald Trump
Donald Trump at Wikipedia's sister projects:

    Definitions from Wiktionary
    Media from Commons
    News from Wikinews
    Quotations from Wikiquote
    Texts from Wikisource
    Textbooks from Wikibooks
    Resources from Wikiversity
    Data from Wikidata

Portals:

    icon 1980s
    icon 1990s
    2000s
    icon 2010s
    icon 2020s
    Biography
    icon Business
    Conservatism
    virus icon COVID-19
    flag Florida
    flag New York (state)
    flag New York City
    icon Politics
    icon Television
    flag United States

Authority control databases Edit this at Wikidata
Categories:

    Donald Trump1946 birthsLiving peopleTrumpism20th-century American businesspeople20th-century American non-fiction writers21st-century American businesspeople21st-century American criminals21st-century American non-fiction writers21st-century Florida politicians21st-century New York (state) politicians21st-century presidents of the United StatesAmerican billionairesAmerican businesspeople convicted of crimesAmerican businesspeople in real estateAmerican casino industry businesspeopleAmerican chairpersons of corporationsAmerican conspiracy theoristsAmerican hoteliersAmerican Internet company foundersAmerican male criminalsAmerican nationalistsAmerican people of German descentAmerican people of Scottish descentAmerican people of the War in Afghanistan (2001–2021)American politicians convicted of fraudAmerican reality television producersAmerican rhetoriciansAmerican shooting survivorsAmerican television hostsAttempted assassination of Donald Trump in FloridaAttempted assassination of Donald Trump in PennsylvaniaBusinesspeople from Queens, New YorkCandidates in the 2000 United States presidential electionCandidates in the 2016 United States presidential electionCandidates in the 2020 United States presidential electionCandidates in the 2024 United States presidential electionCOVID-19 conspiracy theoristsCriminals from ManhattanFlorida politicians convicted of crimesFlorida RepublicansIg Nobel laureatesIlleistsImpeached presidents of the United StatesJanuary 6 United States Capitol attackNew York (state) independentsNew York (state) politicians convicted of crimesNew York (state) RepublicansNew York Military Academy alumniPeople associated with the January 6 United States Capitol attackPeople charged with racketeeringPeople from Jamaica, QueensPeople stripped of honorary degreesPresidents of the United StatesReform Party of the United States of America politiciansRepublican Party (United States) presidential nomineesRepublican Party presidents of the United StatesRight-wing populists in the United StatesThe State of Georgia v. Donald J. Trump, et al. defendantsTelevision personalities from New York CityTelevision producers from Queens, New YorkTime Person of the YearTrump familyThe Trump Organization employeesTrump–Ukraine scandalUnited States Football League executivesWharton School alumniWWE Hall of Fame inductees

    This page was last edited on 20 December 2024
'''

topics = ['The Wall with Mexico']
start = time.time()
print(check_long_text(text, topics))
end = time.time()

print(end - start)